# Paragon Geeks — Transaction-Level Revenue & Operations Analysis

## Project Overview

This project presents a **full end-to-end analytics workflow** built on real transaction data from *Paragon Geeks*, a multi-location electronics repair and retail business. The objective is to transform raw point-of-sale exports into **audit-safe, decision-ready insights** that reflect real-world business complexity.

Unlike academic datasets, this analysis intentionally retains **messy, incomplete, and undocumented transactions** to preserve financial accuracy and operational realism.

---

## Business Objectives

The primary goals of this analysis are to:

- Reconcile **total revenue across documented and undocumented transactions**
- Identify **core revenue drivers** by repair category, device type, brand, and model
- Evaluate **documentation coverage and data quality patterns**
- Analyze **pricing structure, transaction value tiers, and operational mix**
- Produce **clean analytical outputs** suitable for dashboards, reporting, and AI systems

---

## Data Sources

- Square POS transaction exports  
  - **2024 Sales Data**
  - **2025 Sales Data**

These files include mixed-format text fields, partial documentation, refunds, bundled services, and retail-only transactions.

---

## Analytical Approach

This notebook follows a **production-style analytics pipeline**, including:

1. Raw data ingestion and normalization  
2. Semantic text cleaning and transaction classification  
3. Separation of documented vs. undocumented revenue  
4. KPI construction aligned with business operations  
5. Audit-safe aggregation and validation  
6. Executive-ready summary outputs  

All transformations are **explicit, reproducible, and traceable**.

---

## Key Design Principles

- **Revenue truth comes first** — no data discarded to inflate cleanliness  
- **Documentation ≠ validity** — undocumented revenue is still real revenue  
- **Business logic over academic shortcuts**  
- **Explainability at every stage**  
- **Outputs designed for BI, SQL, SAS, and AI reuse**

---

## Tools & Skills Demonstrated

- Python (Pandas, NumPy)
- Text normalization & rule-based classification
- KPI and metric design
- Business analytics & operational analysis
- Data validation and reconciliation
- Portfolio-ready data engineering practices

---

## Expected Outputs

By the end of this notebook, the analysis produces:

- Transaction-level analytics tables
- Revenue and documentation coverage metrics
- Repair, device, brand, and pricing insights
- Time-based performance analysis
- Exportable datasets for visualization and modeling

---

*The analysis begins below.*


In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)


In [2]:
path_2024 = r"C:\Users\kevin\OneDrive\Desktop\raw data\square data\2024 sales.csv"
path_2025 = r"C:\Users\kevin\OneDrive\Desktop\raw data\square data\2025 sales.csv"

df_2024 = pd.read_csv(path_2024)
df_2025 = pd.read_csv(path_2025)

df_2024["source_year"] = 2024
df_2025["source_year"] = 2025

df_raw = pd.concat([df_2024, df_2025], ignore_index=True)

print("RAW SHAPE:", df_raw.shape)


RAW SHAPE: (1922, 69)


In [3]:
def build_text_raw(row):
    if row["source_year"] == 2024:
        return f"{row.get('Item','')} {row.get('Notes','')}"
    else:
        return f"{row.get('Description','')}"

df = df_raw.copy()
df["text_raw"] = df.apply(build_text_raw, axis=1)


In [4]:
def clean_text(x):
    if pd.isna(x):
        return ""
    x = str(x).lower()
    x = re.sub(r"custom amount\s*-\s*", "", x)
    x = re.sub(r"custom amount", "", x)
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

df["text_clean"] = df["text_raw"].apply(clean_text)


In [5]:
def parse_net_sales(x):
    if pd.isna(x):
        return 0.0
    
    x = str(x).strip()
    
    # Handle accounting negatives: (345.00) → -345.00
    if x.startswith("(") and x.endswith(")"):
        x = "-" + x[1:-1]
    
    # Remove currency formatting
    x = x.replace("$", "").replace(",", "")
    
    try:
        return float(x)
    except ValueError:
        return 0.0


In [6]:
df["net_sales_num"] = df["Net Sales"].apply(parse_net_sales)


In [7]:
print(df["net_sales_num"].describe())

print("\nNegative values check:")
print((df["net_sales_num"] < 0).sum())

print("\nSample negative rows:")
display(
    df[df["net_sales_num"] < 0][
        ["source_year", "Net Sales", "net_sales_num", "text_clean"]
    ].head(10)
)


count    1922.000000
mean      136.794537
std       131.323088
min      -345.000000
25%        80.000000
50%       120.000000
75%       165.000000
max      3000.000000
Name: net_sales_num, dtype: float64

Negative values check:
8

Sample negative rows:


,source_year,Net Sales,net_sales_num,text_clean
531,2024,($345.00),-345.0,s23 ultra screen repair canceled order
579,2024,($60.00),-60.0,replacement service pixel 6 pro defective part
807,2025,-$125.00,-125.0,
1367,2025,-$30.00,-30.0,
1412,2025,-$30.00,-30.0,
1560,2025,-$320.00,-320.0,
1620,2025,-$220.00,-220.0,
1837,2025,-$150.00,-150.0,


In [8]:
# ======================================================================================
# PARAGON GEEKS — POS ANALYTICS PROJECT
# PHASE 3: DOCUMENTATION CLASSIFICATION (LOCKED)
# ======================================================================================

# -------------------------------
# Documentation flag
# -------------------------------
df["documentation_flag"] = df["text_clean"].apply(
    lambda x: "documented" if x.strip() != "" else "undocumented"
)

# -------------------------------
# Validation counts
# -------------------------------
print("\n==============================")
print("DOCUMENTATION CLASSIFICATION")
print("==============================")

doc_counts = df["documentation_flag"].value_counts()

print("\nDocumentation counts:")
print(doc_counts)

print("\nDocumentation percentages:")
print((doc_counts / len(df)).round(3))

# -------------------------------
# Cross-check by year
# -------------------------------
print("\nBy source_year:")
print(
    df.groupby(["source_year", "documentation_flag"])
      .size()
      .unstack(fill_value=0)
)

# -------------------------------
# Sample undocumented rows
# -------------------------------
print("\nSample UNDOCUMENTED rows:")
display(
    df[df["documentation_flag"] == "undocumented"]
    [["source_year", "text_raw", "text_clean", "Net Sales", "net_sales_num"]]
    .sample(10, random_state=42)
)

# -------------------------------
# Sample documented rows
# -------------------------------
print("\nSample DOCUMENTED rows:")
display(
    df[df["documentation_flag"] == "documented"]
    [["source_year", "text_raw", "text_clean", "Net Sales", "net_sales_num"]]
    .sample(10, random_state=42)
)



DOCUMENTATION CLASSIFICATION

Documentation counts:
documentation_flag
documented      1172
undocumented     750
Name: count, dtype: int64

Documentation percentages:
documentation_flag
documented      0.61
undocumented    0.39
Name: count, dtype: float64

By source_year:
documentation_flag  documented  undocumented
source_year                                 
2024                       654             0
2025                       518           750

Sample UNDOCUMENTED rows:


,source_year,text_raw,text_clean,Net Sales,net_sales_num
1585,2025,Custom Amount,,$120.00,120.0
1327,2025,Custom Amount,,$20.00,20.0
957,2025,Custom Amount,,$50.00,50.0
1110,2025,Custom Amount,,$160.00,160.0
1212,2025,Custom Amount,,$120.00,120.0
1812,2025,Custom Amount,,$160.00,160.0
1297,2025,Custom Amount,,$130.00,130.0
985,2025,Custom Amount,,$180.00,180.0
1616,2025,Custom Amount,,$160.00,160.0
1892,2025,Custom Amount,,$120.00,120.0



Sample DOCUMENTED rows:


,source_year,text_raw,text_clean,Net Sales,net_sales_num
626,2024,Custom Amount nan,nan,$100.00,100.00
220,2024,15 Pro max Screen Replacement nan,15 pro max screen replacement nan,$270.00,270.00
683,2025,Custom Amount - 14 pro max screen repair,14 pro max screen repair,$130.00,130.00
1235,2025,Custom Amount - iPad 10 screen repair,ipad 10 screen repair,$120.00,120.00
174,2024,Custom Amount iPad air repair,ipad air repair,$110.62,110.62
1421,2025,Custom Amount - iPhone 15 back glass,iphone 15 back glass,$125.00,125.00
731,2025,Custom Amount - iPhone 13 pro battery replacem...,iphone 13 pro battery replacement,$80.00,80.00
988,2025,"Custom Amount - battery, Custom Amount - a54 b...",battery a54 battery,$68.29,68.29
58,2024,Custom Amount nan,nan,$100.00,100.00
240,2024,Custom Amount iPad 8gen screen repair,ipad 8gen screen repair,$108.00,108.00


In [9]:
# HARD RESET — destroy prior text state
df = df_raw.copy()

df.drop(columns=[c for c in df.columns if c.startswith("text_")], errors="ignore", inplace=True)

print("Text columns after reset:")
print([c for c in df.columns if c.startswith("text_")])


Text columns after reset:
[]


In [10]:
df["text_raw"] = ""

mask_2024 = df["source_year"] == 2024
mask_2025 = df["source_year"] == 2025

df.loc[mask_2024, "text_raw"] = (
    df.loc[mask_2024, "Item"].fillna("") + " " +
    df.loc[mask_2024, "Notes"].fillna("")
)

df.loc[mask_2025, "text_raw"] = df.loc[mask_2025, "Description"].fillna("")


In [11]:
display(
    df.loc[mask_2024, ["Item", "Notes", "text_raw"]].sample(10, random_state=1)
)


,Item,Notes,text_raw
488,iPhone 11 screen repair,NaN,iPhone 11 screen repair
443,15 pro max screen repair,NaN,15 pro max screen repair
595,iPad Air 5 screen repair,NaN,iPad Air 5 screen repair
574,12 pro max speaker replacement,NaN,12 pro max speaker replacement
41,Custom Amount,NaN,Custom Amount
601,Custom Amount,NaN,Custom Amount
573,iPhone 11 screen repair,NaN,iPhone 11 screen repair
248,Custom Amount,SE Screen Repair,Custom Amount SE Screen Repair
520,IPhone SE,NaN,IPhone SE
23,Custom Amount,NaN,Custom Amount


In [12]:
import re

def clean_text(x):
    if pd.isna(x):
        return ""
    x = str(x).lower()
    x = re.sub(r"custom amount\s*-\s*", "", x)
    x = re.sub(r"custom amount", "", x)
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

df["text_clean"] = df["text_raw"].apply(clean_text)


In [13]:
print("Empty text_clean by year:")
print(
    df.groupby("source_year")["text_clean"]
      .apply(lambda s: (s == "").sum())
)


Empty text_clean by year:
source_year
2024    203
2025    751
Name: text_clean, dtype: int64


In [14]:
display(
    df.loc[
        (df["source_year"] == 2024) & (df["text_clean"] != ""),
        ["Item", "Notes", "text_raw", "text_clean"]
    ].head(20)
)


,Item,Notes,text_raw,text_clean
0,Custom Amount,iPhone 8 Plus front and back 2x,Custom Amount iPhone 8 Plus front and back 2x,iphone 8 plus front and back 2x
6,Custom Amount,pixel 7 screen repair,Custom Amount pixel 7 screen repair,pixel 7 screen repair
7,Custom Amount,a53 screen replacement,Custom Amount a53 screen replacement,a53 screen replacement
15,Custom Amount,iPhone 11 screen repair -mobile\nChristmas eve,Custom Amount iPhone 11 screen repair -mobile\...,iphone 11 screen repair mobile christmas eve
17,Custom Amount,imac 2022 with ram upgrade,Custom Amount imac 2022 with ram upgrade,imac 2022 with ram upgrade
18,Custom Amount,iPhone 12 Pro screen repairs (minor imperfecti...,Custom Amount iPhone 12 Pro screen repairs (mi...,iphone 12 pro screen repairs minor imperfectio...
19,Custom Amount,iPhone 14 back glass replacement,Custom Amount iPhone 14 back glass replacement,iphone 14 back glass replacement
22,Custom Amount,15 pro screen repair \n,Custom Amount 15 pro screen repair \n,15 pro screen repair
26,Custom Amount,iPhone 13 screen replacement (replaced screen ...,Custom Amount iPhone 13 screen replacement (re...,iphone 13 screen replacement replaced screen f...
29,Custom Amount,15 Promax back glass replacement,Custom Amount 15 Promax back glass replacement,15 promax back glass replacement


In [15]:
df["documentation_flag"] = df["text_clean"].apply(
    lambda x: "documented" if x != "" else "undocumented"
)

print(df["documentation_flag"].value_counts())

print(
    df.groupby(["source_year", "documentation_flag"])
      .size()
      .unstack(fill_value=0)
)


documentation_flag
documented      968
undocumented    954
Name: count, dtype: int64
documentation_flag  documented  undocumented
source_year                                 
2024                       451           203
2025                       517           751


In [16]:
df_doc = df[df["documentation_flag"] == "documented"].copy()

print(df_doc.shape)


(968, 72)


cleaned and seperated docmented and undocmented 

In [17]:
df_doc = df[df["documentation_flag"] == "documented"].copy()


In [18]:
from collections import Counter

all_words = (
    df_doc["text_clean"]
    .str.split()
    .explode()
)

word_freq = Counter(all_words)


In [19]:
pd.DataFrame(word_freq.most_common(40), columns=["token", "count"])


,token,count
0,screen,614
1,repair,572
2,iphone,327
3,pro,194
4,replacement,169
5,ipad,121
6,14,100
7,max,98
8,13,97
9,11,93


In [20]:
pd.DataFrame(word_freq.most_common()[-40:], columns=["token", "count"])


,token,count
0,reminder,1
1,97,1
2,lg,1
3,2nd,1
4,wiring,1
5,9pro,1
6,mexico,1
7,kamado,1
8,vape,1
9,chip,1


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    ngram_range=(2, 3),
    min_df=5,
    stop_words=None
)

ngrams = cv.fit_transform(df_doc["text_clean"])
ngram_freq = ngrams.sum(axis=0).A1

ngram_df = pd.DataFrame(
    zip(cv.get_feature_names_out(), ngram_freq),
    columns=["ngram", "count"]
).sort_values("count", ascending=False)

ngram_df.head(30)


,ngram,count
161,screen repair,495
133,pro max,92
80,iphone 11,69
111,max screen,64
88,iphone 13,63
135,pro max screen,62
164,screen replacement,55
112,max screen repair,54
168,ultra screen,53
4,11 screen,50


In [22]:
df_doc[df_doc["text_clean"].str.contains("back", na=False)][
    ["text_clean"]
].sample(20, random_state=42)


,text_clean
123,iphone xr front and back
725,15 pro max front and back
225,back glass replacement
0,iphone 8 plus front and back 2x
1044,14 pro max back front and battery replacement
1119,15 pro max screen battery back glass ps5 scree...
1571,iphone 14 promax battery and backglass
1260,iphone 16 plus back glass replacement
1142,14 pro max screen repair original 14 pro max b...
634,iphone 13 pro max back glass replacement


In [23]:
df_doc = df[df["documentation_flag"] == "documented"].copy()

print(df_doc.shape)


(968, 72)


In [24]:
REPAIR_TOKENS = {
    "screen": ["screen", "lcd", "display"],
    "back_glass": ["back glass", "backglass", "rear glass"],
    "battery": ["battery"],
    "charging_port": ["charging port", "charge port", "port cleaning"],
    "camera": ["camera", "lens"],
    "speaker_mic": ["speaker", "mic", "microphone"],
    "water_damage": ["water damage", "liquid damage"],
    "board_level": ["motherboard", "logic board", "board repair"],
    "diagnostic": ["diagnostic", "diagnosis", "inspection"],
    "software_setup": ["setup", "data transfer", "restore", "update"]
}


In [25]:
def token_hit(text, tokens):
    return any(tok in text for tok in tokens)

for repair, tokens in REPAIR_TOKENS.items():
    df_doc[f"repair_{repair}"] = df_doc["text_clean"].apply(
        lambda x: token_hit(x, tokens)
    )


In [26]:
repair_flag_cols = [c for c in df_doc.columns if c.startswith("repair_")]

df_doc["repair_event_count"] = df_doc[repair_flag_cols].sum(axis=1)


In [27]:
def parse_net_sales(x):
    if pd.isna(x):
        return 0.0

    x = str(x).strip()

    # Accounting-style negatives: (345.00) → -345.00
    if x.startswith("(") and x.endswith(")"):
        x = "-" + x[1:-1]

    # Handle leading minus sign cases: -$125.00
    x = x.replace("$", "").replace(",", "")

    try:
        return float(x)
    except ValueError:
        return 0.0


In [28]:
df["net_sales_num"] = df["Net Sales"].apply(parse_net_sales)


In [29]:
print("net_sales_num in df:", "net_sales_num" in df.columns)
df["net_sales_num"].describe()


net_sales_num in df: True


count    1922.000000
mean      136.794537
std       131.323088
min      -345.000000
25%        80.000000
50%       120.000000
75%       165.000000
max      3000.000000
Name: net_sales_num, dtype: float64

In [30]:
df_doc = df[df["documentation_flag"] == "documented"].copy()

print("net_sales_num in df_doc:", "net_sales_num" in df_doc.columns)
print(df_doc.shape)


net_sales_num in df_doc: True
(968, 73)


In [31]:
REPAIR_TOKENS = {
    "screen": ["screen", "lcd", "display"],
    "back_glass": ["back glass", "backglass", "rear glass"],
    "battery": ["battery"],
    "charging_port": ["charging port", "charge port", "port cleaning"],
    "camera": ["camera", "lens"],
    "speaker_mic": ["speaker", "mic", "microphone"],
    "water_damage": ["water damage", "liquid damage"],
    "board_level": ["motherboard", "logic board", "board repair"],
    "diagnostic": ["diagnostic", "diagnosis", "inspection"],
    "software_setup": ["setup", "data transfer", "restore", "update"]
}


In [32]:
def token_hit(text, tokens):
    return any(tok in text for tok in tokens)

for repair, tokens in REPAIR_TOKENS.items():
    df_doc[f"repair_{repair}"] = df_doc["text_clean"].apply(
        lambda x: token_hit(x, tokens)
    )


In [33]:
repair_flag_cols = [c for c in df_doc.columns if c.startswith("repair_")]

df_doc["repair_event_count"] = df_doc[repair_flag_cols].sum(axis=1)


In [34]:
print("repair_event_count in df_doc:", "repair_event_count" in df_doc.columns)
print(df_doc["repair_event_count"].value_counts().sort_index())


repair_event_count in df_doc: True
repair_event_count
0    196
1    733
2     32
3      5
4      2
Name: count, dtype: int64


In [35]:
display(
    df_doc[df_doc["repair_event_count"] == 0]
    [["text_clean", "net_sales_num"]]
    .head(15)
)


,text_clean,net_sales_num
0,iphone 8 plus front and back 2x,200.00
17,imac 2022 with ram upgrade,3000.00
35,airpod pro 2 case,15.00
123,iphone xr front and back,194.00
135,14 pro max,230.00
136,repair,110.00
141,port replacement,110.00
147,13 port,100.00
153,iphone xs max,130.00
156,14 pro port replacement,120.00


In [36]:
df_doc["device_only_flag"] = (
    (df_doc["repair_event_count"] == 0) &
    (df_doc["text_clean"].str.contains(
        r"iphone|ipad|imac|macbook|watch|galaxy|pixel|ipad|tablet|airpod|airpods",
        regex=True
    ))
)


In [37]:
print(df_doc["device_only_flag"].value_counts())

display(
    df_doc[df_doc["device_only_flag"]]
    [["text_clean", "net_sales_num"]]
    .head(15)
)


device_only_flag
False    879
True      89
Name: count, dtype: int64


,text_clean,net_sales_num
0,iphone 8 plus front and back 2x,200.00
17,imac 2022 with ram upgrade,3000.00
35,airpod pro 2 case,15.00
123,iphone xr front and back,194.00
153,iphone xs max,130.00
174,ipad air repair,110.62
184,apple watch se 44mm,150.00
189,tablet install,50.00
210,iphone 12 promax,130.00
211,ipad 13 cable reset,175.00


In [38]:
def classify_device_type(text):
    if re.search(r"iphone|galaxy|pixel|phone|promax|ultra|plus|xr|xs", text):
        return "phone"
    if re.search(r"ipad|tablet", text):
        return "tablet"
    if re.search(r"imac|macbook|laptop|pc|computer", text):
        return "computer"
    if re.search(r"ps5|ps4|playstation|xbox|nintendo|switch", text):
        return "console"
    if re.search(r"watch|apple watch", text):
        return "watch"
    if re.search(r"airpod|airpods|case|protector|charger|cable", text):
        return "accessory"
    if re.search(r"install|setup|diagnostic|repair$", text):
        return "non-device"
    return "unknown"


In [39]:
df_doc["device_type"] = df_doc["text_clean"].apply(classify_device_type)


In [40]:
def classify_brand(text):
    if re.search(r"iphone|ipad|macbook|imac|airpod|airpods|apple watch", text):
        return "Apple"
    if re.search(r"samsung|galaxy|s20|s21|s22|s23|s24|note", text):
        return "Samsung"
    if re.search(r"pixel|google", text):
        return "Google"
    if re.search(r"sony|playstation|ps5|ps4", text):
        return "Sony"
    if re.search(r"nintendo|switch", text):
        return "Nintendo"
    if re.search(r"pc|windows", text):
        return "PC"
    return "Unknown"


In [41]:
df_doc["brand"] = df_doc["text_clean"].apply(classify_brand)


In [42]:
print(df_doc["device_type"].value_counts())

print(df_doc["brand"].value_counts())

display(
    df_doc[
        ["text_clean", "device_type", "brand", "repair_event_count", "device_only_flag"]
    ].sample(20, random_state=42)
)


device_type
phone         491
non-device    157
unknown       142
tablet        114
computer       28
accessory      24
watch           6
console         6
Name: count, dtype: int64
brand
Apple      460
Unknown    349
Samsung    123
Google      27
PC           5
Sony         4
Name: count, dtype: int64


,text_clean,device_type,brand,repair_event_count,device_only_flag
812,iphone 14 promax and case,phone,Apple,0,True
439,s22 ultra screen repair,phone,Samsung,1,False
587,ipad 12 9 4th gen repair,tablet,Apple,0,True
705,14 pro max screen repair,non-device,Unknown,1,False
167,14 plus screen repair,phone,Unknown,1,False
1693,s21 ultra screen repair,phone,Samsung,1,False
1376,iphone xr screen repair,phone,Apple,1,False
694,iphone 11 and ipad 9 screen repair,phone,Apple,1,False
189,tablet install,tablet,Unknown,0,True
768,14 plus back glass,phone,Unknown,1,False


In [43]:
def classify_device_type(text):
    # Phones FIRST (most specific)
    if re.search(r"iphone|galaxy|pixel|promax|pro max|ultra|plus|xr|xs|\b\d{2}\b", text):
        return "phone"

    # Tablets
    if re.search(r"ipad|tablet", text):
        return "tablet"

    # Computers
    if re.search(r"imac|macbook|laptop|pc|computer", text):
        return "computer"

    # Consoles
    if re.search(r"ps5|ps4|playstation|xbox|nintendo|switch", text):
        return "console"

    # Watches
    if re.search(r"watch|apple watch", text):
        return "watch"

    # Accessories
    if re.search(r"airpod|airpods|case|protector|charger|cable", text):
        return "accessory"

    # Non-device services LAST
    if re.search(r"install|setup|diagnostic", text):
        return "non-device"

    return "unknown"


In [44]:
df_doc["device_type"] = df_doc["text_clean"].apply(classify_device_type)


In [45]:
def classify_brand(text):
    # Apple explicit
    if re.search(r"iphone|ipad|macbook|imac|airpod|airpods|apple watch", text):
        return "Apple"

    # Apple implicit iPhone models (14, 15, etc.)
    if re.search(r"\b(1[0-9]|20)\b\s*(pro|max|plus)?", text):
        return "Apple"

    if re.search(r"samsung|galaxy|s20|s21|s22|s23|s24|note", text):
        return "Samsung"

    if re.search(r"pixel|google", text):
        return "Google"

    if re.search(r"sony|playstation|ps5|ps4", text):
        return "Sony"

    if re.search(r"nintendo|switch", text):
        return "Nintendo"

    if re.search(r"pc|windows", text):
        return "PC"

    return "Unknown"


In [46]:
df_doc["brand"] = df_doc["text_clean"].apply(classify_brand)


In [47]:
print(df_doc["device_type"].value_counts())
print(df_doc["brand"].value_counts())

display(
    df_doc[
        ["text_clean", "device_type", "brand", "repair_event_count", "device_only_flag"]
    ].sample(25, random_state=42)
)


device_type
phone         655
unknown       155
tablet         92
computer       23
accessory      22
non-device     11
watch           5
console         5
Name: count, dtype: int64
brand
Apple      637
Unknown    187
Samsung    109
Google      27
PC           5
Sony         3
Name: count, dtype: int64


,text_clean,device_type,brand,repair_event_count,device_only_flag
812,iphone 14 promax and case,phone,Apple,0,True
439,s22 ultra screen repair,phone,Samsung,1,False
587,ipad 12 9 4th gen repair,phone,Apple,0,True
705,14 pro max screen repair,phone,Apple,1,False
167,14 plus screen repair,phone,Apple,1,False
1693,s21 ultra screen repair,phone,Samsung,1,False
1376,iphone xr screen repair,phone,Apple,1,False
694,iphone 11 and ipad 9 screen repair,phone,Apple,1,False
189,tablet install,tablet,Unknown,0,True
768,14 plus back glass,phone,Apple,1,False


In [48]:
tx_agg = (
    df_doc
    .groupby("Transaction ID")
    .agg(
        revenue=("net_sales_num", "sum"),
        rows=("Transaction ID", "size"),
        repair_events=("repair_event_count", "sum"),
        max_repairs_in_row=("repair_event_count", "max"),
        device_only_rows=("device_only_flag", "sum"),
        has_repair=("repair_event_count", lambda x: (x > 0).any()),
        has_device_only=("device_only_flag", lambda x: (x > 0).any()),
        device_count=("device_type", lambda x: x.nunique()),
        primary_device_type=("device_type", lambda x: x.mode().iloc[0] if not x.mode().empty else "unknown"),
        primary_brand=("brand", lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown"),
    )
    .reset_index()
)


In [49]:
tx_agg["multi_repair_flag"] = tx_agg["repair_events"] > 1
tx_agg["multi_device_flag"] = tx_agg["device_count"] > 1


In [50]:
print("Transaction rows:", tx_agg.shape[0])
print("Total revenue:", round(tx_agg["revenue"].sum(), 2))

print("\nMulti-repair transactions:")
print(tx_agg["multi_repair_flag"].value_counts())

print("\nMulti-device transactions:")
print(tx_agg["multi_device_flag"].value_counts())

display(tx_agg.sample(15, random_state=42))


Transaction rows: 933
Total revenue: 155265.6

Multi-repair transactions:
multi_repair_flag
False    871
True      62
Name: count, dtype: int64

Multi-device transactions:
multi_device_flag
False    913
True      20
Name: count, dtype: int64


,Transaction ID,revenue,rows,repair_events,max_repairs_in_row,device_only_rows,has_repair,has_device_only,device_count,primary_device_type,primary_brand,multi_repair_flag,multi_device_flag
830,tn0c63dtE1BGr3drlkfhn6LcX9YZY,185.00,1,1,1,0,True,False,1,phone,Apple,False,False
70,3sEmTH0WXEl4GekvdBfCYqnqrkVZY,100.00,1,1,1,0,True,False,1,phone,Apple,False,False
631,h9YBTHCoOKPUkPV1Lw4P1ok4rPZZY,0.01,1,1,1,0,True,False,1,phone,Apple,False,False
506,XoiWRv5YyWGL9OQe7vwdv8RqtiaZY,555.00,1,3,3,0,True,False,1,phone,Apple,True,False
704,lt16Q2bcjq9oWKFtIiaNHf2LQgUZY,320.00,1,0,0,0,False,False,1,phone,Samsung,False,False
96,5pgFjfZ4P4jg0wbwuvpdp0jDGsKZY,130.00,1,1,1,0,True,False,1,phone,Apple,False,False
465,V3wLQW0OWIK3UvETwdTpOjBojCFZY,100.00,1,1,1,0,True,False,1,phone,Apple,False,False
86,5LRpfwnndHl9UQ6jFkpNQkAeJqLZY,280.00,1,1,1,0,True,False,1,phone,Samsung,False,False
530,ZbJ1BHV58HtloN3nKwYZrGojrFbZY,90.00,1,1,1,0,True,False,1,unknown,Unknown,False,False
350,NXMEENLUepjT0gbiTO911qfbotdZY,100.00,1,1,1,0,True,False,1,phone,Apple,False,False


In [51]:
 # Revenue by repair_event_count
df_doc.groupby("repair_event_count")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
)

# Revenue by device_type
df_doc.groupby("device_type")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
)

# Revenue by brand
df_doc.groupby("brand")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
)


,rows,revenue,avg_ticket
brand,,,
Apple,637,107677.11,169.037849
Google,27,4695.00,173.888889
PC,5,555.00,111.000000
Samsung,109,22809.20,209.258716
Sony,3,345.00,115.000000
Unknown,187,19184.29,102.589786


In [52]:
df_doc.groupby("device_type")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_values("revenue", ascending=False)


,rows,revenue,avg_ticket
device_type,,,
phone,655,110006.89,167.949450
unknown,155,17456.09,112.619935
tablet,92,15741.62,171.104565
computer,23,8066.00,350.695652
accessory,22,1380.00,62.727273
non-device,11,1205.00,109.545455
watch,5,745.00,149.000000
console,5,665.00,133.000000


In [53]:
df_doc.groupby("repair_event_count")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_index()


,rows,revenue,avg_ticket
repair_event_count,,,
0,196,29534.82,150.687857
1,733,116971.91,159.579686
2,32,6878.87,214.964687
3,5,1790.00,358.000000
4,2,90.00,45.000000


In [54]:
repair_cols = [c for c in df_doc.columns if c.startswith("repair_")]

service_mix = (
    df_doc
    .loc[df_doc["repair_event_count"] > 0, repair_cols]
    .mul(df_doc["net_sales_num"], axis=0)
    .sum()
    .sort_values(ascending=False)
)

service_mix


repair_event_count       136459.65
repair_screen            106086.21
repair_battery            10478.29
repair_back_glass          8969.15
repair_camera               2930.0
repair_water_damage         2315.0
repair_charging_port        2300.0
repair_board_level          2136.0
repair_diagnostic            790.0
repair_software_setup        230.0
repair_speaker_mic           225.0
dtype: object

In [55]:
repair_cols = [
    "repair_screen",
    "repair_battery",
    "repair_back_glass",
    "repair_camera",
    "repair_water_damage",
    "repair_charging_port",
    "repair_board_level",
    "repair_diagnostic",
    "repair_software_setup",
    "repair_speaker_mic"
]

repair_counts = (
    df_doc[repair_cols]
    .sum()
    .sort_values(ascending=False)
)

repair_counts


repair_screen            604
repair_battery            85
repair_back_glass         52
repair_camera             23
repair_charging_port      22
repair_diagnostic         14
repair_water_damage        9
repair_board_level         6
repair_speaker_mic         4
repair_software_setup      1
dtype: int64

In [56]:
repair_summary = pd.DataFrame({
    "count": df_doc[repair_cols].sum(),
    "revenue": df_doc[repair_cols].mul(df_doc["net_sales_num"], axis=0).sum()
}).sort_values("revenue", ascending=False)

repair_summary["avg_ticket"] = (
    repair_summary["revenue"] / repair_summary["count"]
)

repair_summary


,count,revenue,avg_ticket
repair_screen,604,106086.21,175.639421
repair_battery,85,10478.29,123.274000
repair_back_glass,52,8969.15,172.483654
repair_camera,23,2930.00,127.391304
repair_water_damage,9,2315.00,257.222222
repair_charging_port,22,2300.00,104.545455
repair_board_level,6,2136.00,356.000000
repair_diagnostic,14,790.00,56.428571
repair_software_setup,1,230.00,230.000000
repair_speaker_mic,4,225.00,56.250000


In [57]:
df_doc["service_unknown_flag"] = (
    (df_doc["repair_event_count"] == 0) &
    (~df_doc["device_only_flag"]) == False
)


In [58]:
df_doc["service_unknown_flag"] = (
    df_doc["repair_event_count"] == 0
)


In [59]:
service_summary = pd.DataFrame({
    "count": df_doc[repair_cols].sum(),
    "revenue": df_doc[repair_cols].mul(df_doc["net_sales_num"], axis=0).sum()
})

# Add Unknown service bucket
service_summary.loc["service_unknown"] = {
    "count": (df_doc["repair_event_count"] == 0).sum(),
    "revenue": df_doc.loc[df_doc["repair_event_count"] == 0, "net_sales_num"].sum()
}

service_summary["avg_ticket"] = (
    service_summary["revenue"] / service_summary["count"]
)

service_summary = service_summary.sort_values("revenue", ascending=False)

service_summary


,count,revenue,avg_ticket
repair_screen,604,106086.21,175.639421
service_unknown,196,29534.82,150.687857
repair_battery,85,10478.29,123.274000
repair_back_glass,52,8969.15,172.483654
repair_camera,23,2930.00,127.391304
repair_water_damage,9,2315.00,257.222222
repair_charging_port,22,2300.00,104.545455
repair_board_level,6,2136.00,356.000000
repair_diagnostic,14,790.00,56.428571
repair_software_setup,1,230.00,230.000000


In [60]:
tx_device_counts = (
    df_doc
    .groupby("Transaction ID")["device_type"]
    .nunique()
    .rename("device_count")
)


In [61]:
df_doc = df_doc.merge(
    tx_device_counts,
    on="Transaction ID",
    how="left"
)


In [62]:
df_doc["multi_device_flag"] = df_doc["device_count"] > 1


In [63]:
print(df_doc["multi_device_flag"].value_counts())

display(
    df_doc[df_doc["multi_device_flag"]]
    [["text_clean", "device_type", "brand", "net_sales_num", "Transaction ID"]]
    .head(10)
)


multi_device_flag
False    922
True      46
Name: count, dtype: int64


,text_clean,device_type,brand,net_sales_num,Transaction ID
52,screen protector,accessory,Unknown,20.0,uKg0jnlaYGaE9pGs5GyeZr3eV
53,13 port,phone,Apple,100.0,uKg0jnlaYGaE9pGs5GyeZr3eV
54,camera lens,unknown,Unknown,50.0,uKg0jnlaYGaE9pGs5GyeZr3eV
103,ipad 9 screen repair,tablet,Apple,120.0,yoa4MGkJmxWwg6Dq90JZcJkeV
104,lcd repair,unknown,Unknown,100.0,yoa4MGkJmxWwg6Dq90JZcJkeV
134,iphone 14 screen repair,phone,Apple,200.0,28ZF758RncBzgCveij1NNXpeV
135,back glass replacement,unknown,Unknown,100.0,28ZF758RncBzgCveij1NNXpeV
188,back glass replacement,unknown,Unknown,100.0,gN9G4YMhpbWnEl7UjABKeuAfV
189,iphone 12 screen replacement,phone,Apple,120.0,gN9G4YMhpbWnEl7UjABKeuAfV
197,camera lens and protector,accessory,Unknown,25.0,ctuukKHyRBCK5lND2L27Jg3eV


In [64]:
df_doc.groupby("multi_device_flag")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
)


,rows,revenue,avg_ticket
multi_device_flag,,,
False,922,150310.4,163.026464
True,46,4955.2,107.721739


In [65]:
df_doc["multi_repair_flag"] = df_doc["repair_event_count"] > 1


In [66]:
df_doc.groupby("multi_repair_flag")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
)


,rows,revenue,avg_ticket
multi_repair_flag,,,
False,929,146506.73,157.703692
True,39,8758.87,224.586410


In [67]:
import re
import numpy as np

def detect_model_family(text: str) -> str:
    if not isinstance(text, str) or text.strip() == "":
        return "Unknown"

    t = text.lower()

    # Apple families
    if re.search(r"\biphone\b|\bpromax\b|\bpro max\b|\bxs\b|\bxr\b|\bse\b", t):
        return "iPhone"
    if re.search(r"\bipad\b", t):
        return "iPad"
    if re.search(r"\bmacbook\b|\bimac\b|\bmac\b", t):
        return "Mac"
    if re.search(r"\bapple watch\b|\bwatch\b", t):
        return "Apple Watch"
    if re.search(r"\bairpod\b|\bairpods\b", t):
        return "AirPods/Accessory"

    # Samsung families
    if re.search(r"\bs2[0-9]\b|\bs2[0-9]\s?(ultra|plus)?\b|\bs21\b|\bs22\b|\bs23\b|\bs24\b", t):
        return "Galaxy S"
    if re.search(r"\ba[0-9]{2}\b", t):
        return "Galaxy A"
    if re.search(r"\bnote\b", t):
        return "Galaxy Note"

    # Google families
    if re.search(r"\bpixel\b", t):
        return "Pixel"

    # Console families
    if re.search(r"\bps5\b|\bps4\b|\bplaystation\b", t):
        return "PlayStation"
    if re.search(r"\bxbox\b", t):
        return "Xbox"
    if re.search(r"\bnintendo\b|\bswitch\b", t):
        return "Nintendo"

    # PC / generic computer
    if re.search(r"\bpc\b|\bwindows\b|\blaptop\b|\bdesktop\b|\bimac\b", t):
        return "Computer"

    # Accessory / retail
    if re.search(r"\bcase\b|\bscreen protector\b|\bprotector\b|\bcable\b|\bcharger\b|\blens\b", t):
        return "Accessory"

    return "Unknown"

df_doc["device_model_family"] = df_doc["text_clean"].apply(detect_model_family)
df_doc["device_model_family"].value_counts().head(20)


device_model_family
iPhone               434
Unknown              200
iPad                 114
Galaxy S              94
Pixel                 27
Accessory             27
Galaxy A              26
Mac                   16
Galaxy Note           13
Computer               8
PlayStation            3
Apple Watch            3
Xbox                   2
AirPods/Accessory      1
Name: count, dtype: int64

In [68]:
def detect_model_exact(text: str) -> str:
    if not isinstance(text, str) or text.strip() == "":
        return "Unknown"
    t = text.lower()

    # iPhone exact patterns (captures: iphone 14 pro max, 13, 11, etc.)
    m = re.search(r"\biphone\s*(\d{1,2})\s*(pro\s*max|promax|pro|plus|mini)?\b", t)
    if m:
        num = m.group(1)
        suffix = (m.group(2) or "").replace("promax", "pro max").strip()
        suffix = (" " + suffix) if suffix else ""
        return f"iPhone {num}{suffix}".strip().title().replace("Iphone", "iPhone")

    # iPhone shorthand without 'iphone' (xr/xs)
    if re.search(r"\bxr\b", t): return "iPhone XR"
    if re.search(r"\bxs max\b", t): return "iPhone XS Max"
    if re.search(r"\bxs\b", t): return "iPhone XS"
    if re.search(r"\bse\b", t): return "iPhone SE"

    # iPad exact (ipad 9, ipad air, ipad mini, ipad pro 12.9)
    m = re.search(r"\bipad\s*(air|mini|pro)?\s*([0-9]{1,2}(\.[0-9])?)?\s*(\d(th|rd|nd|st)\s*gen)?\b", t)
    if "ipad" in t:
        # keep it simple and readable
        if "ipad air" in t: return "iPad Air"
        if "ipad mini" in t: return "iPad Mini"
        if "ipad pro" in t: return "iPad Pro"
        m2 = re.search(r"\bipad\s*([0-9]{1,2})\b", t)
        if m2: return f"iPad {m2.group(1)}"
        return "iPad"

    # Galaxy S exact
    m = re.search(r"\bs(2[0-9])\s*(ultra|plus)?\b", t)
    if m:
        model = f"Galaxy S{m.group(1)}"
        if m.group(2): model += f" {m.group(2).title()}"
        return model

    # Galaxy Note exact
    m = re.search(r"\bnote\s*([0-9]{1,2})\s*(plus|ultra)?\b", t)
    if m:
        model = f"Galaxy Note {m.group(1)}"
        if m.group(2): model += f" {m.group(2).title()}"
        return model

    # Pixel exact
    m = re.search(r"\bpixel\s*([0-9]{1,2})\s*(pro|xl|a)?\b", t)
    if m:
        model = f"Pixel {m.group(1)}"
        if m.group(2): model += f" {m.group(2).upper() if m.group(2) in ['xl','a'] else m.group(2).title()}"
        return model

    # Consoles
    if "ps5" in t or "playstation 5" in t: return "PS5"
    if "ps4" in t or "playstation 4" in t: return "PS4"
    if "xbox" in t: return "Xbox"
    if "switch" in t or "nintendo" in t: return "Nintendo Switch"

    # Accessories (keep explicit)
    if "screen protector" in t: return "Screen Protector"
    if "camera lens" in t or "lens" in t: return "Camera Lens"
    if "case" in t: return "Case"
    if "cable" in t: return "Cable"
    if "charger" in t: return "Charger"
    if "airpod" in t: return "AirPods"

    return "Unknown"

df_doc["device_model_exact"] = df_doc["text_clean"].apply(detect_model_exact)
df_doc["device_model_exact"].value_counts().head(30)


device_model_exact
Unknown              340
iPhone 11             64
iPhone 13             46
iPad 9                34
iPhone 12             33
Galaxy S23 Ultra      28
Galaxy S22 Ultra      21
iPhone 14 Pro Max     21
iPhone 14             20
iPad                  19
iPhone XR             19
iPad Pro              18
iPhone SE             18
Screen Protector      17
iPhone 15             15
iPhone 15 Pro Max     15
iPad Air              14
iPad Mini             12
iPhone 8               9
Galaxy S24 Ultra       9
Camera Lens            9
iPhone 13 Pro Max      9
iPhone 15 Pro          8
Galaxy S20             8
iPhone 12 Pro Max      8
Galaxy S21             7
Galaxy S21 Ultra       7
iPhone 13 Pro          7
Case                   6
Galaxy S23             6
Name: count, dtype: int64

In [69]:
df_doc.groupby("device_model_family")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_values("revenue", ascending=False)


,rows,revenue,avg_ticket
device_model_family,,,
iPhone,434,62221.49,143.367488
Unknown,200,28511.00,142.555000
iPad,114,21416.62,187.865088
Galaxy S,94,20923.40,222.589362
Mac,16,8014.00,500.875000
Pixel,27,4695.00,173.888889
Galaxy A,26,3244.09,124.772692
Galaxy Note,13,2595.00,199.615385
Accessory,27,1730.00,64.074074


In [70]:
model_exact_summary = df_doc.groupby("device_model_exact")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_values("revenue", ascending=False)

model_exact_summary.head(30)


,rows,revenue,avg_ticket
device_model_exact,,,
Unknown,340,57613.96,169.452824
Galaxy S23 Ultra,28,7514.00,268.357143
iPhone 11,64,6808.00,106.375000
iPhone 13,46,6502.61,141.361087
Galaxy S22 Ultra,21,5456.40,259.828571
iPad Pro,18,5150.00,286.111111
iPad 9,34,4275.00,125.735294
iPad Mini,12,4240.00,353.333333
iPhone 12,33,3990.00,120.909091


In [71]:
unknown_models = df_doc[df_doc["device_model_exact"] == "Unknown"]
print("Unknown exact models:", unknown_models.shape[0])

unknown_models["text_clean"].value_counts().head(30)


Unknown exact models: 340


text_clean
14 pro max screen repair          16
11 pro max screen repair           9
15 pro max screen repair           9
15 pro screen repair               7
back glass replacement             7
13 screen repair                   6
13 pro max screen repair           6
14 pro screen repair               6
15 screen repair                   5
15 pro max screen replacement      5
14 plus screen repair              5
battery replacement                4
clickmate                          4
15 promax screen repair            4
16 pro screen repair               3
diagnostics                        3
13 pro screen repair               3
14 pro back glass replacement      3
port replacement                   3
lost mary                          3
14 promax screen repair            3
charging port repair               3
11 pro screen repair               3
a14 5g screen repair               2
moto g 2024 screen repair          2
12 mini battery replacement        2
surface pro 1798 screen rep

In [72]:
# ======================================================================================
# PARAGON GEEKS — POS ANALYTICS PROJECT
# BASELINE SNAPSHOT (DOCUMENTED ROWS ONLY)
# LOCK THIS CELL — DO NOT MODIFY AFTER RUNNING
# ======================================================================================

import pandas as pd

# -------------------------------
# SAFETY CHECKS
# -------------------------------
required_cols = [
    "net_sales_num",
    "device_type",
    "brand",
    "device_model_family",
    "device_model_exact",
    "repair_event_count",
    "multi_repair_flag",
    "multi_device_flag",
    "Transaction ID"
]

missing = [c for c in required_cols if c not in df_doc.columns]
if missing:
    raise ValueError(f"Missing required columns in df_doc: {missing}")

assert df_doc.shape[0] == 968, "Baseline row count is NOT 968 — STOP"

# -------------------------------
# BASELINE METRICS
# -------------------------------
BASELINE_META = {
    "rows": df_doc.shape[0],
    "total_revenue": df_doc["net_sales_num"].sum(),
    "avg_ticket": df_doc["net_sales_num"].mean(),
    "min_ticket": df_doc["net_sales_num"].min(),
    "max_ticket": df_doc["net_sales_num"].max(),
}

baseline_meta = pd.DataFrame(BASELINE_META, index=["BASELINE"])

# -------------------------------
# STANDARD AGG FUNCTION
# -------------------------------
def agg_table(df, group_col):
    return (
        df.groupby(group_col)["net_sales_num"]
          .agg(rows="count", revenue="sum", avg_ticket="mean")
          .sort_values("revenue", ascending=False)
    )

# -------------------------------
# CORE BASELINE TABLES
# -------------------------------
baseline_device_type = agg_table(df_doc, "device_type")
baseline_brand = agg_table(df_doc, "brand")
baseline_model_family = agg_table(df_doc, "device_model_family")
baseline_model_exact = agg_table(df_doc, "device_model_exact")

# -------------------------------
# REPAIR / SERVICE BASELINE
# -------------------------------
repair_cols = [c for c in df_doc.columns if c.startswith("repair_")]

repair_long = (
    df_doc[repair_cols + ["net_sales_num"]]
    .melt(id_vars="net_sales_num", var_name="repair_type", value_name="flag")
    .query("flag == True")
)

baseline_repairs = (
    repair_long.groupby("repair_type")["net_sales_num"]
    .agg(rows="count", revenue="sum", avg_ticket="mean")
    .sort_values("revenue", ascending=False)
)

# -------------------------------
# COMPLEXITY BASELINE
# -------------------------------
baseline_multi_repair = agg_table(df_doc, "multi_repair_flag")
baseline_multi_device = agg_table(df_doc, "multi_device_flag")

baseline_repair_event_count = agg_table(df_doc, "repair_event_count")

# -------------------------------
# OPTIONAL: TRANSACTION-LEVEL SNAPSHOT (REFERENCE ONLY)
# -------------------------------
transaction_snapshot = (
    df_doc.groupby("Transaction ID")["net_sales_num"]
    .sum()
    .to_frame("transaction_revenue")
)

# -------------------------------
# DISPLAY SNAPSHOT (ORDER MATTERS)
# -------------------------------
print("\n==============================")
print("BASELINE META")
print("==============================")
display(baseline_meta)

print("\n==============================")
print("DEVICE TYPE")
print("==============================")
display(baseline_device_type)

print("\n==============================")
print("BRAND")
print("==============================")
display(baseline_brand)

print("\n==============================")
print("MODEL EXACT (PRE-REFINEMENT)")
print("==============================")
display(baseline_model_exact.head(30))

print("\n==============================")
print("REPAIR / SERVICE MIX")
print("==============================")
display(baseline_repairs)

print("\n==============================")
print("REPAIR EVENT COUNT")
print("==============================")
display(baseline_repair_event_count)

print("\n==============================")
print("MULTI-REPAIR FLAG")
print("==============================")
display(baseline_multi_repair)

print("\n==============================")
print("MULTI-DEVICE FLAG")
print("==============================")
display(baseline_multi_device)

# -------------------------------
# OPTIONAL: PERSIST BASELINE (UNCOMMENT TO LOCK)
# -------------------------------
# baseline_meta.to_csv("baseline_meta.csv")
# baseline_device_type.to_csv("baseline_device_type.csv")
# baseline_brand.to_csv("baseline_brand.csv")
# baseline_model_family.to_csv("baseline_model_family.csv")
# baseline_model_exact.to_csv("baseline_model_exact.csv")
# baseline_repairs.to_csv("baseline_repairs.csv")
# baseline_multi_repair.to_csv("baseline_multi_repair.csv")
# baseline_multi_device.to_csv("baseline_multi_device.csv")
# baseline_repair_event_count.to_csv("baseline_repair_event_count.csv")

# df_doc.to_pickle("df_doc_BASELINE_LOCKED.pkl")

print("\nBASELINE LOCKED — SAFE TO PROCEED TO UNKNOWN REDUCTION")



BASELINE META


,rows,total_revenue,avg_ticket,min_ticket,max_ticket
BASELINE,968,155265.6,160.398347,-345.0,3000.0



DEVICE TYPE


,rows,revenue,avg_ticket
device_type,,,
phone,655,110006.89,167.949450
unknown,155,17456.09,112.619935
tablet,92,15741.62,171.104565
computer,23,8066.00,350.695652
accessory,22,1380.00,62.727273
non-device,11,1205.00,109.545455
watch,5,745.00,149.000000
console,5,665.00,133.000000



BRAND


...


MULTI-REPAIR FLAG


,rows,revenue,avg_ticket
multi_repair_flag,,,
False,929,146506.73,157.703692
True,39,8758.87,224.586410



MULTI-DEVICE FLAG


,rows,revenue,avg_ticket
multi_device_flag,,,
False,922,150310.4,163.026464
True,46,4955.2,107.721739



BASELINE LOCKED — SAFE TO PROCEED TO UNKNOWN REDUCTION


In [73]:
unknown_mask = df_doc["device_model_exact"] == "Unknown"
df_unknown = df_doc.loc[unknown_mask].copy()

print("Unknown model rows:", df_unknown.shape[0])
print("Unknown model revenue:", df_unknown["net_sales_num"].sum())


Unknown model rows: 340
Unknown model revenue: 57613.96


In [74]:
MODEL_RULES = [
    # iPhone Pro Max
    (r"\b15\s*pro\s*max\b", "iPhone 15 Pro Max"),
    (r"\b14\s*pro\s*max\b", "iPhone 14 Pro Max"),
    (r"\b13\s*pro\s*max\b", "iPhone 13 Pro Max"),
    (r"\b12\s*pro\s*max\b", "iPhone 12 Pro Max"),
    (r"\b11\s*pro\s*max\b", "iPhone 11 Pro Max"),

    # iPhone Pro
    (r"\b15\s*pro\b", "iPhone 15 Pro"),
    (r"\b14\s*pro\b", "iPhone 14 Pro"),
    (r"\b13\s*pro\b", "iPhone 13 Pro"),
    (r"\b12\s*pro\b", "iPhone 12 Pro"),
    (r"\b11\s*pro\b", "iPhone 11 Pro"),

    # iPhone Base
    (r"\biphone\s*15\b|\b15\s*screen\b", "iPhone 15"),
    (r"\biphone\s*14\b|\b14\s*screen\b", "iPhone 14"),
    (r"\biphone\s*13\b|\b13\s*screen\b", "iPhone 13"),
    (r"\biphone\s*12\b|\b12\s*screen\b", "iPhone 12"),
    (r"\biphone\s*11\b|\b11\s*screen\b", "iPhone 11"),
]


In [75]:
df_unknown["device_model_refined"] = "Unknown"

for pattern, model in MODEL_RULES:
    mask = (
        df_unknown["device_model_refined"] == "Unknown"
    ) & (
        df_unknown["text_clean"].str.contains(pattern, regex=True)
    )

    df_unknown.loc[mask, "device_model_refined"] = model


In [76]:
before = df_unknown["device_model_exact"].value_counts().head(5)
after = df_unknown["device_model_refined"].value_counts().head(10)

print("BEFORE:")
print(before)

print("\nAFTER:")
print(after)


BEFORE:
device_model_exact
Unknown    340
Name: count, dtype: int64

AFTER:
device_model_refined
Unknown              211
iPhone 14 Pro Max     31
iPhone 15 Pro Max     24
iPhone 14 Pro         12
iPhone 13 Pro Max     11
iPhone 15 Pro         10
iPhone 11 Pro Max     10
iPhone 13              8
iPhone 13 Pro          6
iPhone 12 Pro Max      6
Name: count, dtype: int64


In [77]:
df_doc.loc[
    df_unknown.index, "device_model_exact_refined"
] = df_unknown["device_model_refined"]

# Fill non-unknown rows
df_doc["device_model_exact_refined"] = (
    df_doc["device_model_exact_refined"]
    .fillna(df_doc["device_model_exact"])
)


In [78]:
print("Original Unknowns:", (df_doc["device_model_exact"] == "Unknown").sum())
print("Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

# Revenue check
assert df_doc["net_sales_num"].sum() == 155265.6
assert df_doc.shape[0] == 968


Original Unknowns: 340
Remaining Unknowns: 211


In [79]:
df_unknown_2 = df_doc[
    df_doc["device_model_exact_refined"] == "Unknown"
].copy()

print("Remaining Unknown rows:", df_unknown_2.shape[0])
print("Remaining Unknown revenue:", df_unknown_2["net_sales_num"].sum())


Remaining Unknown rows: 211
Remaining Unknown revenue: 34008.09


In [80]:
from collections import Counter

tokens = (
    df_unknown_2["text_clean"]
    .str.split()
    .explode()
)

token_counts = Counter(tokens)

pd.DataFrame(
    token_counts.most_common(40),
    columns=["token", "count"]
)


,token,count
0,repair,91
1,screen,85
2,replacement,42
3,port,20
4,plus,20
5,battery,19
6,deposit,16
7,and,14
8,back,13
9,pro,12


In [81]:
MICRO_RULES = [
    # Samsung A-series
    (r"\ba15\b", "Galaxy A15"),
    (r"\ba14\s*5g\b", "Galaxy A14 5G"),
    (r"\ba32\b", "Galaxy A32"),

    # Samsung Foldables
    (r"\bz\s*flip\b", "Galaxy Z Flip"),

    # Motorola
    (r"\bmoto\s*g\b", "Moto G"),

    # Apple laptops
    (r"\bmacbook\b", "MacBook"),

    # Generic computers (do NOT over-specify)
    (r"\bpc\b|\bcomputer\b", "Computer"),
]


In [82]:
df_doc["device_model_exact_refined"] == "Unknown"


0      False
1      False
2       True
3      False
4       True
       ...  
963    False
964    False
965    False
966    False
967    False
Name: device_model_exact_refined, Length: 968, dtype: bool

In [83]:
(df_doc["device_model_exact_refined"] == "Unknown").sum()


211

In [84]:
df_doc.loc[
    df_doc["device_model_exact_refined"] == "Unknown",
    "text_clean"
].value_counts().head(50)


text_clean
back glass replacement                                        7
14 plus screen repair                                         5
clickmate                                                     4
battery replacement                                           4
port replacement                                              3
charging port repair                                          3
diagnostics                                                   3
16 pro screen repair                                          3
lost mary                                                     3
a32 screen repair                                             2
8 plus screen repair                                          2
moto g 2024 screen repair                                     2
z flip 6 screen repair                                        2
a14 5g screen repair                                          2
a13 screen repair                                             2
mexican mango geekbar        

In [85]:
FINAL_MICRO_RULES = [
    (r"\ba32\b", "Galaxy A32"),
    (r"\ba14\s*5g\b", "Galaxy A14 5G"),
    (r"\ba15\b", "Galaxy A15"),
    (r"\ba54\b", "Galaxy A54"),
    (r"\bz\s*flip\b|\bzflip\b", "Galaxy Z Flip"),
    (r"\bmoto\s*g\b", "Moto G"),
    (r"\bmacbook\b", "MacBook"),
    (r"\bsurface\s*pro\b", "Surface Pro"),
]


In [86]:
baseline_unknowns = (df_doc["device_model_exact"] == "Unknown").sum()
final_unknowns = (df_doc["device_model_exact_refined"] == "Unknown").sum()

print(f"Baseline Unknowns: {baseline_unknowns}")
print(f"Remaining Unknowns: {final_unknowns}")
print(f"Reduction: {baseline_unknowns - final_unknowns}")
print(f"Reduction %: {(baseline_unknowns - final_unknowns) / baseline_unknowns:.1%}")


Baseline Unknowns: 340
Remaining Unknowns: 211
Reduction: 129
Reduction %: 37.9%


In [87]:
new_models_added = (
    df_doc.loc[
        df_doc["device_model_exact"] == "Unknown",
        "device_model_exact_refined"
    ]
    .value_counts()
    .drop("Unknown", errors="ignore")
)

new_models_added.head(15)


device_model_exact_refined
iPhone 14 Pro Max    31
iPhone 15 Pro Max    24
iPhone 14 Pro        12
iPhone 13 Pro Max    11
iPhone 15 Pro        10
iPhone 11 Pro Max    10
iPhone 13             8
iPhone 13 Pro         6
iPhone 12 Pro Max     6
iPhone 11 Pro         5
iPhone 15             5
iPhone 12 Pro         1
Name: count, dtype: int64

In [88]:
def unknown_reason(text):
    if any(k in text for k in ["screen", "battery", "port", "glass", "repair"]):
        return "Service-only (no model)"
    if any(k in text for k in ["deposit", "paid", "tip"]):
        return "Payment / Admin"
    if any(k in text for k in ["clickmate", "geek", "lost mary", "bar"]):
        return "Smoke / Vape Retail"
    if any(k in text for k in ["diagnostic", "install", "reinstallation"]):
        return "General Service"
    return "Ambiguous / Mixed"

df_unknown_final = df_doc[
    df_doc["device_model_exact_refined"] == "Unknown"
].copy()

df_unknown_final["unknown_reason"] = df_unknown_final["text_clean"].apply(unknown_reason)


In [89]:
df_unknown_final["unknown_reason"].value_counts()


unknown_reason
Service-only (no model)    142
Ambiguous / Mixed           30
Smoke / Vape Retail         19
General Service             11
Payment / Admin              9
Name: count, dtype: int64

In [90]:
for reason in df_unknown_final["unknown_reason"].unique():
    print("\n", reason.upper())
    display(
        df_unknown_final[df_unknown_final["unknown_reason"] == reason]
        [["text_clean", "net_sales_num"]]
        .head(5)
    )



 SERVICE-ONLY (NO MODEL)


,text_clean,net_sales_num
2,a53 screen replacement,100.0
12,partial payment iphone x screen replacement,80.0
33,14plus screen repair,140.0
38,samsung a32 screen repair,130.8
41,battery replacement,60.0



 AMBIGUOUS / MIXED


,text_clean,net_sales_num
4,imac 2022 with ram upgrade,3000.0
209,water damage,150.0
243,after pay fee,12.0
245,consulting,750.0
260,imac ram upgrade 64gb,260.0



 GENERAL SERVICE


,text_clean,net_sales_num
88,tablet install,50.0
211,dual sim installation,65.0
301,mac harddrive install,100.0
518,samsung diagnostics,50.0
521,diagnostics,20.0



 PAYMENT / ADMIN


,text_clean,net_sales_num
139,1 plus 7t deposit,40.0
596,moto g 25 deposit,50.0
606,tip for kevin,20.0
622,nord n300 deposit,50.0
654,deposit for hp laptop,85.0



 SMOKE / VAPE RETAIL


,text_clean,net_sales_num
681,clickmate,30.0
692,clickmate,30.0
702,2x clickmate,60.0
709,lost mary,30.0
713,clickmate,30.0


In [91]:
FINAL_TARGET_RULES = [
    # Apple
    (r"\b14\s*plus\b", "iPhone 14 Plus"),
    (r"\b13\s*mini\b", "iPhone 13 Mini"),
    (r"\b12\s*mini\b", "iPhone 12 Mini"),
    (r"\biphone\s*x\b", "iPhone X"),
    (r"\bimac\s*2022\b", "iMac 2022"),
    (r"\bimac\b", "iMac"),
    (r"\bmacbook\s*a2338\b", "MacBook Air M1"),
    (r"\bmacbook\b", "MacBook"),

    # Samsung
    (r"\ba53\b", "Galaxy A53"),
    (r"\ba32\b", "Galaxy A32"),
    (r"\ba54\b", "Galaxy A54"),
    (r"\ba15\b", "Galaxy A15"),
    (r"\ba14\b", "Galaxy A14"),
    (r"\bz\s*flip\s*6\b", "Galaxy Z Flip 6"),
    (r"\bz\s*flip\s*3\b", "Galaxy Z Flip 3"),
    (r"\bz\s*flip\b", "Galaxy Z Flip"),

    # OnePlus
    (r"\b1\s*plus\s*7t\b", "OnePlus 7T"),
    (r"\bnord\s*n300\b", "OnePlus Nord N300"),

    # Motorola
    (r"\bmoto\s*g\s*25\b", "Moto G25"),
    (r"\bmoto\s*g\b", "Moto G"),

    # Microsoft
    (r"\bsurface\s*pro\s*1798\b", "Surface Pro 1798"),

    # HP / Asus (generic but acceptable)
    (r"\bhp\s*laptop\b", "HP Laptop"),
    (r"\basus\s*vivobook\b", "Asus VivoBook"),
]


In [92]:
mask_unknown = df_doc["device_model_exact_refined"] == "Unknown"

for pattern, model in FINAL_TARGET_RULES:
    hit = mask_unknown & df_doc["text_clean"].str.contains(pattern, regex=True)
    df_doc.loc[hit, "device_model_exact_refined"] = model


In [93]:
print("Final Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

assert df_doc["net_sales_num"].sum() == 155265.6
assert df_doc.shape[0] == 968


Final Remaining Unknowns: 142


In [94]:
df_unknown_final = df_doc[
    df_doc["device_model_exact_refined"] == "Unknown"
].copy()

print("Final Unknown rows:", df_unknown_final.shape[0])
print("Final Unknown revenue:", df_unknown_final["net_sales_num"].sum())


Final Unknown rows: 142
Final Unknown revenue: 18449.0


In [95]:
from collections import Counter

tokens = (
    df_unknown_final["text_clean"]
    .str.split()
    .explode()
)

token_counts = Counter(tokens)

pd.DataFrame(
    token_counts.most_common(40),
    columns=["token", "count"]
)


,token,count
0,repair,47
1,screen,41
2,replacement,31
3,port,17
4,battery,14
5,and,13
6,plus,12
7,back,11
8,deposit,10
9,15,9


In [96]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    ngram_range=(2, 3),
    min_df=3
)

ngrams = cv.fit_transform(df_unknown_final["text_clean"])
ngram_freq = ngrams.sum(axis=0).A1

ngram_df = pd.DataFrame(
    zip(cv.get_feature_names_out(), ngram_freq),
    columns=["ngram", "count"]
).sort_values("count", ascending=False)

ngram_df.head(25)


,ngram,count
21,screen repair,32
12,glass replacement,8
4,back glass,8
5,back glass replacement,8
6,battery replacement,8
22,screen replacement,7
16,plus screen repair,6
15,plus screen,6
18,port replacement,5
13,lost mary,5


In [97]:
import re

def aggressive_model_resolver(text):
    t = text.lower()

    # ------------------------
    # APPLE — IMPLIED MODELS
    # ------------------------
    # iPhone Pro / Pro Max
    if re.search(r"\b16\s*pro\b", t):
        return "iPhone 16 Pro"
    if re.search(r"\b16\s*pro\s*max\b", t):
        return "iPhone 16 Pro Max"
    if re.search(r"\b15\s*pro\s*max\b", t):
        return "iPhone 15 Pro Max"
    if re.search(r"\b15\s*pro\b", t):
        return "iPhone 15 Pro"
    if re.search(r"\b14\s*pro\s*max\b", t):
        return "iPhone 14 Pro Max"
    if re.search(r"\b14\s*pro\b", t):
        return "iPhone 14 Pro"

    # iPhone Plus
    if re.search(r"\b15\s*plus\b", t):
        return "iPhone 15 Plus"
    if re.search(r"\b14\s*plus\b", t):
        return "iPhone 14 Plus"
    if re.search(r"\b13\s*plus\b", t):
        return "iPhone 13 Plus"

    # iPhone base (numeric shorthand)
    if re.search(r"\b15\b", t):
        return "iPhone 15"
    if re.search(r"\b14\b", t):
        return "iPhone 14"
    if re.search(r"\b13\b", t):
        return "iPhone 13"
    if re.search(r"\b12\b", t):
        return "iPhone 12"
    if re.search(r"\b11\b", t):
        return "iPhone 11"
    if re.search(r"\b8\b", t):
        return "iPhone 8"

    # Mini
    if re.search(r"\bmini\b", t):
        return "iPhone Mini"

    # ------------------------
    # SAMSUNG — A & S SERIES
    # ------------------------
    if re.search(r"\ba13\b", t):
        return "Galaxy A13"
    if re.search(r"\ba32\b", t):
        return "Galaxy A32"
    if re.search(r"\ba53\b", t):
        return "Galaxy A53"
    if re.search(r"\ba54\b", t):
        return "Galaxy A54"
    if re.search(r"\bs10\b", t):
        return "Galaxy S10"
    if re.search(r"\bs21\b", t):
        return "Galaxy S21"
    if re.search(r"\bs22\b", t):
        return "Galaxy S22"
    if re.search(r"\bs23\b", t):
        return "Galaxy S23"

    # Z / Flip
    if re.search(r"\bz\s*flip\b", t):
        return "Galaxy Z Flip"

    # ------------------------
    # GOOGLE / PIXEL
    # ------------------------
    if re.search(r"\bpixel\b", t):
        return "Google Pixel"

    # ------------------------
    # COMPUTERS
    # ------------------------
    if re.search(r"\bmacbook\b", t):
        return "MacBook"
    if re.search(r"\bimac\b", t):
        return "iMac"
    if re.search(r"\bpc\b", t):
        return "PC"

    # ------------------------
    # ACCESSORIES / NON-PHONE
    # ------------------------
    if re.search(r"screen protector|case|lens", t):
        return "Accessory"
    if re.search(r"clickmate|geekbar|lost mary|vape", t):
        return "Smoke / Vape"

    return "Unknown"


In [98]:
mask = df_doc["device_model_exact_refined"] == "Unknown"

df_doc.loc[mask, "device_model_exact_refined"] = (
    df_doc.loc[mask, "text_clean"]
    .apply(aggressive_model_resolver)
)

print("Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

# Safety checks
assert df_doc.shape[0] == 968
assert round(df_doc["net_sales_num"].sum(), 2) == 155265.60


Remaining Unknowns: 98


In [99]:
df_unknown_final = df_doc[
    df_doc["device_model_exact_refined"] == "Unknown"
].copy()

df_unknown_final.shape


(98, 94)

In [100]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    min_df=3,
    stop_words="english"
)

X = cv.fit_transform(df_unknown_final["text_clean"])

token_freq = (
    pd.DataFrame(
        zip(cv.get_feature_names_out(), X.sum(axis=0).A1),
        columns=["token", "count"]
    )
    .sort_values("count", ascending=False)
)

token_freq.head(50)


,token,count
15,repair,32
18,screen,29
16,replacement,28
13,port,15
2,battery,13
5,deposit,10
8,glass,8
17,samsung,7
4,computer,6
7,geek,5


In [101]:
cv_ng = CountVectorizer(
    ngram_range=(2, 3),
    min_df=2,
    stop_words="english"
)

X_ng = cv_ng.fit_transform(df_unknown_final["text_clean"])

ngram_freq = (
    pd.DataFrame(
        zip(cv_ng.get_feature_names_out(), X_ng.sum(axis=0).A1),
        columns=["ngram", "count"]
    )
    .sort_values("count", ascending=False)
)

ngram_freq.head(50)


,ngram,count
17,screen repair,21
3,battery replacement,8
10,glass replacement,8
19,screen replacement,6
4,charging port,5
9,geek bar,4
13,port replacement,4
5,charging port repair,3
6,computer repair,3
7,computer screen,3


In [102]:
display(
    df_unknown_final[
        df_unknown_final["text_clean"].str.contains(
            r"iphone|apple|samsung|surface|watch|plus|pro|computer",
            regex=True
        )
    ][["text_clean", "net_sales_num"]]
    .head(60)
)


,text_clean,net_sales_num
102,iphone c screen and battery,170.0
163,apple watch screen replacement,175.0
194,one plus 10 pro battery replacement,80.0
421,surface pro 1866 screen repair,200.0
461,iphone 6s battery and screen repair,90.0
465,16 plus battery and back glass replacement,150.0
474,surface pro 7 screen repair,200.0
501,samsung port,40.0
513,samsung charging port,70.0
518,samsung diagnostics,50.0


In [103]:
def infer_device_from_text(text):
    t = text.lower()

    # Apple Watch
    if "apple watch" in t or ("watch" in t and "apple" in t):
        return "Apple Watch"

    # Microsoft Surface
    if "surface pro" in t:
        return "Surface Pro"

    # iPhone implied
    if "iphone" in t or re.search(r"\b(1[0-6]|plus|pro|max)\b", t):
        return "iPhone (Implied)"

    # Samsung implied
    if "samsung" in t or "5g" in t:
        return "Samsung Galaxy (Implied)"

    # Computer implied
    if "computer" in t or "pc" in t:
        return "Computer (Generic)"

    return "Unknown"


In [104]:
mask = df_doc["device_model_exact_refined"] == "Unknown"

df_doc.loc[mask, "device_model_exact_refined"] = (
    df_doc.loc[mask, "text_clean"]
    .apply(infer_device_from_text)
)


In [105]:
print("Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

df_doc["device_model_exact_refined"].value_counts().head(25)


Remaining Unknowns: 71


device_model_exact_refined
Unknown              71
iPhone 11            64
iPhone 13            56
iPhone 14 Pro Max    52
iPhone 15 Pro Max    39
iPad 9               34
iPhone 12            33
Galaxy S23 Ultra     28
iPhone 15            25
Galaxy S22 Ultra     21
iPhone 13 Pro Max    20
iPhone 14            20
iPhone XR            19
iPad                 19
iPhone 15 Pro        18
iPad Pro             18
iPhone SE            18
iPhone 14 Pro        17
Screen Protector     17
Smoke / Vape         15
iPhone 12 Pro Max    14
iPad Air             14
iPhone 13 Pro        13
iPhone 11 Pro Max    13
iPad Mini            12
Name: count, dtype: int64

In [106]:
remaining_unknowns = df_doc.loc[
    df_doc["device_model_exact_refined"] == "Unknown",
    ["text_clean", "net_sales_num", "device_type", "brand"]
].sort_values("net_sales_num", ascending=False)

remaining_unknowns


,text_clean,net_sales_num,device_type,brand
245,consulting,750.0,unknown,Unknown
635,1tb hard drive install and os install,240.0,non-device,Unknown
867,ram install and system update,230.0,non-device,Unknown
184,s10e screen repair,170.0,unknown,Unknown
682,ho screen repair,160.0,unknown,Unknown
239,back glass replacement,160.0,unknown,Unknown
613,battery replacement,155.0,unknown,Unknown
113,back glass replacement,150.0,unknown,Unknown
209,water damage,150.0,unknown,Unknown
723,windows re installation,150.0,non-device,PC


In [107]:
def final_model_inference(text):
    t = text.lower()

    # Apple
    if re.search(r"\b15pm\b|\b15 pro max\b", t):
        return "iPhone 15 Pro Max"
    if re.search(r"\bx port\b|\biphone x\b", t):
        return "iPhone X"

    # Samsung
    if re.search(r"\bs10e\b", t):
        return "Galaxy S10e"
    if re.search(r"\bs20fe\b", t):
        return "Galaxy S20 FE"
    if re.search(r"\ba03s\b|\ba12\b|\ba35\b|\ba52\b", t):
        return "Galaxy A Series"
    if re.search(r"\bs6 lite\b|\btab a9\b", t):
        return "Galaxy Tab Series"

    # OnePlus
    if re.search(r"\bnord 200\b|\bn200\b", t):
        return "OnePlus Nord N200"

    # Motorola
    if re.search(r"\bmoto g stylus\b|\bm g stylus\b", t):
        return "Moto G Stylus"
    if re.search(r"\brazr\b", t):
        return "Motorola Razr"

    # Computers / Tablets
    if re.search(r"\bsurface pro\b", t):
        return "Surface Pro"
    if re.search(r"\bchromebook\b", t):
        return "Chromebook"
    if re.search(r"\blenovo\b", t):
        return "Lenovo Laptop"
    if re.search(r"\bhp envy\b", t):
        return "HP Envy"
    if re.search(r"\bmac\b", t):
        return "MacBook"
    if re.search(r"\bbeats studio 3\b", t):
        return "Beats Studio 3"

    # Watch
    if re.search(r"\b5 series\b", t):
        return "Apple Watch Series 5"

    return "Unknown"


In [108]:
mask = df_doc["device_model_exact_refined"] == "Unknown"

df_doc.loc[mask, "device_model_exact_refined"] = (
    df_doc.loc[mask, "text_clean"]
    .apply(final_model_inference)
)

print("Final Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

assert df_doc.shape[0] == 968
assert df_doc["net_sales_num"].sum() == 155265.6


Final Remaining Unknowns: 53


In [109]:
# ==============================
# FINAL DEVICE MODEL SUMMARY
# ==============================

model_final = (
    df_doc
    .groupby("device_model_exact_refined")["net_sales_num"]
    .agg(
        rows="count",
        revenue="sum",
        avg_ticket="mean"
    )
    .sort_values("revenue", ascending=False)
)

# Split top models vs remaining unknowns
top_models = model_final.head(25)
unknown_models = model_final.loc[["Unknown"]]

print("\n==============================")
print("FINAL DEVICE MODEL SUMMARY")
print("==============================")

display(top_models)

print("\n==============================")
print("REMAINING UNRESOLVED MODELS")
print("==============================")

display(unknown_models)

print("\nValidation:")
print("Total rows:", df_doc.shape[0])
print("Remaining Unknowns:",
      (df_doc["device_model_exact_refined"] == "Unknown").sum())

assert df_doc.shape[0] == 968
assert round(df_doc["net_sales_num"].sum(), 2) == 155265.60



FINAL DEVICE MODEL SUMMARY


,rows,revenue,avg_ticket
device_model_exact_refined,,,
iPhone 14 Pro Max,52,9711.00,186.750000
iPhone 13,56,7937.61,141.743036
iPhone 15 Pro Max,40,7793.87,194.846750
Galaxy S23 Ultra,28,7514.00,268.357143
iPhone 11,64,6808.00,106.375000
iPhone 15,25,6740.00,269.600000
Galaxy S22 Ultra,21,5456.40,259.828571
iPad Pro,18,5150.00,286.111111
Unknown,53,4828.00,91.094340



REMAINING UNRESOLVED MODELS


,rows,revenue,avg_ticket
device_model_exact_refined,,,
Unknown,53,4828.0,91.09434



Validation:
Total rows: 968
Remaining Unknowns: 53


In [110]:
df_doc["brand_refined"] = df_doc["brand"]
df_doc["device_type_refined"] = df_doc["device_type"]


In [111]:
apple_models = r"iphone|ipad|macbook|imac|apple watch|airpods"
samsung_models = r"galaxy|s23|s22|s21|s20|note|a\d+|z flip|z fold"
google_models = r"pixel"
sony_models = r"playstation"
pc_models = r"surface|lenovo|hp|asus|acer|chromebook|windows"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(apple_models, case=False, na=False),
    "brand_refined"
] = "Apple"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(samsung_models, case=False, na=False),
    "brand_refined"
] = "Samsung"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(google_models, case=False, na=False),
    "brand_refined"
] = "Google"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(sony_models, case=False, na=False),
    "brand_refined"
] = "Sony"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(pc_models, case=False, na=False),
    "brand_refined"
] = "PC"


In [112]:
df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(r"iphone|galaxy|pixel|one plus|motorola", case=False, na=False),
    "device_type_refined"
] = "phone"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(r"ipad|tablet", case=False, na=False),
    "device_type_refined"
] = "tablet"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(r"macbook|imac|surface|lenovo|hp|asus|acer|chromebook", case=False, na=False),
    "device_type_refined"
] = "computer"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(r"apple watch", case=False, na=False),
    "device_type_refined"
] = "watch"

df_doc.loc[
    df_doc["device_model_exact_refined"].str.contains(r"playstation|xbox", case=False, na=False),
    "device_type_refined"
] = "console"


In [113]:
df_doc.groupby("brand_refined")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_values("revenue", ascending=False)

df_doc.groupby("device_type_refined")["net_sales_num"].agg(
    rows="count",
    revenue="sum",
    avg_ticket="mean"
).sort_values("revenue", ascending=False)


,rows,revenue,avg_ticket
device_type_refined,,,
phone,710,114243.98,160.907014
tablet,115,21466.62,186.666261
computer,36,10781.00,299.472222
unknown,71,5639.00,79.422535
accessory,21,1220.00,58.095238
non-device,7,775.00,110.714286
console,5,665.00,133.000000
watch,3,475.00,158.333333


In [114]:
if "activity_type" not in df_doc.columns:
    df_doc["activity_type"] = "unclassified"


In [115]:
display(
    df_doc.loc[
        df_doc["device_type_refined"] == "unknown",
        ["text_clean", "net_sales_num", "activity_type" if "activity_type" in df_doc.columns else None]
    ]
)


,text_clean,net_sales_num,activity_type
41,battery replacement,60.0,unclassified
43,speaker repair,50.0,unclassified
45,repair,110.0,unclassified
48,port replacement,110.0,unclassified
54,camera lens,50.0,unclassified
104,lcd repair,100.0,unclassified
113,back glass replacement,150.0,unclassified
132,camera lens replacement,50.0,unclassified
135,back glass replacement,100.0,unclassified
141,beats studio 3 battery replacement,85.0,unclassified


In [116]:
cols = ["text_clean", "net_sales_num"]
if "activity_type" in df_doc.columns:
    cols.append("activity_type")

display(
    df_doc.loc[
        df_doc["device_type_refined"] == "unknown",
        cols
    ]
)


,text_clean,net_sales_num,activity_type
41,battery replacement,60.0,unclassified
43,speaker repair,50.0,unclassified
45,repair,110.0,unclassified
48,port replacement,110.0,unclassified
54,camera lens,50.0,unclassified
104,lcd repair,100.0,unclassified
113,back glass replacement,150.0,unclassified
132,camera lens replacement,50.0,unclassified
135,back glass replacement,100.0,unclassified
141,beats studio 3 battery replacement,85.0,unclassified


In [117]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=2)
X = cv.fit_transform(
    df_doc.loc[df_doc["device_type_refined"] == "unknown", "text_clean"]
)

pd.DataFrame(
    zip(cv.get_feature_names_out(), X.sum(axis=0).A1),
    columns=["token", "count"]
).sort_values("count", ascending=False).head(25)


,token,count
26,replacement,19
25,repair,17
27,screen,8
3,back,8
24,port,8
14,glass,7
5,battery,6
8,clickmate,6
12,geek,6
15,lens,5


In [118]:
PHONE_ONLY_TOKENS = [
    "back glass",
    "lens",
    "camera lens"
]


In [119]:
HEADPHONE_TOKENS = [
    "beats",
    "airpods",
    "headphone",
    "earbuds",
    "studio"
]


In [120]:
TV_TOKENS = [
    "tv",
    "television"
]


In [121]:
def refine_device_type(row):
    text = row["text_clean"]
    current = row["device_type_refined"]

    if current != "unknown":
        return current

    # TV
    if any(tok in text for tok in TV_TOKENS):
        return "tv"

    # Headphones
    if any(tok in text for tok in HEADPHONE_TOKENS):
        return "headphones"

    # Phone-only repairs
    if any(tok in text for tok in PHONE_ONLY_TOKENS):
        return "phone"

    return "unknown"


In [122]:
df_doc["device_type_refined"] = df_doc.apply(refine_device_type, axis=1)


In [123]:
df_doc[df_doc["device_type_refined"] == "unknown"][
    ["text_clean", "net_sales_num", "activity_type"]
].sort_values("net_sales_num", ascending=False)


,text_clean,net_sales_num,activity_type
245,consulting,750.0,unclassified
682,ho screen repair,160.0,unclassified
613,battery replacement,155.0,unclassified
209,water damage,150.0,unclassified
902,moto g stylus,140.0,unclassified
497,moto g 5g screen repair,135.0,unclassified
860,m g stylus screen repair,130.0,unclassified
910,moto g 2024 screen repair,130.0,unclassified
844,moto g 2024 screen repair,125.0,unclassified
739,cookies gelatti clickmate pink lemonade kirby ...,120.0,unclassified


In [124]:
if "activity_type" not in df_doc.columns:
    df_doc["activity_type"] = "unclassified"


In [125]:
df_doc[df_doc["device_type_refined"] == "non-device"][
    ["text_clean", "net_sales_num", "activity_type"]
]


,text_clean,net_sales_num,activity_type
211,dual sim installation,65.0,unclassified
521,diagnostics,20.0,unclassified
635,1tb hard drive install and os install,240.0,unclassified
684,diagnostics,40.0,unclassified
723,windows re installation,150.0,unclassified
798,diagnostics,30.0,unclassified
867,ram install and system update,230.0,unclassified


In [126]:
PHONE_KEYWORDS = [
    "iphone", "pm", "pro max", "pro",
    "moto", "motorola", "razr", "g stylus", "g power",
    "nord", "one plus", "oneplus",
    "galaxy s", "s10", "s20", "s21", "s22", "s23", "s24",
    "a03", "a12", "a13", "a32", "a35", "a52",
    "z flip", "zfold", "ho screen"
]

TABLET_KEYWORDS = [
    "ipad",
    "tab",
    "s6 lite"
    # ❌ surface pro REMOVED from tablet
]

COMPUTER_KEYWORDS = [
    "macbook", "imac",
    "chromebook", "chrome book",
    "laptop", "pc",
    "lenovo", "hp", "acer", "compaq",
    "hard drive", "os install",
    "windows", "ram install",
    "surface pro"  # ✅ CORRECTED: surface pro = computer
]

WATCH_KEYWORDS = [
    "apple watch", "watch", "series"
]

HEADPHONE_KEYWORDS = [
    "beats", "studio", "headphones", "earbuds"
]

TV_KEYWORDS = [
    "tv", "television"
]

SMOKE_VAPE_KEYWORDS = [
    "geek bar", "geekbar",
    "clickmate", "click mate",
    "lost mary", "lostmary",
    "mango", "mint", "berry", "clear",
    "kamado", "vape",
    "rillos", "back woods", "backwoods",
    "pipe", "bong","hawaiian b"
]


In [127]:
def final_device_type(text):
    if pd.isna(text):
        return "unknown"

    t = text.lower()

    # Retail first (override everything)
    if any(k in t for k in SMOKE_VAPE_KEYWORDS):
        return "smoke_vape"

    # Explicit device classes
    if any(k in t for k in TV_KEYWORDS):
        return "tv"

    if any(k in t for k in HEADPHONE_KEYWORDS):
        return "headphones"

    if any(k in t for k in WATCH_KEYWORDS):
        return "watch"

    if any(k in t for k in COMPUTER_KEYWORDS):
        return "computer"

    if any(k in t for k in TABLET_KEYWORDS):
        return "tablet"

    if any(k in t for k in PHONE_KEYWORDS):
        return "phone"

    return "unknown"


In [128]:
df_doc["device_type_final"] = df_doc["text_clean"].apply(final_device_type)


In [129]:
df_doc[df_doc["device_type_final"] == "unknown"][
    ["text_clean", "net_sales_num", "activity_type"]
].sort_values("net_sales_num", ascending=False)


,text_clean,net_sales_num,activity_type
245,consulting,750.00,unclassified
531,note 20 ultra water damage battery screen and ...,415.00,unclassified
404,note 20 ultra screen repair,370.00,unclassified
670,15 plus front back and motherboard,350.00,unclassified
170,15 front and back repair,300.00,unclassified
128,note 10 plus screen repair,300.00,unclassified
801,computer repair,300.00,unclassified
250,pixel 7a screen repair,230.00,unclassified
142,note 20 ultra screen replacement,220.00,unclassified
689,pixel 8 screen replacement,220.00,unclassified


In [130]:
df_doc[df_doc["device_type_final"] == "smoke_vape"][
    ["text_clean", "net_sales_num"]
].head(26)


,text_clean,net_sales_num
573,hawaiian b,25.0
579,berry,20.0
587,clear,20.0
681,clickmate,30.0
685,click mate,60.0
692,clickmate,30.0
702,2x clickmate,60.0
709,lost mary,30.0
713,clickmate,30.0
720,the last geekbar,30.0


In [131]:
device_final = (
    df_doc.groupby("device_type_final")
    .agg(
        rows=("net_sales_num", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

device_final


,rows,revenue,avg_ticket
device_type_final,,,
phone,609,96379.69,158.258933
tablet,119,21956.62,184.509412
unknown,169,21041.29,124.504675
computer,36,13661.00,379.472222
watch,8,1075.00,134.375000
smoke_vape,25,937.00,37.480000
tv,1,130.00,130.000000
headphones,1,85.00,85.000000


In [132]:
import numpy as np
import pandas as pd

# -----------------------------
# 1) Strong inference from refined model (best source of truth)
# -----------------------------
MODEL_TO_DEVICE = {
    # Phones
    "iPhone": "phone",
    "Galaxy": "phone",
    "Pixel": "phone",
    "OnePlus": "phone",
    "Motorola": "phone",

    # Tablets
    "iPad": "tablet",
    "Galaxy Tab": "tablet",

    # Computers
    "MacBook": "computer",
    "iMac": "computer",
    "Surface Pro": "computer",
    "Chromebook": "computer",

    # Wearables
    "Apple Watch": "watch",

    # Consoles
    "PlayStation": "console",
    "PS5": "console",
    "PS4": "console",
    "Xbox": "console",
}

def device_from_model(model):
    if pd.isna(model):
        return None
    m = str(model)

    # direct known exacts
    if m in ["MacBook", "iMac", "Surface Pro", "Chromebook", "Apple Watch", "PlayStation", "Xbox"]:
        return MODEL_TO_DEVICE.get(m)

    # prefix-based (safe)
    if m.startswith("iPhone"):
        return "phone"
    if m.startswith("Galaxy Note") or m.startswith("Galaxy S") or m.startswith("Galaxy A") or m.startswith("Galaxy Z"):
        return "phone"
    if m.startswith("Pixel"):
        return "phone"
    if m.startswith("iPad"):
        return "tablet"

    return None

# -----------------------------
# 2) Strong keyword inference for rows where model isn't helpful
# -----------------------------
SMOKE_VAPE_KWS = [
    "geek bar", "geekbar", "clickmate", "click mate", "lost mary", "lostmary",
    "vape", "rillos", "backwoods", "back woods", "pipe", "bong",
    "mango", "mint", "berry", "clear", "stone mints", "sour cran"
]

CONSOLE_KWS = ["ps5", "playstation", "ps4", "xbox"]

COMPUTER_KWS = [
    "macbook", "imac", "chromebook", "chrome book", "surface pro",
    "laptop", "lenovo", "hp", "acer", "compaq", "vivobook",
    "hard drive", "os install", "windows", "ram install"
]

TABLET_KWS = ["ipad", "tab", "s6 lite", "tablet"]

PHONE_KWS = [
    "iphone", "pixel", "galaxy", "note", "zflip", "z flip",
    "oneplus", "one plus", "nord", "motorola", "moto", "razr",
    "a03", "a12", "a13", "a14", "a15", "a20", "a32", "a35", "a52", "a53", "a54",
    "s10", "s20", "s21", "s22", "s23", "s24",
    "pm", "pro max", "14pm", "15pm"
]

ACCESSORY_KWS = ["phone case", "case", "screen protector", "protector", "charger", "cable"]

HEADPHONE_KWS = ["beats", "studio 3", "headphones", "earbuds"]

TV_KWS = ["tv", "television"]

WATCH_KWS = ["apple watch", "watch s", "watch series", "series"]

def device_from_text(text):
    if pd.isna(text):
        return "unknown"
    t = str(text).lower()

    # Retail override first
    if any(k in t for k in SMOKE_VAPE_KWS):
        return "smoke_vape"

    # explicit types
    if any(k in t for k in TV_KWS):
        return "tv"
    if any(k in t for k in HEADPHONE_KWS):
        return "headphones"
    if any(k in t for k in WATCH_KWS):
        return "watch"
    if any(k in t for k in CONSOLE_KWS):
        return "console"
    if any(k in t for k in COMPUTER_KWS):
        return "computer"
    if any(k in t for k in TABLET_KWS):
        return "tablet"
    if any(k in t for k in PHONE_KWS):
        return "phone"
    if any(k in t for k in ACCESSORY_KWS):
        return "accessory"

    return "unknown"

# -----------------------------
# 3) Apply: model-first, then text-based fallback
# -----------------------------
# Use your refined model column if present
MODEL_COL = "device_model_exact_refined" if "device_model_exact_refined" in df_doc.columns else None

model_guess = None
if MODEL_COL:
    model_guess = df_doc[MODEL_COL].apply(device_from_model)

df_doc["device_type_final_v2"] = np.where(
    model_guess.notna(),
    model_guess,
    df_doc["text_clean"].apply(device_from_text)
)

# -----------------------------
# 4) Snapshot
# -----------------------------
device_type_v2_summary = (
    df_doc.groupby("device_type_final_v2")["net_sales_num"]
    .agg(rows="count", revenue="sum", avg_ticket="mean")
    .sort_values("revenue", ascending=False)
)

display(device_type_v2_summary)

print("Remaining unknown rows:", (df_doc["device_type_final_v2"] == "unknown").sum())
print("Total rows:", df_doc.shape[0])
print("Revenue check:", df_doc["net_sales_num"].sum())


,rows,revenue,avg_ticket
device_type_final_v2,,,
phone,703,113483.98,161.428137
tablet,117,21656.62,185.099316
computer,33,10416.00,315.636364
unknown,57,5382.00,94.421053
accessory,21,1395.00,66.428571
console,6,1220.00,203.333333
smoke_vape,24,912.00,38.000000
watch,5,585.00,117.000000
tv,1,130.00,130.000000


Remaining unknown rows: 57
Total rows: 968
Revenue check: 155265.6


In [133]:
# Final unknown review – sorted by revenue so you see important ones first
final_unknowns = (
    df_doc[df_doc["device_type_final_v2"] == "unknown"]
    [["text_clean", "net_sales_num", "activity_type"]]
    .sort_values("net_sales_num", ascending=False)
)

display(final_unknowns)

print("Remaining unknown rows:", final_unknowns.shape[0])
print("Unknown revenue:", final_unknowns["net_sales_num"].sum())



,text_clean,net_sales_num,activity_type
245,consulting,750.0,unclassified
801,computer repair,300.0,unclassified
821,lg computer screen replacement,220.0,unclassified
682,ho screen repair,160.0,unclassified
239,back glass replacement,160.0,unclassified
613,battery replacement,155.0,unclassified
113,back glass replacement,150.0,unclassified
714,computer repair soldering on board,150.0,unclassified
209,water damage,150.0,unclassified
717,pc build parts provided and untested,150.0,unclassified


Remaining unknown rows: 57
Unknown revenue: 5382.0


In [134]:
def final_device_type_v3(text):
    t = text.lower()

    # Smoke / Vape
    if "hawaiian b" in t:
        return "smoke_vape"

    # Tablet
    if "s 6 lite" in t:
        return "tablet"

    # Computer
    if any(k in t for k in [
        "computer", "pc ", " pc", "dell", "inspiron",
        "lg computer", "pc build", "pc wiring"
    ]):
        return "computer"

    # Phone (explicit device indicators)
    if any(k in t for k in [
        "back glass",
        "lens",
        "camera lens",
        "ho screen",
        "m g stylus",
        "stylus",
        "dual sim",
        "promax",
        "pro max",
        "one plus",
        "1 plus",
        "samsung 22",
        "ultra",
    ]):
        return "phone"

    # Otherwise keep existing classification
    return "unknown"


# Apply ONLY to current unknowns
mask = df_doc["device_type_final_v2"] == "unknown"
df_doc.loc[mask, "device_type_final_v3"] = (
    df_doc.loc[mask, "text_clean"]
    .apply(final_device_type_v3)
)

# Fill non-unknowns forward
df_doc["device_type_final_v3"] = (
    df_doc["device_type_final_v3"]
    .fillna(df_doc["device_type_final_v2"])
)

# Validation
print(
    df_doc["device_type_final_v3"]
    .value_counts()
)

print("Remaining unknown rows:",
      (df_doc["device_type_final_v3"] == "unknown").sum())

print("Revenue check:",
      df_doc["net_sales_num"].sum())


device_type_final_v3
phone         722
tablet        118
computer       41
unknown        28
smoke_vape     25
accessory      21
console         6
watch           5
headphones      1
tv              1
Name: count, dtype: int64
Remaining unknown rows: 28
Revenue check: 155265.6


In [135]:
df_doc["device_type_final"] = df_doc["device_type_final_v3"]


In [136]:
def infer_brand_from_model(model):
    if model is None:
        return "Unknown"

    m = model.lower()

    if any(k in m for k in ["iphone", "ipad", "mac", "imac", "apple watch", "airpods"]):
        return "Apple"

    if "galaxy" in m or "note" in m or "a" in m and "galaxy" in m:
        return "Samsung"

    if "pixel" in m:
        return "Google"

    if "xbox" in m:
        return "Microsoft"

    if "playstation" in m or "ps" in m:
        return "Sony"

    if "oneplus" in m or "one plus" in m:
        return "OnePlus"

    if "motorola" in m or "moto" in m:
        return "Motorola"

    if "lg" in m:
        return "LG"

    if "lenovo" in m:
        return "Lenovo"

    if "dell" in m:
        return "Dell"

    if "hp" in m:
        return "HP"

    return "Unknown"
def infer_brand_from_model(model):
    if model is None:
        return "Unknown"

    m = model.lower()

    if any(k in m for k in ["iphone", "ipad", "mac", "imac", "apple watch", "airpods"]):
        return "Apple"

    if "galaxy" in m or "note" in m or "a" in m and "galaxy" in m:
        return "Samsung"

    if "pixel" in m:
        return "Google"

    if "xbox" in m:
        return "Microsoft"

    if "playstation" in m or "ps" in m:
        return "Sony"

    if "oneplus" in m or "one plus" in m:
        return "OnePlus"

    if "motorola" in m or "moto" in m:
        return "Motorola"

    if "lg" in m:
        return "LG"

    if "lenovo" in m:
        return "Lenovo"

    if "dell" in m:
        return "Dell"

    if "hp" in m:
        return "HP"

    return "Unknown"


In [137]:
df_doc["brand_final"] = df_doc["device_model_exact_refined"].apply(
    infer_brand_from_model
)

df_doc["brand_final"].value_counts()


brand_final
Apple        645
Samsung      156
Unknown      119
Google        27
Motorola       8
Sony           4
OnePlus        3
HP             3
Microsoft      2
Lenovo         1
Name: count, dtype: int64

In [138]:
mask = (
    (df_doc["brand_final"] == "Unknown") &
    (df_doc["device_type_final"].isin(["phone", "tablet", "computer"]))
)

df_doc.loc[mask, "brand_final"] = (
    df_doc.loc[mask, "text_clean"]
    .str.lower()
    .apply(lambda t:
           "Apple" if "iphone" in t or "ipad" in t or "mac" in t else
           "Samsung" if "samsung" in t or "galaxy" in t else
           "Google" if "pixel" in t else
           "Motorola" if "moto" in t else
           "OnePlus" if "one plus" in t or "oneplus" in t else
           "Unknown")
)


In [139]:
print(df_doc["brand_final"].value_counts())

print("Rows:", df_doc.shape[0])
print("Revenue check:", df_doc["net_sales_num"].sum())


brand_final
Apple        648
Samsung      157
Unknown      114
Google        27
Motorola       9
Sony           4
OnePlus        3
HP             3
Microsoft      2
Lenovo         1
Name: count, dtype: int64
Rows: 968
Revenue check: 155265.6


In [140]:
mask_smoke = df_doc["device_type_final"] == "smoke_vape"
df_doc.loc[mask_smoke, "brand_final"] = "Smoke / Vape"


In [141]:
mask_accessory = df_doc["device_type_final"] == "accessory"
df_doc.loc[mask_accessory, "brand_final"] = "Accessory"


In [142]:
mask_watch = df_doc["device_type_final"] == "watch"
df_doc.loc[mask_watch, "brand_final"] = "Apple"


In [143]:
mask_console = df_doc["device_type_final"] == "console"

df_doc.loc[mask_console & df_doc["text_clean"].str.contains("xbox", case=False), "brand_final"] = "Microsoft"
df_doc.loc[mask_console & df_doc["text_clean"].str.contains("ps|playstation", case=False), "brand_final"] = "Sony"


In [144]:
mask_computer = (df_doc["device_type_final"] == "computer") & (df_doc["brand_final"] == "Unknown")

df_doc.loc[mask_computer, "brand_final"] = (
    df_doc.loc[mask_computer, "text_clean"]
    .str.lower()
    .apply(lambda t:
           "Dell" if "dell" in t else
           "HP" if "hp" in t else
           "Lenovo" if "lenovo" in t else
           "LG" if "lg" in t else
           "Unknown")
)


In [145]:
mask_mobile = (
    (df_doc["brand_final"] == "Unknown") &
    (df_doc["device_type_final"].isin(["phone", "tablet"]))
)

df_doc.loc[mask_mobile, "brand_final"] = (
    df_doc.loc[mask_mobile, "text_clean"]
    .str.lower()
    .apply(lambda t:
           "Apple" if any(k in t for k in ["iphone", "ipad"]) else
           "Samsung" if any(k in t for k in ["samsung", "galaxy"]) else
           "Google" if "pixel" in t else
           "Motorola" if "moto" in t else
           "OnePlus" if "one plus" in t or "oneplus" in t else
           "Unknown")
)


In [146]:
df_doc["brand_final"].value_counts()
print("Remaining Unknown brands:", (df_doc["brand_final"] == "Unknown").sum())
print("Revenue check:", df_doc["net_sales_num"].sum())


Remaining Unknown brands: 63
Revenue check: 155265.6


In [147]:
unknown_brand_df = df_doc[df_doc["brand_final"] == "Unknown"]

unknown_brand_df[
    ["text_clean", "device_type_final", "activity_type", "net_sales_num"]
].value_counts().head(63)


text_clean                                      device_type_final  activity_type  net_sales_num
back glass replacement                          phone              unclassified   100.0            2
                                                                                  105.0            2
battery replacement                             unknown            unclassified   80.0             2
charging port repair                            unknown            unclassified   70.0             2
port replacement                                unknown            unclassified   75.0             1
port removal                                    unknown            unclassified   20.0             1
port clean out                                  unknown            unclassified   45.0             1
pc wiring                                       computer           unclassified   75.0             1
13 promax lens replacement                      phone              unclassified   65.0          

In [148]:
mask_unknown = df_doc["brand_final"] == "Unknown"

df_doc.loc[mask_unknown, "brand_final"] = (
    df_doc.loc[mask_unknown, "text_clean"]
    .str.lower()
    .apply(lambda t:
        "Apple" if any(k in t for k in ["iphone", "ipad", "ios"]) else
        "Samsung" if any(k in t for k in ["samsung", "galaxy", "s6 lite", "a32", "a53", "a15", "s22", "s23"]) else
        "Google" if "pixel" in t else
        "Motorola" if any(k in t for k in ["moto", "razr", "g stylus"]) else
        "OnePlus" if any(k in t for k in ["one plus", "nord"]) else
        "Microsoft" if "surface" in t else
        "Unknown"
    )
)


In [149]:
mask_smoke = df_doc["device_type_final"] == "smoke_vape"
df_doc.loc[mask_smoke, "brand_final"] = "Smoke / Vape"


In [150]:
df_doc["brand_final"].value_counts()
print("Remaining Unknown brands:", (df_doc["brand_final"] == "Unknown").sum())
print("Revenue check:", df_doc["net_sales_num"].sum())


Remaining Unknown brands: 59
Revenue check: 155265.6


In [151]:
def refine_brand_final(text, current_brand):
    if current_brand != "Unknown":
        return current_brand

    t = text.lower()

    # Apple
    if any(k in t for k in [
        "iphone", "promax", "13pm", "15 promax", "ho screen",
        "camera lens", "lens replacement", "back glass"
    ]):
        return "Apple"

    if "beats studio" in t:
        return "Apple"

    # Microsoft
    if "surface pro" in t:
        return "Microsoft"

    # Google
    if "chromebook" in t or "chrome book" in t:
        return "Google"

    # Samsung
    if "s 6 lite" in t or "samsung" in t:
        return "Samsung"

    # Asus
    if "asus" in t:
        return "Asus"

    # Acer
    if "acer" in t:
        return "Acer"

    # HP (Compaq)
    if "compaq" in t:
        return "HP"

    return current_brand


df_doc["brand_final"] = df_doc.apply(
    lambda r: refine_brand_final(r["text_clean"], r["brand_final"]),
    axis=1
)

# Validation
print("Remaining Unknown brands:", (df_doc["brand_final"] == "Unknown").sum())
print("Revenue check:", df_doc["net_sales_num"].sum())


Remaining Unknown brands: 36
Revenue check: 155265.6


In [152]:
# ==============================
# FINAL BRAND SUMMARY (LOCKED)
# ==============================

print("\n==============================")
print("FINAL BRAND SUMMARY")
print("==============================")

brand_final_summary = (
    df_doc.groupby("brand_final")["net_sales_num"]
    .agg(rows="count", revenue="sum", avg_ticket="mean")
    .sort_values("revenue", ascending=False)
)

display(brand_final_summary)

print("\nValidation:")
print("Total rows:", df_doc.shape[0])
print("Remaining Unknown brands:", (df_doc["brand_final"] == "Unknown").sum())
print("Revenue check:", df_doc["net_sales_num"].sum())

# ==============================
# REMAINING UNKNOWN BRANDS (SHOW ROWS)
# ==============================

print("\n==============================")
print("REMAINING UNKNOWN BRANDS (DETAIL)")
print("==============================")

df_unknown_brand = df_doc.loc[df_doc["brand_final"] == "Unknown", [
    "text_clean",
    "device_type_final",
    "device_model_exact_refined",
    "activity_type",
    "net_sales_num",
    "Transaction ID"
]].sort_values("net_sales_num", ascending=False)

print("Unknown brand rows:", df_unknown_brand.shape[0])
print("Unknown brand revenue:", df_unknown_brand["net_sales_num"].sum())

display(df_unknown_brand.head(75))  # show top 75 by $ first

# ==============================
# UNKNOWN BRAND TEXT FREQUENCY + TOKENS
# ==============================

print("\n==============================")
print("UNKNOWN BRAND TEXT FREQUENCY (TOP 50)")
print("==============================")

display(df_unknown_brand["text_clean"].value_counts().head(50))

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=2)
X = cv.fit_transform(df_unknown_brand["text_clean"])

token_df = (
    pd.DataFrame({"token": cv.get_feature_names_out(), "count": X.sum(axis=0).A1})
    .sort_values("count", ascending=False)
)

print("\n==============================")
print("UNKNOWN BRAND TOKENS (TOP 40)")
print("==============================")
display(token_df.head(40))



FINAL BRAND SUMMARY


,rows,revenue,avg_ticket
brand_final,,,
Apple,667,108841.11,163.180075
Samsung,158,30873.49,195.401835
Google,29,4875.00,168.103448
Unknown,36,3947.00,109.638889
Accessory,21,1395.00,66.428571
Microsoft,6,1230.00,205.000000
Motorola,9,1015.00,112.777778
Smoke / Vape,25,937.00,37.480000
Sony,4,900.00,225.000000



Validation:
Total rows: 968
Remaining Unknown brands: 36
Revenue check: 155265.6

REMAINING UNKNOWN BRANDS (DETAIL)
Unknown brand rows: 36
Unknown brand revenue: 3947.0


,text_clean,device_type_final,device_model_exact_refined,activity_type,net_sales_num,Transaction ID
245,consulting,unknown,Unknown,unclassified,750.0,klkM2JUHEz423GCkW4Tfs73eV
801,computer repair,computer,Computer (Generic),unclassified,300.0,LQbNaEGRxnhIMCxVthcXkMOrtVPZY
635,1tb hard drive install and os install,computer,Unknown,unclassified,240.0,7oTRA7TyZdc5iPCppp9X6u2uFgbZY
867,ram install and system update,computer,Unknown,unclassified,230.0,PgKlyJlAgHDTo3Q73bUJzpsBkSIZY
613,battery replacement,unknown,Unknown,unclassified,155.0,l1JuoTjz1qcZsekWqpUOf3w6fXEZY
209,water damage,unknown,Unknown,unclassified,150.0,72Ym0rzAOnhyODibsAvADMAEPDaZY
714,computer repair soldering on board,computer,Computer (Generic),unclassified,150.0,TeK8Te8UJXr3NPvetISm47k5RT7YY
717,pc build parts provided and untested,computer,PC,unclassified,150.0,VNhaGy2FoI1vzNwZzUIWwzmlH2VZY
723,windows re installation,computer,Unknown,unclassified,150.0,1tLTOOTVNh9nK9i1tnmKvyp5eRdZY
952,tv repair,tv,Unknown,unclassified,130.0,B546kcuWttbqRKeIBfPNKrIdGTRZY



UNKNOWN BRAND TEXT FREQUENCY (TOP 50)


text_clean
battery replacement                               4
charging port repair                              3
diagnostics                                       3
port replacement                                  3
consulting                                        1
tip for kevin                                     1
port removal                                      1
port clean out                                    1
tablet install                                    1
battery replacement one year warranty included    1
service call                                      1
pc service                                        1
speaker repair                                    1
dual sim installation                             1
computer screen repair deposit                    1
pc wiring                                         1
computer repair                                   1
lcd repair                                        1
repair                                            1
t


UNKNOWN BRAND TOKENS (TOP 40)


,token,count
9,repair,10
8,port,8
10,replacement,8
1,battery,5
5,install,4
0,and,3
2,charging,3
3,computer,3
4,diagnostics,3
7,pc,3


In [153]:
def assign_repair_category(text, activity):
    t = str(text).lower()

    # ------------------------
    # RETAIL & ADMIN (FIRST)
    # ------------------------
    if activity in ["retail_smoke_vape"]:
        return "Retail – Smoke / Vape"

    if activity in ["retail_accessory"]:
        return "Retail – Accessory"

    if activity in ["admin_payment"]:
        return "Admin / Fees / Deposits"

    # ------------------------
    # SERVICE (NON-REPAIR)
    # ------------------------
    if activity in ["service"]:
        if any(k in t for k in ["diagnostic", "diagnostics"]):
            return "Diagnostics"
        if any(k in t for k in ["install", "setup", "software", "os", "windows"]):
            return "Software / Setup"
        if any(k in t for k in ["consult", "service call"]):
            return "Service Call / Consulting"
        return "Non-Repair / Unknown"

    # ------------------------
    # REPAIR CATEGORIES
    # ------------------------
    if any(k in t for k in ["screen", "lcd", "display"]):
        return "Screen Repair"

    if any(k in t for k in ["battery"]):
        return "Battery Repair"

    if any(k in t for k in ["back glass"]):
        return "Back Glass Repair"

    if any(k in t for k in ["charging port", "port replacement", "port repair", "port clean"]):
        return "Charging Port Repair"

    if any(k in t for k in ["camera", "lens"]):
        return "Camera Repair"

    if any(k in t for k in ["water damage", "liquid"]):
        return "Water Damage Repair"

    if any(k in t for k in ["board", "solder"]):
        return "Board-Level Repair"

    if any(k in t for k in ["speaker", "mic"]):
        return "Speaker / Mic Repair"

    if activity == "repair":
        return "Other Device Repair"

    return "Non-Repair / Unknown"


In [154]:
df_doc["repair_category_final"] = df_doc.apply(
    lambda r: assign_repair_category(r["text_clean"], r["activity_type"]),
    axis=1
)

df_doc["is_repair"] = df_doc["repair_category_final"].isin([
    "Screen Repair",
    "Battery Repair",
    "Back Glass Repair",
    "Charging Port Repair",
    "Camera Repair",
    "Water Damage Repair",
    "Board-Level Repair",
    "Speaker / Mic Repair",
    "Other Device Repair"
])


In [155]:
df_doc.groupby("repair_category_final").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean")
).sort_values("revenue", ascending=False)


,rows,revenue,avg_ticket
repair_category_final,,,
Screen Repair,604,106086.21,175.639421
Non-Repair / Unknown,192,28689.83,149.426198
Battery Repair,68,7163.29,105.342500
Back Glass Repair,38,5530.27,145.533421
Charging Port Repair,33,3175.00,96.212121
Board-Level Repair,6,1846.00,307.666667
Camera Repair,19,1330.00,70.000000
Water Damage Repair,5,1250.00,250.000000
Speaker / Mic Repair,3,195.00,65.000000


In [156]:
df_doc.groupby("is_repair").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean")
)


,rows,revenue,avg_ticket
is_repair,,,
False,192,28689.83,149.426198
True,776,126575.77,163.113106


In [157]:
df_doc.groupby("multi_repair_flag").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean")
)


,rows,revenue,avg_ticket
multi_repair_flag,,,
False,929,146506.73,157.703692
True,39,8758.87,224.586410


In [158]:
device_repair_matrix = (
    df_doc[df_doc["is_repair"]]
    .groupby(["device_type_final", "repair_category_final"])
    .agg(
        rows=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

device_repair_matrix


rows   revenue  avg_ticket
device_type_final repair_category_final                            
phone             Screen Repair           477  81378.21  170.604214
tablet            Screen Repair            90  17866.00  198.511111
phone             Battery Repair           52   5518.29  106.120962
                  Back Glass Repair        37   5360.27  144.872162
computer          Screen Repair            18   4747.00  263.722222
phone             Charging Port Repair     17   1560.00   91.764706
computer          Board-Level Repair        4   1246.00  311.500000
phone             Camera Repair            17   1230.00   72.352941
tablet            Battery Repair            9   1055.00  117.222222
accessory         Screen Repair            15   1045.00   69.666667
phone             Water Damage Repair       3    820.00  273.333333
tablet            Charging Port Repair      5    790.00  158.000000
phone             Board-Level Repair        2    600.00  300.000000
unknown           Charging Port Repair      8    570.00   71.250000
console           Screen Repair             1    555.00  555.000000
unknown           Battery Repair            5    425.00   85.000000
watch             Screen Repair             2    395.00  197.500000
computer          Water Damage Repair       1    280.00  280.000000
console           Charging Port Repair      2    205.00  102.500000
accessory         Back Glass Repair         1    170.00  170.000000
unknown           Water Damage Repair       1    150.00  150.000000
phone             Speaker / Mic Repair      2    145.00   72.500000
accessory         Camera Repair             2    100.00   50.000000
unknown           Screen Repair             1    100.00  100.000000
headphones        Battery Repair            1     85.00   85.000000
watch             Battery Repair            1     80.00   80.000000
accessory         Charging Port Repair      1     50.00   50.000000
unknown           Speaker / Mic Repair      1     50.00   50.000000

In [159]:
phone_repair_breakdown = (
    df_doc[
        (df_doc["device_type_final"] == "phone") &
        (df_doc["is_repair"])
    ]
    .groupby("repair_category_final")
    .agg(
        rows=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

phone_repair_breakdown


,rows,revenue,avg_ticket
repair_category_final,,,
Screen Repair,477,81378.21,170.604214
Battery Repair,52,5518.29,106.120962
Back Glass Repair,37,5360.27,144.872162
Charging Port Repair,17,1560.00,91.764706
Camera Repair,17,1230.00,72.352941
Water Damage Repair,3,820.00,273.333333
Board-Level Repair,2,600.00,300.000000
Speaker / Mic Repair,2,145.00,72.500000


In [160]:
df_doc[df_doc["is_repair"]].groupby("multi_repair_flag").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean"),
    median_ticket=("net_sales_num", "median")
)


,rows,revenue,avg_ticket,median_ticket
multi_repair_flag,,,,
False,737,117816.90,159.860109,130.0
True,39,8758.87,224.586410,170.0


In [161]:
top_repairs = (
    df_doc[df_doc["is_repair"]]
    .groupby("repair_category_final")["net_sales_num"]
    .sum()
    .sort_values(ascending=False)
)

(top_repairs / top_repairs.sum()).cumsum()


repair_category_final
Screen Repair           0.838124
Battery Repair          0.894717
Back Glass Repair       0.938408
Charging Port Repair    0.963492
Board-Level Repair      0.978076
Camera Repair           0.988584
Water Damage Repair     0.998459
Speaker / Mic Repair    1.000000
Name: net_sales_num, dtype: float64

In [162]:
REPAIR_SERVICE_TOKENS = {
    "screen": {
        "screen", "lcd", "display", "glass screen"
    },
    "battery": {
        "battery", "batt"
    },
    "back_glass": {
        "back glass", "rear glass", "rear"
    },
    "charging_port": {
        "port", "charging port", "usb", "lightning"
    },
    "camera": {
        "camera", "lens"
    },
    "speaker_mic": {
        "speaker", "mic", "microphone"
    },
    "water_damage": {
        "water", "liquid", "moisture"
    },
    "board_level": {
        "board", "motherboard", "logic board", "solder"
    },
    "diagnostic": {
        "diagnostic", "diagnostics", "diag", "inspection"
    },
    "software_setup": {
        "install", "installation", "setup", "reset",
        "restore", "unlock", "update", "software"
    },
    "service_call": {
        "consulting", "service call"
    }
}


In [163]:
# ======================================================================================
# BUILD CANONICAL TOKEN COLUMN (REQUIRED FOR EVENT EXTRACTION)
# ======================================================================================

df_doc["tokens"] = (
    df_doc["text_clean"]
    .fillna("")
    .str.lower()
    .str.split()
)

# Sanity check
df_doc["tokens"].head()


0              [iphone, 8, plus, front, and, back, 2x]
1                           [pixel, 7, screen, repair]
2                           [a53, screen, replacement]
3    [iphone, 11, screen, repair, mobile, christmas...
4                     [imac, 2022, with, ram, upgrade]
Name: tokens, dtype: object

In [164]:
# ======================================================================================
# PHASE 3B — ROW-LEVEL EVENT DETECTION (ADDITIVE)
# ======================================================================================

for category, token_set in REPAIR_SERVICE_TOKENS.items():
    df_doc[f"event_{category}"] = df_doc["tokens"].apply(
        lambda t: any(tok in t for tok in token_set)
    )

# Quick validation
df_doc[[c for c in df_doc.columns if c.startswith("event_")]].sum()


event_screen            604
event_battery            85
event_back_glass          0
event_charging_port      60
event_camera             23
event_speaker_mic         3
event_water_damage       11
event_board_level         6
event_diagnostic         15
event_software_setup     14
event_service_call        1
dtype: int64

In [165]:
# ======================================================================================
# PHASE 3C — EVENT AUDIT (READ-ONLY)
# ======================================================================================

event_summary = []

for category in REPAIR_SERVICE_TOKENS:
    rows = df_doc[f"event_{category}"].sum()
    revenue = df_doc.loc[df_doc[f"event_{category}"], "net_sales_num"].sum()
    
    event_summary.append({
        "event": category,
        "rows_with_event": rows,
        "revenue_exposed": revenue
    })

event_summary_df = (
    pd.DataFrame(event_summary)
    .sort_values("rows_with_event", ascending=False)
)

display(event_summary_df)


,event,rows_with_event,revenue_exposed
0,screen,604,106086.21
1,battery,85,10478.29
3,charging_port,60,7075.00
4,camera,23,2930.00
8,diagnostic,15,820.00
9,software_setup,14,2230.00
6,water_damage,11,2885.00
7,board_level,6,1786.00
5,speaker_mic,3,195.00
10,service_call,1,750.00


In [166]:
# ======================================================================================
# PHASE 3D — UNKNOWN EVENT EXTRACTION
# ======================================================================================

EVENT_COLUMNS = [f"event_{k}" for k in REPAIR_SERVICE_TOKENS]

df_doc["has_any_event"] = df_doc[EVENT_COLUMNS].any(axis=1)

unknown_event_df = df_doc[
    (df_doc["is_repair"] | df_doc["activity_type"].isin(["service"])) &
    (~df_doc["has_any_event"])
][[
    "source_year",
    "text_clean",
    "device_type_final",
    "brand_final",
    "repair_category_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False)

print(f"Unknown repair/service rows: {len(unknown_event_df)}")
print(f"Unknown repair/service revenue: ${unknown_event_df['net_sales_num'].sum():,.2f}")

display(unknown_event_df.head(60))


Unknown repair/service rows: 38
Unknown repair/service revenue: $5,700.27


,source_year,text_clean,device_type_final,brand_final,repair_category_final,net_sales_num
238,2024,macbook a2338 keyboard repair,computer,Apple,Board-Level Repair,500.00
881,2025,iphone 15 front and back glass repair,phone,Apple,Back Glass Repair,240.00
820,2025,iphone 13 front and back glass,phone,Apple,Back Glass Repair,210.00
34,2024,14 pro back glass replacement,phone,Apple,Back Glass Repair,200.00
212,2024,14 pro max back glass repair,phone,Apple,Back Glass Repair,198.27
785,2025,14promax back glass and case,accessory,Accessory,Back Glass Repair,170.00
408,2024,14 pro back glass replacement,phone,Apple,Back Glass Repair,170.00
431,2024,iphone 13 pro max back glass replacement,phone,Apple,Back Glass Repair,170.00
64,2024,14 promax back glass repair,phone,Apple,Back Glass Repair,162.00
933,2025,14 pro back glass replacement,phone,Apple,Back Glass Repair,160.00


In [167]:
# ======================================================================================
# PHASE 4A — IMPLICIT SCREEN RECOVERY (FRONT / FRONT+BACK)
# ======================================================================================

FRONT_SCREEN_TOKENS = {"front"}
BACK_ONLY_EXCLUDES = {"back glass"}

df_doc["has_front_token"] = df_doc["text_clean"].str.contains(r"\bfront\b", regex=True)

df_doc["front_back_screen_recovery"] = (
    df_doc["has_front_token"] &
    ~df_doc["event_screen"] &
    df_doc["is_repair"]
)

recovered_screen_rows = df_doc["front_back_screen_recovery"].sum()
recovered_screen_revenue = df_doc.loc[
    df_doc["front_back_screen_recovery"],
    "net_sales_num"
].sum()

print(f"Recovered implicit screen rows: {recovered_screen_rows}")
print(f"Recovered implicit screen revenue: ${recovered_screen_revenue:,.2f}")

# Apply recovery
df_doc.loc[df_doc["front_back_screen_recovery"], "event_screen"] = True


Recovered implicit screen rows: 5
Recovered implicit screen revenue: $1,520.00


In [168]:
# ======================================================================================
# PHASE 4B — GENERIC REPAIR EVENT (LAST RESORT)
# ======================================================================================

df_doc["event_generic_repair"] = (
    df_doc["is_repair"] &
    ~df_doc[[c for c in df_doc.columns if c.startswith("event_")]].any(axis=1)
)

generic_rows = df_doc["event_generic_repair"].sum()
generic_revenue = df_doc.loc[df_doc["event_generic_repair"], "net_sales_num"].sum()

print(f"Generic repair rows (no part specificity): {generic_rows}")
print(f"Generic repair revenue: ${generic_revenue:,.2f}")


Generic repair rows (no part specificity): 36
Generic repair revenue: $5,250.27


In [169]:
final_event_summary = []

for col in sorted([c for c in df_doc.columns if c.startswith("event_")]):
    final_event_summary.append({
        "event": col.replace("event_", ""),
        "rows": df_doc[col].sum(),
        "revenue": df_doc.loc[df_doc[col], "net_sales_num"].sum()
    })

final_event_df = (
    pd.DataFrame(final_event_summary)
    .sort_values("revenue", ascending=False)
)

display(final_event_df)


,event,rows,revenue
7,screen,609,107606.21
1,battery,85,10478.29
4,charging_port,60,7075.00
6,generic_repair,36,5250.27
3,camera,23,2930.00
11,water_damage,11,2885.00
9,software_setup,14,2230.00
2,board_level,6,1786.00
5,diagnostic,15,820.00
8,service_call,1,750.00


In [170]:
# ======================================================================================
# PHASE 5A — TOKEN INVENTORY (AUTHORITATIVE)
# ======================================================================================

from collections import Counter

token_counter = Counter()

df_doc["text_clean"].str.split().apply(token_counter.update)

token_inventory = (
    pd.DataFrame(token_counter.items(), columns=["token", "count"])
    .sort_values("count", ascending=False)
)

display(token_inventory.head(50))


,token,count
9,screen,614
10,repair,572
0,iphone,327
23,pro,194
12,replacement,169
62,ipad,121
32,14,100
61,max,98
35,13,97
13,11,93


In [171]:
# ======================================================================================
# PHASE 6A — BACK GLASS TOKEN DEFINITION
# ======================================================================================

BACK_GLASS_CORE = {"glass"}
BACK_GLASS_POSITION = {"back", "rear"}
BACK_GLASS_EXCLUDES = {"woods", "rillos", "dutch", "backwoods"}

def is_back_glass(tokens):
    return (
        any(t in tokens for t in BACK_GLASS_POSITION)
        and any(t in tokens for t in BACK_GLASS_CORE)
        and not any(t in tokens for t in BACK_GLASS_EXCLUDES)
    )


In [172]:
# ======================================================================================
# PHASE 6B — BACK GLASS EVENT RECOVERY
# ======================================================================================

df_doc["event_back_glass"] = df_doc["text_clean"].str.split().apply(is_back_glass)

rows_bg = df_doc["event_back_glass"].sum()
rev_bg = df_doc.loc[df_doc["event_back_glass"], "net_sales_num"].sum()

print(f"Back glass rows recovered: {rows_bg}")
print(f"Back glass revenue recovered: ${rev_bg:,.2f}")


Back glass rows recovered: 49
Back glass revenue recovered: $8,619.14


In [173]:
# ======================================================================================
# PHASE 7 — EVENT HIERARCHY ENFORCEMENT
# ======================================================================================

EVENT_PRIORITY = [
    "back_glass",
    "screen",
    "battery",
    "charging_port",
    "camera",
    "water_damage",
    "board_level",
    "software_setup",
    "diagnostic",
    "speaker_mic",
    "service_call",
    "generic_repair"
]

event_cols = [f"event_{e}" for e in EVENT_PRIORITY]

df_doc["final_event"] = None

for e in EVENT_PRIORITY:
    mask = (df_doc[f"event_{e}"]) & (df_doc["final_event"].isna())
    df_doc.loc[mask, "final_event"] = e


In [174]:
final_event_summary = (
    df_doc
    .groupby("final_event")
    .agg(
        rows=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

display(final_event_summary)


,rows,revenue,avg_ticket
final_event,,,
screen,601,104777.34,174.338336
back_glass,49,8619.14,175.900816
battery,62,6043.29,97.472419
charging_port,44,4065.00,92.386364
software_setup,11,1375.00,125.000000
camera,18,1255.00,69.722222
water_damage,5,1060.00,212.000000
board_level,3,896.00,298.666667
service_call,1,750.00,750.000000


In [175]:
# ======================================================================================
# PHASE 9A — SECONDARY EVENT TOKEN SETS
# ======================================================================================

SECONDARY_TOKENS = {
    "back_glass": {
        "back", "rear", "housing"
    },
    "battery": {
        "battery", "batt"
    },
    "charging_port": {
        "port", "charging"
    },
    "camera": {
        "camera", "lens"
    },
    "screen": {
        "screen", "lcd", "display"
    },
    "water_damage": {
        "water", "liquid", "corrosion"
    }
}


In [176]:
# ======================================================================================
# PHASE 9B — SECONDARY EVENT FLAGS
# ======================================================================================

for event, tokenset in SECONDARY_TOKENS.items():
    df_doc[f"secondary_{event}"] = df_doc["text_clean"].str.split().apply(
        lambda t: any(tok in t for tok in tokenset)
    )


In [177]:
# ======================================================================================
# PHASE B1 — EXPANDED REPAIR / SERVICE TOKEN MAP
# ======================================================================================

SECONDARY_REPAIR_SERVICE_TOKENS = {

    # -------------------------
    # PHONE / DEVICE HARDWARE
    # -------------------------
    "screen": {"screen", "lcd", "display"},
    "back_glass": {"back", "rear", "housing", "glass"},
    "battery": {"battery", "batt"},
    "charging_port": {"port", "charging", "usb", "typec", "lightning"},
    "camera": {"camera", "lens"},
    "speaker_mic": {"speaker", "mic", "microphone", "earpiece"},
    "proximity_sensor": {"proximity", "sensor", "faceid", "face", "flex"},
    "button": {"button", "power", "volume"},
    "antenna": {"antenna", "signal"},
    "board_level": {"motherboard", "board", "ic", "solder", "chip"},

    # -------------------------
    # LIQUID / DAMAGE
    # -------------------------
    "water_damage": {"water", "liquid", "corrosion"},

    # -------------------------
    # COMPUTER / TABLET SERVICES
    # -------------------------
    "software_setup": {
        "install", "installation", "setup", "reset", "restore",
        "update", "unlock", "os", "windows", "macos"
    },
    "diagnostic": {"diagnostic", "diagnostics", "diag", "inspection"},
    "hardware_install": {
        "ram", "ssd", "harddrive", "drive", "storage"
    },
    "pc_wiring": {"wiring", "cable", "connector"},
    "cleaning_service": {"clean", "cleanup", "cleaning"},
    
    # -------------------------
    # ADMIN / FEES
    # -------------------------
    "deposit": {"deposit"},
    "fee": {"fee", "afterpay", "tip"}
}


In [178]:
# ======================================================================================
# PHASE B2 — APPLY SECONDARY EVENT FLAGS
# ======================================================================================

for event, tokenset in SECONDARY_REPAIR_SERVICE_TOKENS.items():
    df_doc[f"secondary_{event}"] = df_doc["text_clean"].str.split().apply(
        lambda t: any(tok in t for tok in tokenset)
    )


In [179]:
# ======================================================================================
# PHASE C — TRUE EVENT COVERAGE SUMMARY
# ======================================================================================

coverage = []

for event in SECONDARY_REPAIR_SERVICE_TOKENS:
    rows = df_doc[f"secondary_{event}"].sum()
    revenue = df_doc.loc[df_doc[f"secondary_{event}"], "net_sales_num"].sum()
    
    coverage.append({
        "event": event,
        "rows": rows,
        "revenue": revenue
    })

coverage_df = (
    pd.DataFrame(coverage)
    .sort_values("rows", ascending=False)
)

coverage_df


,event,rows,revenue
0,screen,604,106086.21
2,battery,85,10478.29
1,back_glass,84,14932.14
3,charging_port,61,7205.00
4,camera,23,2930.00
16,deposit,16,1837.00
12,diagnostic,15,820.00
11,software_setup,14,2230.00
10,water_damage,11,2885.00
15,cleaning_service,9,770.00


In [180]:
df_doc["tokens"] = df_doc["text_clean"].str.split()


In [181]:
 df_doc["tokens"].head()



0              [iphone, 8, plus, front, and, back, 2x]
1                           [pixel, 7, screen, repair]
2                           [a53, screen, replacement]
3    [iphone, 11, screen, repair, mobile, christmas...
4                     [imac, 2022, with, ram, upgrade]
Name: tokens, dtype: object

In [182]:
EVENT_TOKEN_MAP = {
    "screen": {
        "screen", "lcd", "display", "touch", "digitizer"
    },

    "battery": {
        "battery", "batt"
    },

    "back_glass": {
        "back", "rear", "glass"
    },

    "charging_port": {
        "port", "charging"
    },

    "camera": {
        "camera", "lens"
    },

    "speaker_mic": {
        "speaker", "mic", "microphone", "earpiece"
    },

    "proximity_sensor": {
        "proximity", "sensor", "faceid", "face", "ear"
    },

    "button": {
        "button", "power", "volume", "home"
    },

    "antenna": {
        "antenna", "signal", "service", "no", "bars"
    },

    "board_level": {
        "board", "motherboard", "micro", "microsoldering",
        "solder", "hdmi", "ic", "chip"
    },

    "water_damage": {
        "water", "liquid", "wet", "corrosion"
    },

    "diagnostic": {
        "diagnostic", "diagnostics", "diag", "inspection", "check"
    },

    "software_setup": {
        "install", "installation", "setup", "reset", "restore",
        "update", "unlock", "software"
    },

    "pc_wiring": {
        "wiring", "wire", "cable"
    },

    "hardware_install": {
        "ram", "ssd", "harddrive", "drive", "storage"
    },

    "deposit": {
        "deposit", "paid"
    },

    "fee": {
        "fee", "afterpay", "tip"
    }
}


In [183]:
def has_any_token(tokens, token_set):
    return any(t in token_set for t in tokens)

for event, token_set in EVENT_TOKEN_MAP.items():
    df_doc[f"event_{event}"] = df_doc["tokens"].apply(
        lambda t: has_any_token(t, token_set)
    )


In [184]:
BACK_GLASS_EXCLUDES = {"woods", "backwoods", "rillos", "dutch"}

df_doc["event_back_glass"] = (
    df_doc["event_back_glass"] &
    ~df_doc["tokens"].apply(lambda t: has_any_token(t, BACK_GLASS_EXCLUDES))
)


In [185]:
event_summary = []

for event in EVENT_TOKEN_MAP:
    rows = df_doc[f"event_{event}"].sum()
    revenue = df_doc.loc[df_doc[f"event_{event}"], "net_sales_num"].sum()

    event_summary.append({
        "event": event,
        "rows": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

event_summary_df = (
    pd.DataFrame(event_summary)
    .sort_values("revenue", ascending=False)
)

display(event_summary_df)


,event,rows,revenue,avg_ticket
0,screen,604,106086.21,175.639421
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
3,charging_port,61,7205.00,118.114754
14,hardware_install,5,3830.00,766.000000
4,camera,23,2930.00,127.391304
10,water_damage,11,2885.00,262.272727
9,board_level,12,2596.00,216.333333
15,deposit,19,2309.00,121.526316
12,software_setup,14,2230.00,159.285714


In [186]:
event_cols = [f"event_{e}" for e in EVENT_TOKEN_MAP]

df_doc["has_any_event"] = df_doc[event_cols].any(axis=1)

generic_candidates = df_doc[
    df_doc["is_repair"] & ~df_doc["has_any_event"]
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]]

generic_candidates.sort_values("net_sales_num", ascending=False).head(60)


,text_clean,device_type_final,brand_final,net_sales_num
238,macbook a2338 keyboard repair,computer,Apple,500.0
439,macbook a2338 keyboard repair,computer,Apple,100.0


In [187]:
event_cols = [c for c in df_doc.columns if c.startswith("event_")]

remaining_unknown_jobs = df_doc[
    df_doc["is_repair"] &
    ~df_doc[event_cols].any(axis=1)
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False)

remaining_unknown_jobs.head(50)


,text_clean,device_type_final,brand_final,net_sales_num


In [188]:
FINAL_EVENT_PRIORITY = [
    "screen",
    "battery",
    "back_glass",
    "charging_port",
    "camera",
    "board_level",
    "proximity_sensor",
    "button",
    "antenna",
    "water_damage",
    "software_setup",
    "diagnostic",
    "hardware_install",
    "pc_wiring",
    "cleaning_service",
    "service_call",
    "deposit",
    "fee",
]

def assign_final_event(row):
    for event in FINAL_EVENT_PRIORITY:
        if row.get(f"event_{event}", False):
            return event
    if row["is_repair"] and row["device_type_final"] != "unknown":
        return "generic_repair_device_only"
    return "unclassified_repair_or_service"

df_doc["final_event"] = df_doc.apply(assign_final_event, axis=1)


In [189]:
df_doc["final_event"].value_counts(dropna=False)


final_event
screen                            604
unclassified_repair_or_service    120
battery                            68
back_glass                         55
charging_port                      43
camera                             17
software_setup                     11
deposit                            11
diagnostic                          8
antenna                             7
water_damage                        5
generic_repair_device_only          4
board_level                         4
button                              2
pc_wiring                           2
fee                                 2
hardware_install                    2
proximity_sensor                    2
service_call                        1
Name: count, dtype: int64

In [190]:
df_doc.groupby("final_event")["net_sales_num"].sum().sum()


155265.6

In [191]:
final_event_summary = (
    df_doc
    .groupby("final_event")
    .agg(
        rows=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

final_event_summary["revenue_share"] = (
    final_event_summary["revenue"] /
    final_event_summary["revenue"].sum()
)

final_event_summary


,rows,revenue,avg_ticket,revenue_share
final_event,,,,
screen,604,106086.21,175.639421,0.683256
unclassified_repair_or_service,120,17303.83,144.198583,0.111447
back_glass,55,9484.27,172.441273,0.061084
battery,68,7163.29,105.342500,0.046136
charging_port,43,4020.00,93.488372,0.025891
hardware_install,2,3260.00,1630.000000,0.020996
software_setup,11,1375.00,125.000000,0.008856
camera,17,1135.00,66.764706,0.007310
water_damage,5,1060.00,212.000000,0.006827


In [192]:
df_doc[
    df_doc["final_event"] == "unclassified_repair_or_service"
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False).head(40)


,text_clean,device_type_final,brand_final,net_sales_num
334,10 macbooks pro 15 inch,phone,Apple,3000.0
721,ipad charge ports,tablet,Apple,450.0
611,iphone 11 pro max,phone,Apple,350.0
203,14 pro fully repair,phone,Apple,350.0
199,15 pro max housing,phone,Apple,320.0
882,s24 ultra,phone,Samsung,320.0
801,computer repair,computer,Unknown,300.0
353,ipad pro repair,tablet,Apple,250.0
388,ipad 12 9 4th gen repair,tablet,Apple,250.0
794,ipad air 4,tablet,Apple,250.0


In [193]:
df_doc.groupby("multi_repair_flag").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean"),
    median_ticket=("net_sales_num", "median")
)


,rows,revenue,avg_ticket,median_ticket
multi_repair_flag,,,,
False,929,146506.73,157.703692,130.0
True,39,8758.87,224.586410,170.0


In [194]:
RETAIL_TOKENS = {
    "case", "cases", "charger", "otterbox",
    "geek", "geekbar", "lost", "mary", "vape", "smoke"
}

df_doc["is_retail_noise"] = (
    df_doc["final_event"] == "unclassified_repair_or_service"
) & df_doc["text_clean"].str.contains(
    "|".join(RETAIL_TOKENS),
    regex=True
)

df_doc.loc[
    df_doc["is_retail_noise"],
    "final_event"
] = "retail_accessory_or_smoke"


In [195]:
df_doc["final_event"].value_counts()


final_event
screen                            604
unclassified_repair_or_service     99
battery                            68
back_glass                         55
charging_port                      43
retail_accessory_or_smoke          21
camera                             17
software_setup                     11
deposit                            11
diagnostic                          8
antenna                             7
water_damage                        5
generic_repair_device_only          4
board_level                         4
button                              2
pc_wiring                           2
fee                                 2
hardware_install                    2
proximity_sensor                    2
service_call                        1
Name: count, dtype: int64

In [196]:
# Coverage
df_doc["final_event"].isna().sum()

# Revenue lock
df_doc.groupby("final_event")["net_sales_num"].sum().sum()

# Multi-repair sanity
df_doc.groupby("multi_repair_flag").agg(
    rows=("Transaction ID", "count"),
    revenue=("net_sales_num", "sum"),
    avg_ticket=("net_sales_num", "mean")
)



,rows,revenue,avg_ticket
multi_repair_flag,,,
False,929,146506.73,157.703692
True,39,8758.87,224.586410


In [197]:
AUDIO_TOKENS = {
    "speaker",
    "speakers",
    "mic",
    "microphone",
    "earpiece",
    "loudspeaker",
    "audio",
    "sound"
}


In [198]:
df_doc["event_audio_repair"] = df_doc["text_clean"].str.contains(
    "|".join(AUDIO_TOKENS),
    regex=True
)


In [199]:
df_doc.loc[
    (df_doc["event_audio_repair"]) &
    (df_doc["final_event"].isin([
        "generic_repair_device_only",
        "unclassified_repair_or_service"
    ])),
    "final_event"
] = "audio_repair"


In [200]:
df_doc["final_event"].value_counts()


final_event
screen                            604
unclassified_repair_or_service     98
battery                            68
back_glass                         55
charging_port                      43
retail_accessory_or_smoke          21
camera                             17
software_setup                     11
deposit                            11
diagnostic                          8
antenna                             7
water_damage                        5
board_level                         4
audio_repair                        3
hardware_install                    2
pc_wiring                           2
generic_repair_device_only          2
fee                                 2
button                              2
proximity_sensor                    2
service_call                        1
Name: count, dtype: int64

In [201]:
HOUSING_TOKENS = {"housing", "frame", "chassis"}

df_doc.loc[
    (df_doc["final_event"] == "generic_repair_device_only") &
    (df_doc["text_clean"].str.contains("|".join(HOUSING_TOKENS))),
    "final_event"
] = "housing_frame"


In [202]:
unclassified = df_doc[
    df_doc["final_event"] == "unclassified_repair_or_service"
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False)

unclassified


,text_clean,device_type_final,brand_final,net_sales_num
334,10 macbooks pro 15 inch,phone,Apple,3000.00
721,ipad charge ports,tablet,Apple,450.00
611,iphone 11 pro max,phone,Apple,350.00
203,14 pro fully repair,phone,Apple,350.00
199,15 pro max housing,phone,Apple,320.00
882,s24 ultra,phone,Samsung,320.00
801,computer repair,computer,Unknown,300.00
794,ipad air 4,tablet,Apple,250.00
388,ipad 12 9 4th gen repair,tablet,Apple,250.00
353,ipad pro repair,tablet,Apple,250.00


In [203]:
generic_device_only = df_doc[
    df_doc["final_event"] == "generic_repair_device_only"
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False)

generic_device_only.head(94)


,text_clean,device_type_final,brand_final,net_sales_num
238,macbook a2338 keyboard repair,computer,Apple,500.0
439,macbook a2338 keyboard repair,computer,Apple,100.0


In [204]:
mask_generic = df_doc["final_event"] == "generic_repair_device_only"


In [205]:
HARDWARE_TOKENS = {
    "keyboard", "ram", "install", "replacement", "pc build"
}

df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("|".join(HARDWARE_TOKENS)),
    "final_event"
] = "hardware_install"


In [206]:
df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("charge port|charging port"),
    "final_event"
] = "charging_port"


In [207]:
df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("housing"),
    "final_event"
] = "back_glass"


In [208]:
df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("front and back|front back"),
    "final_event"
] = "screen"


In [209]:
df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("front and back|front back"),
    "final_event"
] = "screen"


In [210]:
df_doc.loc[
    mask_generic &
    (df_doc["device_type_final"] == "smoke_vape"),
    "final_event"
] = "retail_accessory_or_smoke"


In [211]:
df_doc.loc[
    mask_generic &
    df_doc["text_clean"].str.contains("warranty|repeat|pickup|partial"),
    "final_event"
] = "deposit"


In [212]:
final_counts = df_doc["final_event"].value_counts()
final_counts



final_event
screen                            604
unclassified_repair_or_service     98
battery                            68
back_glass                         55
charging_port                      43
retail_accessory_or_smoke          21
camera                             17
software_setup                     11
deposit                            11
diagnostic                          8
antenna                             7
water_damage                        5
hardware_install                    4
board_level                         4
audio_repair                        3
pc_wiring                           2
fee                                 2
button                              2
proximity_sensor                    2
service_call                        1
Name: count, dtype: int64

In [213]:
# Row integrity
assert df_doc.shape[0] == 968

# Revenue integrity
assert round(df_doc["net_sales_num"].sum(), 2) == 155265.60

# No missing events
df_doc["final_event"].isna().sum()



0

In [214]:
generic_final = (
    df_doc[df_doc["final_event"] == "generic_repair_device_only"]
    [["text_clean", "device_type_final", "brand_final", "net_sales_num"]]
    .sort_values("net_sales_num", ascending=False)
)

generic_final


,text_clean,device_type_final,brand_final,net_sales_num


In [215]:
explicit_repair_mask = (
    df_doc["final_event"] == "generic_repair_device_only"
) & df_doc["text_clean"].str.contains(
    r"\brepair\b|\breplacement\b|\bfix\b", regex=True
)


In [216]:
console_board_mask = (
    explicit_repair_mask &
    (df_doc["device_type_final"] == "console")
)

df_doc.loc[console_board_mask, "final_event"] = "board_level"


In [217]:
console_board_mask = (
    explicit_repair_mask &
    (df_doc["device_type_final"] == "console")
)

df_doc.loc[console_board_mask, "final_event"] = "board_level"


In [218]:
phone_tablet_other_mask = (
    explicit_repair_mask &
    df_doc["device_type_final"].isin(["phone", "tablet"])
)

df_doc.loc[phone_tablet_other_mask, "final_event"] = "other_device_repair"


In [219]:
# No missing events
assert df_doc["final_event"].isna().sum() == 0

# Revenue lock
assert round(df_doc["net_sales_num"].sum(), 2) == 155265.60

# Recheck distribution
df_doc["final_event"].value_counts()


final_event
screen                            604
unclassified_repair_or_service     98
battery                            68
back_glass                         55
charging_port                      43
retail_accessory_or_smoke          21
camera                             17
software_setup                     11
deposit                            11
diagnostic                          8
antenna                             7
water_damage                        5
hardware_install                    4
board_level                         4
audio_repair                        3
pc_wiring                           2
fee                                 2
button                              2
proximity_sensor                    2
service_call                        1
Name: count, dtype: int64

In [220]:
def has_any_token(tokens, token_set):
    return any(t in token_set for t in tokens)


In [221]:
EVENT_TOKEN_MAP = {
    "screen": {
        "screen", "lcd", "display", "digitizer"
    },

    "battery": {
        "battery", "batt"
    },

    "back_glass": {
        "back", "rear", "glass"
    },

    "charging_port": {
        "port", "charging"
    },

    "camera": {
        "camera", "lens"
    },

    "speaker_mic": {
        "speaker", "mic", "microphone", "earpiece"
    },

    "proximity_sensor": {
        "proximity", "sensor", "faceid", "face", "true"
    },

    "button": {
        "button", "power", "volume"
    },

    "antenna": {
        "antenna", "signal", "service"
    },

    "board_level": {
        "hdmi", "board", "motherboard", "logic",
        "microsolder", "solder", "chip", "ic"
    },

    "water_damage": {
        "water", "liquid", "moisture"
    },

    "diagnostic": {
        "diagnostic", "diagnostics", "diag"
    },

    "software_setup": {
        "setup", "install", "installation",
        "reset", "restore", "update", "unlock"
    },

    "hardware_install": {
        "ram", "ssd", "harddrive", "drive", "os"
    },

    "pc_wiring": {
        "wiring", "cable", "wire"
    },

    "deposit": {
        "deposit"
    },

    "fee": {
        "fee", "afterpay", "tip"
    },

    "service_call": {
        "consulting", "service call"
    }
}


In [222]:
BACK_GLASS_EXCLUDES = {
    "woods", "backwoods", "rillos", "dutch"
}


In [223]:
for event, token_set in EVENT_TOKEN_MAP.items():
    df_doc[f"event_{event}"] = df_doc["tokens"].apply(
        lambda t: has_any_token(t, token_set)
    )


In [224]:
df_doc["event_back_glass"] = (
    df_doc["event_back_glass"] &
    ~df_doc["tokens"].apply(lambda t: has_any_token(t, BACK_GLASS_EXCLUDES))
)


In [225]:
event_summary = []

for event in EVENT_TOKEN_MAP:
    rows = df_doc[f"event_{event}"].sum()
    revenue = df_doc.loc[df_doc[f"event_{event}"], "net_sales_num"].sum()

    event_summary.append({
        "event": event,
        "rows": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

event_summary_df = (
    pd.DataFrame(event_summary)
    .sort_values("revenue", ascending=False)
)

display(event_summary_df)


,event,rows,revenue,avg_ticket
0,screen,604,106086.21,175.639421
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
3,charging_port,61,7205.00,118.114754
13,hardware_install,5,3830.00,766.000000
4,camera,23,2930.00,127.391304
10,water_damage,11,2885.00,262.272727
9,board_level,11,2566.00,233.272727
12,software_setup,14,2230.00,159.285714
15,deposit,16,1837.00,114.812500


In [226]:
EVENT_COLUMNS = [f"event_{e}" for e in EVENT_TOKEN_MAP.keys()]

df_doc["has_any_event"] = df_doc[EVENT_COLUMNS].any(axis=1)


In [227]:
df_doc["event_generic_repair"] = (
    ~df_doc["has_any_event"] &
    df_doc["is_repair"]
)

df_doc["event_generic_service"] = (
    ~df_doc["has_any_event"] &
    ~df_doc["is_repair"]
)


In [228]:
residual_summary = (
    df_doc[df_doc["event_generic_repair"] | df_doc["event_generic_service"]]
    .groupby(
        [
            "event_generic_repair",
            "event_generic_service"
        ]
    )
    .agg(
        rows=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
)

display(residual_summary)


,,rows,revenue,avg_ticket
event_generic_repair,event_generic_service,,,
False,True,120,17313.83,144.281917
True,False,2,600.00,300.000000


In [229]:
final_event_summary = []

# Known events
for event in EVENT_TOKEN_MAP:
    rows = df_doc[f"event_{event}"].sum()
    revenue = df_doc.loc[df_doc[f"event_{event}"], "net_sales_num"].sum()

    final_event_summary.append({
        "event": event,
        "rows": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

# Residuals
final_event_summary.append({
    "event": "generic_repair",
    "rows": df_doc["event_generic_repair"].sum(),
    "revenue": df_doc.loc[df_doc["event_generic_repair"], "net_sales_num"].sum(),
    "avg_ticket": (
        df_doc.loc[df_doc["event_generic_repair"], "net_sales_num"].mean()
    )
})

final_event_summary.append({
    "event": "generic_service",
    "rows": df_doc["event_generic_service"].sum(),
    "revenue": df_doc.loc[df_doc["event_generic_service"], "net_sales_num"].sum(),
    "avg_ticket": (
        df_doc.loc[df_doc["event_generic_service"], "net_sales_num"].mean()
    )
})

final_event_summary_df = (
    pd.DataFrame(final_event_summary)
    .sort_values("revenue", ascending=False)
)

display(final_event_summary_df)


,event,rows,revenue,avg_ticket
0,screen,604,106086.21,175.639421
19,generic_service,120,17313.83,144.281917
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
3,charging_port,61,7205.00,118.114754
13,hardware_install,5,3830.00,766.000000
4,camera,23,2930.00,127.391304
10,water_damage,11,2885.00,262.272727
9,board_level,11,2566.00,233.272727
12,software_setup,14,2230.00,159.285714


In [230]:
 generic_repairs = df_doc[
    df_doc["event_generic_repair"]
][[
    "text_clean",
    "device_type_final",
    "brand_final",
    "net_sales_num"
]].sort_values("net_sales_num", ascending=False)

display(generic_repairs.head(84))


,text_clean,device_type_final,brand_final,net_sales_num
238,macbook a2338 keyboard repair,computer,Apple,500.0
439,macbook a2338 keyboard repair,computer,Apple,100.0


In [231]:

from collections import Counter

generic_repair_tokens = Counter(
    token
    for tokens in df_doc.loc[df_doc["event_generic_repair"], "tokens"]
    for token in tokens
)

generic_repair_tokens.most_common(40)


[('macbook', 2), ('a2338', 2), ('keyboard', 2), ('repair', 2)]

In [232]:
CONSOLE_BOARD_TOKENS = {
    "xbox", "ps5", "playstation", "console", "hdmi"
}

df_doc["event_board_level"] = (
    df_doc["event_board_level"] |
    (
        df_doc["event_generic_repair"] &
        df_doc["tokens"].apply(lambda t: any(tok in t for tok in CONSOLE_BOARD_TOKENS))
    )
)


In [233]:
HOUSING_TOKENS = {"housing", "frame", "chassis", "shell"}

df_doc["event_housing_frame"] = (
    df_doc["event_generic_repair"] &
    df_doc["tokens"].apply(lambda t: any(tok in t for tok in HOUSING_TOKENS))
)


In [234]:
FLEX_TOKENS = {"flex", "ribbon", "cable"}

df_doc["event_flex_cable"] = (
    df_doc["event_generic_repair"] &
    df_doc["tokens"].apply(lambda t: any(tok in t for tok in FLEX_TOKENS))
)


In [235]:
df_doc["event_generic_repair"] = (
    df_doc["event_generic_repair"] &
    ~(
        df_doc["event_board_level"] |
        df_doc["event_housing_frame"] |
        df_doc["event_flex_cable"]
    )
)


In [236]:
final_events = [
    "screen", "battery", "back_glass", "charging_port", "camera",
    "board_level", "housing_frame", "flex_cable",
    "software_setup", "diagnostic", "water_damage",
    "proximity_sensor", "button", "antenna", "speaker_mic",
    "hardware_install", "pc_wiring", "deposit", "fee",
    "service_call", "generic_repair", "generic_service"
]

summary = []

for event in final_events:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    summary.append({
        "event": event,
        "rows": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

final_summary_df = (
    pd.DataFrame(summary)
    .sort_values("revenue", ascending=False)
)

display(final_summary_df)


,event,rows,revenue,avg_ticket
0,screen,604,106086.21,175.639421
21,generic_service,120,17313.83,144.281917
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
3,charging_port,61,7205.00,118.114754
15,hardware_install,5,3830.00,766.000000
4,camera,23,2930.00,127.391304
10,water_damage,11,2885.00,262.272727
5,board_level,11,2566.00,233.272727
8,software_setup,14,2230.00,159.285714


In [237]:
SMALL_PART_EVENTS = [
    "event_speaker_mic",
    "event_button",
    "event_proximity_sensor",
    "event_flex_cable",
    "event_antenna"
]

df_doc["event_small_part_repair"] = False

for col in SMALL_PART_EVENTS:
    if col in df_doc:
        df_doc["event_small_part_repair"] |= df_doc[col]


In [238]:
df_doc["event_generic_repair"] = (
    df_doc["event_generic_repair"] &
    ~(
        df_doc["event_board_level"] |
        df_doc["event_housing_frame"] |
        df_doc["event_flex_cable"] |
        df_doc["event_small_part_repair"]
    )
)


In [239]:
FINAL_EVENTS = [
    "screen",
    "battery",
    "back_glass",
    "charging_port",
    "camera",
    "board_level",
    "small_part_repair",
    "water_damage",
    "hardware_install",
    "software_setup",
    "pc_wiring",
    "diagnostic",
    "deposit",
    "fee",
    "service_call",
    "generic_repair",
    "generic_service"
]


In [240]:
final_summary = []

for event in FINAL_EVENTS:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    final_summary.append({
        "event": event,
        "rows": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

final_summary_df = (
    pd.DataFrame(final_summary)
    .sort_values("revenue", ascending=False)
)

display(final_summary_df)


,event,rows,revenue,avg_ticket
0,screen,604,106086.21,175.639421
16,generic_service,120,17313.83,144.281917
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
3,charging_port,61,7205.00,118.114754
8,hardware_install,5,3830.00,766.000000
4,camera,23,2930.00,127.391304
7,water_damage,11,2885.00,262.272727
6,small_part_repair,23,2580.00,112.173913
5,board_level,11,2566.00,233.272727


In [241]:
# Show all columns that look like text
[c for c in df_doc.columns if "text" in c.lower()]


['text_raw', 'text_clean']

In [242]:
df_doc["text_norm"] = df_doc["text_clean"]


In [243]:
# -----------------------------
# Inspect generic SERVICE
# -----------------------------
display(
    df_doc.loc[
        df_doc["event_generic_service"],
        ["text_norm", "net_sales_num"]
    ]
    .value_counts()
    .head(30)
)

# -----------------------------
# Inspect generic REPAIR
# -----------------------------
display(
    df_doc.loc[
        df_doc["event_generic_repair"],
        ["text_norm", "net_sales_num"]
    ]
    .value_counts()
    .head(60)
)


text_norm                                               net_sales_num
iphone 11                                               120.00           5
clickmate                                               30.00            4
lost mary                                               30.00            3
iphone 14 promax                                        150.00           3
mexican mango geekbar                                   30.00            2
otterbox defender series case                           55.00            2
iphone 11                                               100.00           2
10 macbooks pro 15 inch                                 3000.00          1
iphone 15 promax                                        170.00           1
iphone x repair                                         115.00           1
iphone se repeat customer                               85.00            1
iphone se                                               70.00            1
iphone charger                

text_norm                      net_sales_num
macbook a2338 keyboard repair  100.0            1
                               500.0            1
Name: count, dtype: int64

In [244]:
retail_mask = (
    (df_doc["event_generic_service"] | df_doc["event_generic_repair"]) &
    df_doc["text_norm"].str.contains(
        r"vape|geekbar|lost mary|clickmate|otterbox|case|charger",
        regex=True
    )
)

df_doc.loc[retail_mask, "event_retail_accessory_or_smoke"] = True
df_doc.loc[retail_mask, ["event_generic_service", "event_generic_repair"]] = False


In [245]:
repair_word_mask = (
    df_doc["event_generic_service"] &
    df_doc["text_norm"].str.contains(r"repair", regex=True)
)

df_doc.loc[repair_word_mask, "event_generic_repair"] = True
df_doc.loc[repair_word_mask, "event_generic_service"] = False


In [246]:
hardware_mask = (
    (df_doc["event_generic_service"] | df_doc["event_generic_repair"]) &
    df_doc["text_norm"].str.contains(
        r"macbook|keyboard|install|wiring",
        regex=True
    )
)

df_doc.loc[hardware_mask, "event_hardware_install"] = True
df_doc.loc[hardware_mask, ["event_generic_service", "event_generic_repair"]] = False


In [247]:
df_doc["event_generic_service_final"] = df_doc["event_generic_service"]
df_doc["event_generic_repair_final"] = df_doc["event_generic_repair"]


In [248]:
df_doc[
    df_doc["event_generic_repair_final"]
][["text_norm", "net_sales_num"]].value_counts().head(100)


text_norm                             net_sales_num
14 pro fully repair                   350.00           1
6 gen ipad repair                     100.00           1
tv repair                             130.00           1
se repair                             160.00           1
s24 repair                            210.00           1
s22 ultra repair                      200.00           1
s10 repair                            180.00           1
s10 plus repair                       200.00           1
repair                                110.00           1
iphone x repair                       115.00           1
iphone 16 pro repair partial payment  210.00           1
iphone 14 pro repair                  200.00           1
iphone 14 premium repair              200.00           1
iphone 14 plus repair                 130.00           1
iphone 12 repair from yesterday       110.00           1
iphone 11 repair                      105.00           1
ipad pro repair                     

In [249]:
xbox_board_mask = (
    df_doc["event_generic_repair"] &
    df_doc["text_norm"].str.contains(r"xbox", regex=True)
)

df_doc.loc[xbox_board_mask, "event_board_level"] = True
df_doc.loc[xbox_board_mask, "event_generic_repair"] = False


In [250]:
hardware_mask = (
    df_doc["event_generic_repair"] &
    df_doc["text_norm"].str.contains(
        r"tv repair|computer repair",
        regex=True
    )
)

df_doc.loc[hardware_mask, "event_hardware_install"] = True
df_doc.loc[hardware_mask, "event_generic_repair"] = False


In [251]:
df_doc.loc[
    df_doc["text_norm"].str.contains("fully repair"),
    "flag_multi_unspecified"
] = True


In [252]:
df_doc["event_generic_repair_final"] = df_doc["event_generic_repair"]


In [253]:
repair_events = [
    "screen",
    "battery",
    "back_glass",
    "charging_port",
    "camera",
    "board_level",
    "small_part_repair",
    "water_damage",
    "hardware_install",
    "generic_repair_final"
]

repair_summary = []

for event in repair_events:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    repair_summary.append({
        "repair_type": event,
        "transactions": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

repair_summary_df = (
    pd.DataFrame(repair_summary)
    .sort_values("revenue", ascending=False)
)

display(repair_summary_df)


,repair_type,transactions,revenue,avg_ticket
0,screen,604,106086.21,175.639421
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
8,hardware_install,11,7968.00,724.363636
3,charging_port,61,7205.00,118.114754
9,generic_repair_final,21,3600.62,171.458095
4,camera,23,2930.00,127.391304
7,water_damage,11,2885.00,262.272727
5,board_level,12,2716.00,226.333333
6,small_part_repair,23,2580.00,112.173913


In [254]:
service_events = [
    "software_setup",
    "diagnostic",
    "deposit",
    "fee",
    "service_call",
    "generic_service_final"
]

service_summary = []

for event in service_events:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    service_summary.append({
        "service_type": event,
        "transactions": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

service_summary_df = (
    pd.DataFrame(service_summary)
    .sort_values("revenue", ascending=False)
)

display(service_summary_df)


,service_type,transactions,revenue,avg_ticket
5,generic_service_final,72,8994.21,124.919583
0,software_setup,14,2230.00,159.285714
2,deposit,16,1837.00,114.812500
1,diagnostic,15,820.00,54.666667
4,service_call,1,750.00,750.000000
3,fee,3,242.00,80.666667


In [255]:
df_doc[
    df_doc["event_generic_service_final"]
][["text_norm", "net_sales_num"]].value_counts().head(25)


text_norm                                               net_sales_num
iphone 11                                               120.00           5
iphone 14 promax                                        150.00           3
iphone 11                                               100.00           2
12 pro max replacement                                  130.00           1
iphone 15 promax                                        170.00           1
iphone 12 promax north field management                 100.00           1
iphone 12 repeat customer mobile                        120.00           1
iphone 13 mini                                          145.00           1
iphone 13 pro                                           120.00           1
iphone 14 plus backglass tech is picking up the device  0.01             1
iphone 14 pro max                                       180.00           1
iphone 14 promax                                        140.00           1
iphone 14 promax repeat custom

In [256]:
df_doc[
    df_doc["event_generic_repair_final"]
][["text_norm", "net_sales_num"]].value_counts().head(15)


text_norm                             net_sales_num
14 pro fully repair                   350.0            1
iphone 14 premium repair              200.0            1
s24 repair                            210.0            1
s22 ultra repair                      200.0            1
s10 repair                            180.0            1
s10 plus repair                       200.0            1
repair                                110.0            1
iphone x repair                       115.0            1
iphone 16 pro repair partial payment  210.0            1
iphone 14 pro repair                  200.0            1
iphone 14 plus repair                 130.0            1
6 gen ipad repair                     100.0            1
iphone 12 repair from yesterday       110.0            1
iphone 11 repair                      105.0            1
ipad pro repair                       250.0            1
Name: count, dtype: int64

In [257]:
df_doc["event_device_only"] = False


In [258]:
device_only_mask = (
    (df_doc["event_generic_service_final"] | df_doc["event_generic_repair_final"]) &
    df_doc["text_norm"].str.match(
        r"^(iphone|ipad|samsung|galaxy|note|pixel|moto)\b"
    ) &
    ~df_doc["text_norm"].str.contains(
        r"repair|screen|battery|glass|port|camera|diagnostic|setup|fee|deposit",
        regex=True
    )
)

df_doc.loc[device_only_mask, "event_device_only"] = True
df_doc.loc[
    device_only_mask,
    ["event_generic_service_final", "event_generic_repair_final"]
] = False


In [259]:
df_doc[
    df_doc["event_device_only"]
][["text_norm", "net_sales_num"]].value_counts().head(20)


text_norm                         net_sales_num
iphone 11                         120.0            5
iphone 14 promax                  150.0            3
iphone 11                         100.0            2
ipad 9th gen                      110.0            1
iphone 15 promax                  170.0            1
iphone 14 promax repeat customer  100.0            1
iphone 15                         200.0            1
iphone 15 pro max                 150.0            1
                                  230.0            1
iphone se                         70.0             1
iphone 8 se warranty              50.0             1
iphone 14 pro max                 180.0            1
iphone se repeat customer         85.0             1
iphone xr                         95.0             1
iphone xs max                     130.0            1
moto g stylus                     140.0            1
iphone 14 promax                  140.0            1
iphone 13 pro                     120.0            

In [260]:
df_doc[
    df_doc["event_generic_repair_final"]
][["text_norm", "net_sales_num"]].value_counts().head(20)


text_norm                             net_sales_num
14 pro fully repair                   350.00           1
iphone 14 premium repair              200.00           1
s24 repair                            210.00           1
s22 ultra repair                      200.00           1
s10 repair                            180.00           1
s10 plus repair                       200.00           1
repair                                110.00           1
iphone x repair                       115.00           1
iphone 16 pro repair partial payment  210.00           1
iphone 14 pro repair                  200.00           1
iphone 14 plus repair                 130.00           1
6 gen ipad repair                     100.00           1
iphone 12 repair from yesterday       110.00           1
iphone 11 repair                      105.00           1
ipad pro repair                       250.00           1
ipad and iphone 8 repair              130.00           1
ipad air repair                     

In [261]:
repair_events = [
    "screen",
    "battery",
    "back_glass",
    "charging_port",
    "camera",
    "board_level",
    "small_part_repair",
    "water_damage",
    "hardware_install",
    "generic_repair_final",        # explicit generic
    "generic_repair_implicit"      # implicit (device-only)
]

repair_summary = []

for event in repair_events:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    repair_summary.append({
        "repair_type": event,
        "transactions": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

repair_summary_df = (
    pd.DataFrame(repair_summary)
    .sort_values("revenue", ascending=False)
)

display(repair_summary_df)


,repair_type,transactions,revenue,avg_ticket
0,screen,604,106086.21,175.639421
2,back_glass,82,14598.14,178.026098
1,battery,85,10478.29,123.274000
8,hardware_install,11,7968.00,724.363636
3,charging_port,61,7205.00,118.114754
9,generic_repair_final,21,3600.62,171.458095
4,camera,23,2930.00,127.391304
7,water_damage,11,2885.00,262.272727
5,board_level,12,2716.00,226.333333
6,small_part_repair,23,2580.00,112.173913


In [262]:
service_events = [
    "software_setup",
    "diagnostic",
    "deposit",
    "fee",
    "service_call",
    "generic_service_final"
]

service_summary = []

for event in service_events:
    col = f"event_{event}"
    rows = df_doc[col].sum() if col in df_doc else 0
    revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum() if col in df_doc else 0

    service_summary.append({
        "service_type": event,
        "transactions": rows,
        "revenue": revenue,
        "avg_ticket": revenue / rows if rows else 0
    })

service_summary_df = (
    pd.DataFrame(service_summary)
    .sort_values("revenue", ascending=False)
)

display(service_summary_df)


,service_type,transactions,revenue,avg_ticket
5,generic_service_final,33,3909.21,118.460909
0,software_setup,14,2230.00,159.285714
2,deposit,16,1837.00,114.812500
1,diagnostic,15,820.00,54.666667
4,service_call,1,750.00,750.000000
3,fee,3,242.00,80.666667


In [263]:
# Inspect remaining GENERIC REPAIRS
display(
    df_doc[
        df_doc["event_generic_repair_final"]
    ][["text_norm", "net_sales_num"]]
    .value_counts()
    .head(50)
)

# Inspect remaining GENERIC SERVICES
display(
    df_doc[
        df_doc["event_generic_service_final"]
    ][["text_norm", "net_sales_num"]]
    .value_counts()
    .head(50)
)


text_norm                             net_sales_num
14 pro fully repair                   350.00           1
iphone 14 premium repair              200.00           1
s24 repair                            210.00           1
s22 ultra repair                      200.00           1
s10 repair                            180.00           1
s10 plus repair                       200.00           1
repair                                110.00           1
iphone x repair                       115.00           1
iphone 16 pro repair partial payment  210.00           1
iphone 14 pro repair                  200.00           1
iphone 14 plus repair                 130.00           1
6 gen ipad repair                     100.00           1
iphone 12 repair from yesterday       110.00           1
iphone 11 repair                      105.00           1
ipad pro repair                       250.00           1
ipad and iphone 8 repair              130.00           1
ipad air repair                     

text_norm                                               net_sales_num
12 pro max replacement                                  130.00           1
dell inspiron pc cleanup                                130.00           1
warranty iphone 8 se                                    80.00            1
s24 ultra                                               320.00           1
s23 ultra                                               220.00           1
s22 ultra replacement                                   173.20           1
s20 fe                                                  190.00           1
                                                        100.00           1
pc build parts provided and untested                    150.00           1
juan warranty 15pro                                     120.00           1
iphone 14 plus backglass tech is picking up the device  0.01             1
ipad charge ports                                       450.00           1
hawaiian b                    

In [264]:
df_doc["event_unknown_undocumented"] = False


In [265]:
unknown_mask = (
    df_doc["event_generic_repair_final"] &
    (
        df_doc["text_norm"].isna() |
        (df_doc["text_norm"].str.strip() == "") |
        (df_doc["text_norm"].str.len() < 5)
    )
)

df_doc.loc[unknown_mask, "event_unknown_undocumented"] = True
df_doc.loc[unknown_mask, "event_generic_repair_final"] = False


In [266]:
battery_mask = (
    df_doc["event_generic_repair_final"] &
    df_doc["text_norm"].str.contains(
        r"battery replacement",
        regex=True
    ) &
    ~df_doc["text_norm"].str.contains(
        r"and|water damage",
        regex=True
    )
)

df_doc.loc[battery_mask, "event_battery"] = True
df_doc.loc[battery_mask, "event_generic_repair_final"] = False


In [267]:
backglass_mask = (
    df_doc["event_generic_repair_final"] &
    df_doc["text_norm"].str.contains(
        r"back glass replacement",
        regex=True
    )
)

df_doc.loc[backglass_mask, "event_back_glass"] = True
df_doc.loc[backglass_mask, "event_generic_repair_final"] = False


In [268]:
screen_mask = (
    df_doc["event_generic_repair_final"] &
    df_doc["text_norm"].str.contains(
        r"screen replacement|lcd",
        regex=True
    ) &
    ~df_doc["text_norm"].str.contains(
        r"battery|port|camera|water damage",
        regex=True
    )
)

df_doc.loc[screen_mask, "event_screen"] = True
df_doc.loc[screen_mask, "event_generic_repair_final"] = False


In [269]:
port_mask = (
    df_doc["event_generic_repair_final"] &
    df_doc["text_norm"].str.contains(
        r"charging port replacement|port replacement",
        regex=True
    ) &
    ~df_doc["text_norm"].str.contains(
        r"battery|screen|camera|water damage",
        regex=True
    )
)

df_doc.loc[port_mask, "event_charging_port"] = True
df_doc.loc[port_mask, "event_generic_repair_final"] = False


In [270]:
# What is still generic (should now be small + legit)
df_doc[
    df_doc["event_generic_repair_final"]
][["text_norm", "net_sales_num"]].value_counts().head(25)

# Unknown / undocumented (parked for later)
df_doc[
    df_doc["event_unknown_undocumented"]
][["text_norm", "net_sales_num"]].value_counts().head(25)


Series([], Name: count, dtype: int64)

In [271]:
# -----------------------------
# FINAL Repair + Service View
# -----------------------------

final_rows = []

# ---- Repairs ----
repair_events = [
    "screen",
    "back_glass",
    "battery",
    "charging_port",
    "camera",
    "board_level",
    "hardware_install",
    "small_part_repair",
    "water_damage",
    "generic_repair_final"   # irreducible ambiguity only
]

for event in repair_events:
    col = f"event_{event}"
    if col in df_doc:
        rows = int(df_doc[col].sum())
        if rows > 0:
            revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum()
            final_rows.append({
                "category": "Repair",
                "type": event,
                "transactions": rows,
                "revenue": round(revenue, 2),
                "avg_ticket": round(revenue / rows, 2)
            })

# ---- Services ----
service_events = [
    "software_setup",
    "diagnostic",
    "deposit",
    "fee",
    "service_call",
    "generic_service_final"  # should be very small
]

for event in service_events:
    col = f"event_{event}"
    if col in df_doc:
        rows = int(df_doc[col].sum())
        if rows > 0:
            revenue = df_doc.loc[df_doc[col], "net_sales_num"].sum()
            final_rows.append({
                "category": "Service",
                "type": event,
                "transactions": rows,
                "revenue": round(revenue, 2),
                "avg_ticket": round(revenue / rows, 2)
            })

final_view_df = (
    pd.DataFrame(final_rows)
    .sort_values(["category", "revenue"], ascending=[True, False])
)

display(final_view_df)


,category,type,transactions,revenue,avg_ticket
0,Repair,screen,604,106086.21,175.64
1,Repair,back_glass,82,14598.14,178.03
2,Repair,battery,85,10478.29,123.27
6,Repair,hardware_install,11,7968.00,724.36
3,Repair,charging_port,61,7205.00,118.11
9,Repair,generic_repair_final,21,3600.62,171.46
4,Repair,camera,23,2930.00,127.39
8,Repair,water_damage,11,2885.00,262.27
5,Repair,board_level,12,2716.00,226.33
7,Repair,small_part_repair,23,2580.00,112.17


In [272]:
# -----------------------------
# Inspect remaining GENERIC SERVICES (FINAL)
# -----------------------------
display(
    df_doc[
        df_doc["event_generic_service_final"]
    ][["text_norm", "net_sales_num"]]
    .value_counts()
    .head(50)
)


text_norm                                               net_sales_num
12 pro max replacement                                  130.00           1
dell inspiron pc cleanup                                130.00           1
warranty iphone 8 se                                    80.00            1
s24 ultra                                               320.00           1
s23 ultra                                               220.00           1
s22 ultra replacement                                   173.20           1
s20 fe                                                  190.00           1
                                                        100.00           1
pc build parts provided and untested                    150.00           1
juan warranty 15pro                                     120.00           1
iphone 14 plus backglass tech is picking up the device  0.01             1
ipad charge ports                                       450.00           1
hawaiian b                    

In [273]:
# -----------------------------
# Inspect remaining GENERIC REPAIRS (FINAL)
# -----------------------------
display(
    df_doc[
        df_doc["event_generic_repair_final"]
    ][["text_norm", "net_sales_num"]]
    .value_counts()
    .head(75)
)


text_norm                             net_sales_num
14 pro fully repair                   350.00           1
iphone 14 premium repair              200.00           1
s24 repair                            210.00           1
s22 ultra repair                      200.00           1
s10 repair                            180.00           1
s10 plus repair                       200.00           1
repair                                110.00           1
iphone x repair                       115.00           1
iphone 16 pro repair partial payment  210.00           1
iphone 14 pro repair                  200.00           1
iphone 14 plus repair                 130.00           1
6 gen ipad repair                     100.00           1
iphone 12 repair from yesterday       110.00           1
iphone 11 repair                      105.00           1
ipad pro repair                       250.00           1
ipad and iphone 8 repair              130.00           1
ipad air repair                     

In [274]:
# move obvious retail out of generic service
retail_mask = (
    df_doc["event_generic_service_final"] &
    df_doc["text_norm"].str.contains(r"hawaiian", case=False, na=False)
)

df_doc.loc[retail_mask, "event_retail_accessory_or_smoke"] = True
df_doc.loc[retail_mask, "event_generic_service_final"] = False


# move undocumented service entries to unknown bucket
unknown_service_mask = (
    df_doc["event_generic_service_final"] &
    (
        df_doc["text_norm"].isna() |
        (df_doc["text_norm"].str.strip() == "") |
        (df_doc["text_norm"].str.len() < 5)
    )
)

df_doc.loc[unknown_service_mask, "event_unknown_undocumented"] = True
df_doc.loc[unknown_service_mask, "event_generic_service_final"] = False


In [275]:
board_mask = (
    df_doc["event_generic_repair_final"] &
    df_doc["text_norm"].str.contains(r"logic board", regex=True)
)

df_doc.loc[board_mask, "event_board_level"] = True
df_doc.loc[board_mask, "event_generic_repair_final"] = False


In [276]:
df_doc[df_doc["event_generic_service_final"]][["text_norm", "net_sales_num"]]


,text_norm,net_sales_num
44,14 pro max,230.00
84,apple watch se 44mm,150.00
193,12 pro max replacement,130.00
199,15 pro max housing,320.00
363,s22 ultra replacement,173.20
392,s20 fe,190.00
443,13 mini remainder,30.00
455,a14 5g,100.00
476,s20 fe,100.00
550,iphone 14 plus backglass tech is picking up th...,0.01


In [277]:
df_doc[df_doc["event_generic_repair_final"]][["text_norm", "net_sales_num"]].value_counts().head(40)


text_norm                             net_sales_num
14 pro fully repair                   350.00           1
iphone 14 premium repair              200.00           1
s24 repair                            210.00           1
s22 ultra repair                      200.00           1
s10 repair                            180.00           1
s10 plus repair                       200.00           1
repair                                110.00           1
iphone x repair                       115.00           1
iphone 16 pro repair partial payment  210.00           1
iphone 14 pro repair                  200.00           1
iphone 14 plus repair                 130.00           1
6 gen ipad repair                     100.00           1
iphone 12 repair from yesterday       110.00           1
iphone 11 repair                      105.00           1
ipad pro repair                       250.00           1
ipad and iphone 8 repair              130.00           1
ipad air repair                     

In [278]:
# Multi-activity flags
df_doc["flag_multi_repair"] = False
df_doc["flag_multi_service"] = False
df_doc["flag_accessory_bundle"] = False
df_doc["flag_partial_payment"] = False
df_doc["flag_defective_part"] = False


In [279]:
repair_keywords = [
    "screen", "battery", "back glass", "charging port",
    "camera", "lens", "keyboard", "antenna", "speaker"
]

df_doc["repair_keyword_count"] = df_doc["text_norm"].apply(
    lambda x: sum(k in x for k in repair_keywords) if isinstance(x, str) else 0
)

df_doc.loc[
    df_doc["repair_keyword_count"] > 1,
    "flag_multi_repair"
] = True


In [280]:
accessory_mask = df_doc["text_norm"].str.contains(
    r"protector|cable|charger|case|usb",
    regex=True,
    na=False
)

df_doc.loc[accessory_mask, "flag_accessory_bundle"] = True


In [281]:
df_doc.loc[
    df_doc["text_norm"].str.contains("water damage", na=False),
    "event_water_damage"
] = True


In [282]:
df_doc.loc[
    df_doc["text_norm"].str.contains("partial|remaining|deposit", na=False),
    "flag_partial_payment"
] = True

df_doc.loc[
    df_doc["text_norm"].str.contains("defective", na=False),
    "flag_defective_part"
] = True


In [283]:
# =====================================================
# 🔍 MASTER CLASSIFICATION STATUS DASHBOARD
# =====================================================

def show_status(df):
    rows = []

    def add_row(name, mask):
        rows.append({
            "bucket": name,
            "transactions": int(mask.sum()),
            "revenue": round(df.loc[mask, "net_sales_num"].sum(), 2)
        })

    # ---- Repairs (specific + generic) ----
    add_row("Repair – Screen", df["event_screen"])
    add_row("Repair – Battery", df["event_battery"])
    add_row("Repair – Back Glass", df["event_back_glass"])
    add_row("Repair – Charging Port", df["event_charging_port"])
    add_row("Repair – Camera / Lens", df["event_camera"])
    add_row("Repair – Board Level", df["event_board_level"])
    add_row("Repair – Hardware Install", df["event_hardware_install"])
    add_row("Repair – Small Part", df["event_small_part_repair"])
    add_row("Repair – Water Damage", df["event_water_damage"])
    add_row("Repair – Generic (Explicit)", df["event_generic_repair_final"])

    # ---- Services ----
    add_row("Service – Software Setup", df["event_software_setup"])
    add_row("Service – Diagnostic", df["event_diagnostic"])
    add_row("Service – Deposit", df["event_deposit"])
    add_row("Service – Fee", df["event_fee"])
    add_row("Service – Service Call", df["event_service_call"])
    add_row("Service – Generic", df["event_generic_service_final"])

    add_row(
    "Retail – Accessories / Smoke",
    df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)
)


    # ---- Unknown / Undocumented ----
    add_row("Unknown / Undocumented", df["event_unknown_undocumented"])

    # ---- Complexity Flags ----
    add_row("Multi-Repair Jobs", df["flag_multi_repair"])
    add_row("Accessory Bundles", df["flag_accessory_bundle"])
    add_row("Partial Payments", df["flag_partial_payment"])
    add_row("Defective / Adjustment", df["flag_defective_part"])

    return (
        pd.DataFrame(rows)
        .sort_values("revenue", ascending=False)
        .reset_index(drop=True)
    )


display(show_status(df_doc))


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)


,bucket,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Multi-Repair Jobs,43,8518.87
4,Repair – Hardware Install,11,7968.00
5,Repair – Charging Port,61,7205.00
6,Accessory Bundles,51,5872.00
7,Service – Generic,30,3694.21
8,Repair – Generic (Explicit),21,3600.62
9,Repair – Camera / Lens,23,2930.00


In [284]:
# =====================================================
# ✅ FINAL REPAIR / SERVICE PRESENTATION CONSOLIDATION
# =====================================================

# 1. Build raw status table (your existing logic)
status_df = show_status(df_doc)

# 2. Drop buckets we do NOT want to present
drop_buckets = [
    "Unknown / Undocumented",
    "Defective / Adjustment"
]

status_df = status_df[~status_df["bucket"].isin(drop_buckets)].copy()

# 3. Merge small service/admin buckets into one
service_merge_map = {
    "Service – Service Call": "Service – Other / Admin",
    "Service – Generic": "Service – Other / Admin",
    "Service – Fee": "Service – Other / Admin",
}

status_df["bucket_final"] = status_df["bucket"].replace(service_merge_map)

# 4. Re-aggregate AFTER merge (presentation only)
final_repair_service = (
    status_df
    .groupby("bucket_final", as_index=False)
    .agg(
        transactions=("transactions", "sum"),
        revenue=("revenue", "sum")
    )
    .sort_values("revenue", ascending=False)
    .reset_index(drop=True)
)

display(final_repair_service)

# =====================================================
# 🔒 LOCK STAGE
# =====================================================
PIPELINE_STAGE = "REPAIR_SERVICE_PRESENTATION_FINAL_LOCKED"
print("PIPELINE STAGE:", PIPELINE_STAGE)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)


,bucket_final,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Multi-Repair Jobs,43,8518.87
4,Repair – Hardware Install,11,7968.00
5,Repair – Charging Port,61,7205.00
6,Accessory Bundles,51,5872.00
7,Service – Other / Admin,34,4686.21
8,Repair – Generic (Explicit),21,3600.62
9,Repair – Camera / Lens,23,2930.00


PIPELINE STAGE: REPAIR_SERVICE_PRESENTATION_FINAL_LOCKED


In [285]:
[c for c in df_doc.columns if "bucket" in c.lower()]


[]

In [286]:
# =====================================================
# 🔒 PARAGON GEEKS — FINAL PRESENTATION SNAPSHOT (LOCKED)
# =====================================================
# PURPOSE:
# - Executive / portfolio snapshot
# - Uses event-flag aggregation (NOT row-exclusive logic)
# - Repair/Service + Device Type + Brand + Model
# - Numbers are DESCRIPTIVE and OVERLAPPING BY DESIGN
# =====================================================

import pandas as pd

# -----------------------------
# 1️⃣ REPAIR / SERVICE SNAPSHOT
# -----------------------------
status_df = show_status(df_doc)

# Remove buckets we never present
drop_buckets = [
    "Unknown / Undocumented",
    "Defective / Adjustment"
]
status_df = status_df[~status_df["bucket"].isin(drop_buckets)].copy()

# Merge small admin buckets (presentation only)
service_merge_map = {
    "Service – Service Call": "Service – Other / Admin",
    "Service – Generic": "Service – Other / Admin",
    "Service – Fee": "Service – Other / Admin",
}
status_df["bucket_final"] = status_df["bucket"].replace(service_merge_map)

repair_service_snapshot = (
    status_df
    .groupby("bucket_final", as_index=False)
    .agg(
        transactions=("transactions", "sum"),
        revenue=("revenue", "sum")
    )
    .sort_values("revenue", ascending=False)
)

# -----------------------------
# 2️⃣ DEVICE TYPE SNAPSHOT
# -----------------------------
device_snapshot = (
    df_doc
    .groupby("device_type_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

# -----------------------------
# 3️⃣ BRAND SNAPSHOT
# -----------------------------
brand_snapshot = (
    df_doc
    .groupby("brand_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

# -----------------------------
# 4️⃣ MODEL SNAPSHOT (TOP 25)
# -----------------------------
model_snapshot = (
    df_doc
    .groupby("device_model_exact_refined")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
    .head(25)
)

# -----------------------------
# DISPLAY — FINAL SNAPSHOTS
# -----------------------------
print("\n==============================")
print("FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)")
display(repair_service_snapshot)

print("\n==============================")
print("FINAL DEVICE TYPE SNAPSHOT")
display(device_snapshot)

print("\n==============================")
print("FINAL BRAND SNAPSHOT")
display(brand_snapshot)

print("\n==============================")
print("FINAL MODEL SNAPSHOT — TOP 25")
display(model_snapshot)

# -----------------------------
# VALIDATION
# -----------------------------
print("\n==============================")
print("VALIDATION")
print("Rows:", len(df_doc))
print("Revenue:", round(df_doc['net_sales_num'].sum(), 2))

PIPELINE_STAGE = "PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED"
print("\nPIPELINE STAGE:", PIPELINE_STAGE)



FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)


,bucket_final,transactions,revenue
10,Repair – Screen,604,106086.21
3,Repair – Back Glass,82,14598.14
4,Repair – Battery,85,10478.29
1,Multi-Repair Jobs,43,8518.87
9,Repair – Hardware Install,11,7968.00
7,Repair – Charging Port,61,7205.00
0,Accessory Bundles,51,5872.00
16,Service – Other / Admin,34,4686.21
8,Repair – Generic (Explicit),21,3600.62
6,Repair – Camera / Lens,23,2930.00



FINAL DEVICE TYPE SNAPSHOT


,transactions,revenue,avg_ticket
device_type_final,,,
phone,722,115118.98,159.444571
tablet,118,21736.62,184.208644
computer,41,11571.00,282.219512
unknown,28,2487.00,88.821429
accessory,21,1395.00,66.428571
console,6,1220.00,203.333333
smoke_vape,25,937.00,37.480000
watch,5,585.00,117.000000
tv,1,130.00,130.000000



FINAL BRAND SNAPSHOT


,transactions,revenue,avg_ticket
brand_final,,,
Apple,667,108841.11,163.180075
Samsung,158,30873.49,195.401835
Google,29,4875.00,168.103448
Unknown,36,3947.00,109.638889
Accessory,21,1395.00,66.428571
Microsoft,6,1230.00,205.000000
Motorola,9,1015.00,112.777778
Smoke / Vape,25,937.00,37.480000
Sony,4,900.00,225.000000



FINAL MODEL SNAPSHOT — TOP 25


,transactions,revenue,avg_ticket
device_model_exact_refined,,,
iPhone 14 Pro Max,52,9711.00,186.750000
iPhone 13,56,7937.61,141.743036
iPhone 15 Pro Max,40,7793.87,194.846750
Galaxy S23 Ultra,28,7514.00,268.357143
iPhone 11,64,6808.00,106.375000
iPhone 15,25,6740.00,269.600000
Galaxy S22 Ultra,21,5456.40,259.828571
iPad Pro,18,5150.00,286.111111
Unknown,53,4828.00,91.094340



VALIDATION
Rows: 968
Revenue: 155265.6

PIPELINE STAGE: PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED


In [287]:
# =====================================================
# ✅ FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION ONLY)
#    — Multi-Repair handled separately
# =====================================================

# 1️⃣ Build raw status table (unchanged logic)
status_df = show_status(df_doc)

# 2️⃣ Buckets to remove from presentation
drop_buckets = [
    "Multi-Repair Jobs",        # complexity, not service
    "Unknown / Undocumented",   # already zero
    "Defective / Adjustment"    # adjustments handled separately
]

status_df = status_df[~status_df["bucket"].isin(drop_buckets)].copy()

# 3️⃣ Merge small admin/service buckets
service_merge_map = {
    "Service – Service Call": "Service – Other / Admin",
    "Service – Generic": "Service – Other / Admin",
    "Service – Fee": "Service – Other / Admin",
}

status_df["bucket_final"] = status_df["bucket"].replace(service_merge_map)

# 4️⃣ Final presentation aggregation
final_repair_service_presentation = (
    status_df
    .groupby("bucket_final", as_index=False)
    .agg(
        transactions=("transactions", "sum"),
        revenue=("revenue", "sum")
    )
    .sort_values("revenue", ascending=False)
    .reset_index(drop=True)
)

display(final_repair_service_presentation)

# =====================================================
# 🔒 LOCK STAGE
# =====================================================
PIPELINE_STAGE = "REPAIR_SERVICE_PRESENTATION_FINAL_LOCKED_NO_MULTI_REPAIR"
print("PIPELINE STAGE:", PIPELINE_STAGE)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)


,bucket_final,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Repair – Hardware Install,11,7968.00
4,Repair – Charging Port,61,7205.00
5,Accessory Bundles,51,5872.00
6,Service – Other / Admin,34,4686.21
7,Repair – Generic (Explicit),21,3600.62
8,Repair – Camera / Lens,23,2930.00
9,Repair – Water Damage,11,2885.00


PIPELINE STAGE: REPAIR_SERVICE_PRESENTATION_FINAL_LOCKED_NO_MULTI_REPAIR


In [288]:
import pandas as pd

# =====================================================
# 1️⃣ FINAL REPAIR / SERVICE SNAPSHOT (NO COMPLEXITY)
# =====================================================

status_df = show_status(df_doc)

drop_buckets = [
    "Multi-Repair Jobs",
    "Unknown / Undocumented",
    "Defective / Adjustment"
]

status_df = status_df[~status_df["bucket"].isin(drop_buckets)].copy()

service_merge_map = {
    "Service – Service Call": "Service – Other / Admin",
    "Service – Generic": "Service – Other / Admin",
    "Service – Fee": "Service – Other / Admin",
}

status_df["bucket_final"] = status_df["bucket"].replace(service_merge_map)

final_repair_service = (
    status_df
    .groupby("bucket_final", as_index=False)
    .agg(
        transactions=("transactions", "sum"),
        revenue=("revenue", "sum")
    )
    .sort_values("revenue", ascending=False)
)

print("\n==============================")
print("FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)")
display(final_repair_service)


# =====================================================
# 2️⃣ FINAL DEVICE TYPE SNAPSHOT
# =====================================================

device_snapshot = (
    df_doc
    .groupby("device_type_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

print("\n==============================")
print("FINAL DEVICE TYPE SNAPSHOT")
display(device_snapshot)


# =====================================================
# 3️⃣ FINAL BRAND SNAPSHOT
# =====================================================

brand_snapshot = (
    df_doc
    .groupby("brand_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

print("\n==============================")
print("FINAL BRAND SNAPSHOT")
display(brand_snapshot)


# =====================================================
# 4️⃣ FINAL MODEL SNAPSHOT — TOP 25
# =====================================================

model_snapshot = (
    df_doc
    .groupby("device_model_exact_refined")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
    .head(25)
)

print("\n==============================")
print("FINAL MODEL SNAPSHOT — TOP 25")
display(model_snapshot)


# =====================================================
# 🔒 VALIDATION
# =====================================================

print("\n==============================")
print("VALIDATION")
print("Rows:", df_doc.shape[0])
print("Revenue:", round(df_doc["net_sales_num"].sum(), 2))

assert df_doc.shape[0] == 968
assert round(df_doc["net_sales_num"].sum(), 2) == 155265.60

PIPELINE_STAGE = "PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY"
print("\nPIPELINE STAGE:", PIPELINE_STAGE)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)



FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)


,bucket_final,transactions,revenue
9,Repair – Screen,604,106086.21
2,Repair – Back Glass,82,14598.14
3,Repair – Battery,85,10478.29
8,Repair – Hardware Install,11,7968.00
6,Repair – Charging Port,61,7205.00
0,Accessory Bundles,51,5872.00
15,Service – Other / Admin,34,4686.21
7,Repair – Generic (Explicit),21,3600.62
5,Repair – Camera / Lens,23,2930.00
11,Repair – Water Damage,11,2885.00



FINAL DEVICE TYPE SNAPSHOT


,transactions,revenue,avg_ticket
device_type_final,,,
phone,722,115118.98,159.444571
tablet,118,21736.62,184.208644
computer,41,11571.00,282.219512
unknown,28,2487.00,88.821429
accessory,21,1395.00,66.428571
console,6,1220.00,203.333333
smoke_vape,25,937.00,37.480000
watch,5,585.00,117.000000
tv,1,130.00,130.000000



FINAL BRAND SNAPSHOT


,transactions,revenue,avg_ticket
brand_final,,,
Apple,667,108841.11,163.180075
Samsung,158,30873.49,195.401835
Google,29,4875.00,168.103448
Unknown,36,3947.00,109.638889
Accessory,21,1395.00,66.428571
Microsoft,6,1230.00,205.000000
Motorola,9,1015.00,112.777778
Smoke / Vape,25,937.00,37.480000
Sony,4,900.00,225.000000



FINAL MODEL SNAPSHOT — TOP 25


,transactions,revenue,avg_ticket
device_model_exact_refined,,,
iPhone 14 Pro Max,52,9711.00,186.750000
iPhone 13,56,7937.61,141.743036
iPhone 15 Pro Max,40,7793.87,194.846750
Galaxy S23 Ultra,28,7514.00,268.357143
iPhone 11,64,6808.00,106.375000
iPhone 15,25,6740.00,269.600000
Galaxy S22 Ultra,21,5456.40,259.828571
iPad Pro,18,5150.00,286.111111
Unknown,53,4828.00,91.094340



VALIDATION
Rows: 968
Revenue: 155265.6

PIPELINE STAGE: PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY


snapshot

In [289]:
# =====================================================
# ✅ PARAGON GEEKS — FINAL PRESENTATION SNAPSHOTS
# (NO COMPLEXITY • NO MULTI-REPAIR BUCKET)
# =====================================================

import pandas as pd

# -----------------------------
# 1️⃣ FINAL REPAIR / SERVICE SNAPSHOT
# -----------------------------

status_df = show_status(df_doc)

# Buckets to REMOVE from presentation
drop_buckets = [
    "Unknown / Undocumented",
    "Defective / Adjustment",
    "Retail – Accessories / Smoke",
    "Multi-Repair Jobs"   # 🚫 REMOVED as requested
]

status_df = status_df[~status_df["bucket"].isin(drop_buckets)].copy()

# Merge small service/admin buckets
service_merge_map = {
    "Service – Service Call": "Service – Other / Admin",
    "Service – Generic": "Service – Other / Admin",
    "Service – Fee": "Service – Other / Admin",
}

status_df["bucket_final"] = status_df["bucket"].replace(service_merge_map)

final_repair_service = (
    status_df
    .groupby("bucket_final", as_index=False)
    .agg(
        transactions=("transactions", "sum"),
        revenue=("revenue", "sum")
    )
    .sort_values("revenue", ascending=False)
    .reset_index(drop=True)
)

print("\n==============================")
print("FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)")
display(final_repair_service)


# -----------------------------
# 2️⃣ FINAL DEVICE TYPE SNAPSHOT
# -----------------------------

device_snapshot = (
    df_doc
    .groupby("device_type_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

print("\n==============================")
print("FINAL DEVICE TYPE SNAPSHOT")
display(device_snapshot)


# -----------------------------
# 3️⃣ FINAL BRAND SNAPSHOT
# -----------------------------

brand_snapshot = (
    df_doc
    .groupby("brand_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)

print("\n==============================")
print("FINAL BRAND SNAPSHOT")
display(brand_snapshot)


# -----------------------------
# 4️⃣ FINAL MODEL SNAPSHOT (TOP 25)
# -----------------------------

model_snapshot = (
    df_doc
    .groupby("device_model_exact_refined")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
    .head(25)
)

print("\n==============================")
print("FINAL MODEL SNAPSHOT — TOP 25")
display(model_snapshot)


# -----------------------------
# 5️⃣ ACCESSORY & RETAIL KPI SNAPSHOT
# -----------------------------

accessory_kpis = pd.DataFrame([
    {
        "metric": "Accessory Only (Standalone)",
        "transactions": df_doc[df_doc["device_type_final"] == "accessory"]["Transaction ID"].count(),
        "revenue": df_doc[df_doc["device_type_final"] == "accessory"]["net_sales_num"].sum()
    },
    {
        "metric": "Accessory Bundles (With Repair)",
        "transactions": final_repair_service.loc[
            final_repair_service["bucket_final"] == "Accessory Bundles",
            "transactions"
        ].values[0],
        "revenue": final_repair_service.loc[
            final_repair_service["bucket_final"] == "Accessory Bundles",
            "revenue"
        ].values[0]
    },
    {
        "metric": "Smoke / Vape (Pure Retail)",
        "transactions": df_doc[df_doc["device_type_final"] == "smoke_vape"]["Transaction ID"].count(),
        "revenue": df_doc[df_doc["device_type_final"] == "smoke_vape"]["net_sales_num"].sum()
    }
])

print("\n==============================")
print("ACCESSORY & RETAIL KPI SNAPSHOT")
display(accessory_kpis)


# -----------------------------
# 6️⃣ FINAL VALIDATION
# -----------------------------

print("\n==============================")
print("VALIDATION")
print("Rows:", len(df_doc))
print("Revenue:", round(df_doc["net_sales_num"].sum(), 2))

PIPELINE_STAGE = "PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY"
print("\nPIPELINE STAGE:", PIPELINE_STAGE)



FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3255665783.py:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["event_retail_accessory_or_smoke"].fillna(False).astype(bool)


,bucket_final,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Repair – Hardware Install,11,7968.00
4,Repair – Charging Port,61,7205.00
5,Accessory Bundles,51,5872.00
6,Service – Other / Admin,34,4686.21
7,Repair – Generic (Explicit),21,3600.62
8,Repair – Camera / Lens,23,2930.00
9,Repair – Water Damage,11,2885.00



FINAL DEVICE TYPE SNAPSHOT


,transactions,revenue,avg_ticket
device_type_final,,,
phone,722,115118.98,159.444571
tablet,118,21736.62,184.208644
computer,41,11571.00,282.219512
unknown,28,2487.00,88.821429
accessory,21,1395.00,66.428571
console,6,1220.00,203.333333
smoke_vape,25,937.00,37.480000
watch,5,585.00,117.000000
tv,1,130.00,130.000000


...


FINAL MODEL SNAPSHOT — TOP 25


,transactions,revenue,avg_ticket
device_model_exact_refined,,,
iPhone 14 Pro Max,52,9711.00,186.750000
iPhone 13,56,7937.61,141.743036
iPhone 15 Pro Max,40,7793.87,194.846750
Galaxy S23 Ultra,28,7514.00,268.357143
iPhone 11,64,6808.00,106.375000
iPhone 15,25,6740.00,269.600000
Galaxy S22 Ultra,21,5456.40,259.828571
iPad Pro,18,5150.00,286.111111
Unknown,53,4828.00,91.094340



ACCESSORY & RETAIL KPI SNAPSHOT


,metric,transactions,revenue
0,Accessory Only (Standalone),21,1395.0
1,Accessory Bundles (With Repair),51,5872.0
2,Smoke / Vape (Pure Retail),25,937.0



VALIDATION
Rows: 968
Revenue: 155265.6

PIPELINE STAGE: PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY


In [290]:
PIPELINE_STAGE = "PRESENTATION_BASELINE_LOCKED_NO_COMPLEXITY"


In [291]:
print("=== RAW LOCATION VALUES (TOP 30) ===")
display(
    df_doc["Location"]
    .astype(str)
    .value_counts()
    .head(30)
)


=== RAW LOCATION VALUES (TOP 30) ===


Location
Paragon Geeks    615
Northfield       353
Name: count, dtype: int64

end of makr down

In [292]:
# ------------------------------------
# RECREATE UNDOCUMENTED DATAFRAME
# ------------------------------------

df_undoc = df[df["documentation_flag"] == "undocumented"].copy()

print("Undocumented rows:", len(df_undoc))
print("Undocumented revenue:", round(df_undoc["net_sales_num"].sum(), 2))


Undocumented rows: 954
Undocumented revenue: 107653.5


In [293]:
display(
    df_undoc
    .groupby("net_sales_num")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .sort_values("transactions", ascending=False)
    .head(150)
)


,transactions,revenue
net_sales_num,,
100.00,86,8600.00
120.00,82,9840.00
150.00,55,8250.00
130.00,55,7150.00
20.00,41,820.00
50.00,37,1850.00
80.00,34,2720.00
140.00,30,4200.00
30.00,30,900.00


In [294]:
df["documentation_flag"] = df["text_clean"].apply(
    lambda x: "undocumented" if pd.isna(x) or x.strip() == "" else "documented"
)
# ------------------------------------
# PRICE SIGN# ------------------------------------
# PRICE SIGN
# ------------------------------------

df_undoc["price_sign"] = df_undoc["net_sales_num"].apply(
    lambda x: "refund_or_adjustment" if x <= 0 else "positive"
)

df_undoc["price_sign"].value_counts()

# ------------------------------------

df_undoc["price_sign"] = df_undoc["net_sales_num"].apply(
    lambda x: "refund_or_adjustment" if x <= 0 else "positive"
)

df_undoc["price_sign"].value_counts()


price_sign
positive                945
refund_or_adjustment      9
Name: count, dtype: int64

In [295]:
price_bins = [
    -float("inf"), 0,
    25, 50, 75, 100,
    125, 150, 175, 200,
    250, 300, 400, 500,
    float("inf")
]

price_labels = [
    "Refund / Adjustment",
    "$0–25",
    "$25–50",
    "$50–75",
    "$75–100",
    "$100–125",
    "$125–150",
    "$150–175",
    "$175–200",
    "$200–250",
    "$250–300",
    "$300–400",
    "$400–500",
    "$500+"
]

df_undoc["price_tier"] = pd.cut(
    df_undoc["net_sales_num"],
    bins=price_bins,
    labels=price_labels,
    right=False
)


In [296]:
tier_snapshot = (
    df_undoc
    .groupby("price_tier")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("transactions", ascending=False)
)

display(tier_snapshot)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\2996325111.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("price_tier")


,transactions,revenue,avg_ticket
price_tier,,,
$100–125,217,23848.02,109.898710
$125–150,110,14710.00,133.727273
$0–25,108,1886.98,17.472037
$75–100,105,8829.71,84.092476
$150–175,101,15811.13,156.545842
$50–75,76,4327.00,56.934211
$25–50,75,2391.25,31.883333
$200–250,74,15835.00,213.986486
$175–200,32,5872.19,183.505937


In [297]:
comparison = (
    pd.concat([
        df_doc.assign(group="Documented"),
        df_undoc.assign(group="Undocumented")
    ])
    .groupby(["group", "price_tier"])
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
)

display(comparison)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\3101421633.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["group", "price_tier"])


transactions   revenue
group        price_tier                                 
Documented   Refund / Adjustment             0      0.00
             $0–25                           0      0.00
             $25–50                          0      0.00
             $50–75                          0      0.00
             $75–100                         0      0.00
             $100–125                        0      0.00
             $125–150                        0      0.00
             $150–175                        0      0.00
             $175–200                        0      0.00
             $200–250                        0      0.00
             $250–300                        0      0.00
             $300–400                        0      0.00
             $400–500                        0      0.00
             $500+                           0      0.00
Undocumented Refund / Adjustment             6   -875.00
             $0–25                         108   1886.98
             $25–50                         75   2391.25
             $50–75                         76   4327.00
             $75–100                       105   8829.71
             $100–125                      217  23848.02
             $125–150                      110  14710.00
             $150–175                      101  15811.13
             $175–200                       32   5872.19
             $200–250                       74  15835.00
             $250–300                       28   7480.00
             $300–400                       21   7037.22
             $400–500                        0      0.00
             $500+                           1    500.00

In [298]:
price_bins = [
    -float("inf"), 0,
    25, 50, 75, 100,
    125, 150, 175, 200,
    250, 300, 400, 500,
    float("inf")
]

price_labels = [
    "Refund / Adjustment",
    "$0–25",
    "$25–50",
    "$50–75",
    "$75–100",
    "$100–125",
    "$125–150",
    "$150–175",
    "$175–200",
    "$200–250",
    "$250–300",
    "$300–400",
    "$400–500",
    "$500+"
]

# Apply to BOTH
df_undoc["price_tier"] = pd.cut(
    df_undoc["net_sales_num"],
    bins=price_bins,
    labels=price_labels,
    right=False
)

df_doc["price_tier"] = pd.cut(
    df_doc["net_sales_num"],
    bins=price_bins,
    labels=price_labels,
    right=False
)


In [299]:
comparison = (
    pd.concat([
        df_doc.assign(group="Documented"),
        df_undoc.assign(group="Undocumented")
    ])
    .groupby(["group", "price_tier"], observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
)

display(comparison)


transactions   revenue
group        price_tier                                 
Documented   Refund / Adjustment             2   -405.00
             $0–25                          30    491.01
             $25–50                         37   1230.00
             $50–75                         60   3506.29
             $75–100                        82   6777.72
             $100–125                      239  26563.62
             $125–150                      120  16023.69
             $150–175                      114  17800.40
             $175–200                       29   5326.27
             $200–250                      122  25960.00
             $250–300                       43  11413.00
             $300–400                       63  20822.60
             $400–500                       13   5711.00
             $500+                          14  14045.00
Undocumented Refund / Adjustment             6   -875.00
             $0–25                         108   1886.98
             $25–50                         75   2391.25
             $50–75                         76   4327.00
             $75–100                       105   8829.71
             $100–125                      217  23848.02
             $125–150                      110  14710.00
             $150–175                      101  15811.13
             $175–200                       32   5872.19
             $200–250                       74  15835.00
             $250–300                       28   7480.00
             $300–400                       21   7037.22
             $400–500                        0      0.00
             $500+                           1    500.00

In [300]:
tier_name_map = {
    "Refund / Adjustment": "Refunds & Corrections",
    "$0–25": "Low-Volume Accessories / One-Offs",
    "$25–50": "Diagnostics & Minor Services",
    "$50–75": "Entry-Level Repair",
    "$75–100": "Standard Repair (Low)",
    "$100–125": "Standard Repair",
    "$125–150": "Standard Repair (High)",
    "$150–175": "Advanced Repair",
    "$175–200": "Advanced Repair (High)",
    "$200–250": "Premium Repair",
    "$250–300": "High-End Repair",
    "$300–400": "Computer / Complex Repair",
    "$400–500": "Major Repair",
    "$500+": "Exceptional / Outlier Jobs"
}

df_doc["price_tier_name"] = df_doc["price_tier"].map(tier_name_map)
df_undoc["price_tier_name"] = df_undoc["price_tier"].map(tier_name_map)


In [301]:
documented_tiers = (
    df_doc
    .groupby("price_tier", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .reset_index()
)

# Attach readable names AFTER aggregation
documented_tiers["price_tier_name"] = documented_tiers["price_tier"].map(tier_name_map)

# Order nicely
documented_tiers = documented_tiers[
    ["price_tier", "price_tier_name", "transactions", "revenue", "avg_ticket"]
]

print("\nDOCUMENTED PRICE TIERS (CLEAN)")
display(documented_tiers)



DOCUMENTED PRICE TIERS (CLEAN)


,price_tier,price_tier_name,transactions,revenue,avg_ticket
0,Refund / Adjustment,Refunds & Corrections,2,-405.00,-202.500000
1,$0–25,Low-Volume Accessories / One-Offs,30,491.01,16.367000
2,$25–50,Diagnostics & Minor Services,37,1230.00,33.243243
3,$50–75,Entry-Level Repair,60,3506.29,58.438167
4,$75–100,Standard Repair (Low),82,6777.72,82.655122
5,$100–125,Standard Repair,239,26563.62,111.144854
6,$125–150,Standard Repair (High),120,16023.69,133.530750
7,$150–175,Advanced Repair,114,17800.40,156.143860
8,$175–200,Advanced Repair (High),29,5326.27,183.664483
9,$200–250,Premium Repair,122,25960.00,212.786885


In [302]:
undocumented_tiers = (
    df_undoc
    .groupby("price_tier", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .reset_index()
)

undocumented_tiers["price_tier_name"] = undocumented_tiers["price_tier"].map(tier_name_map)

undocumented_tiers = undocumented_tiers[
    ["price_tier", "price_tier_name", "transactions", "revenue", "avg_ticket"]
]

print("\nUNDOCUMENTED PRICE TIERS (CLEAN)")
display(undocumented_tiers)



UNDOCUMENTED PRICE TIERS (CLEAN)


,price_tier,price_tier_name,transactions,revenue,avg_ticket
0,Refund / Adjustment,Refunds & Corrections,6,-875.00,-145.833333
1,$0–25,Low-Volume Accessories / One-Offs,108,1886.98,17.472037
2,$25–50,Diagnostics & Minor Services,75,2391.25,31.883333
3,$50–75,Entry-Level Repair,76,4327.00,56.934211
4,$75–100,Standard Repair (Low),105,8829.71,84.092476
5,$100–125,Standard Repair,217,23848.02,109.898710
6,$125–150,Standard Repair (High),110,14710.00,133.727273
7,$150–175,Advanced Repair,101,15811.13,156.545842
8,$175–200,Advanced Repair (High),32,5872.19,183.505937
9,$200–250,Premium Repair,74,15835.00,213.986486


In [303]:
merged_price_tiers = (
    pd.concat([
        df_doc.assign(group="Documented"),
        df_undoc.assign(group="Undocumented")
    ])
    .groupby(["group", "price_tier"], observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .reset_index()
)

merged_price_tiers["price_tier_name"] = merged_price_tiers["price_tier"].map(tier_name_map)

merged_price_tiers = merged_price_tiers[
    ["group", "price_tier", "price_tier_name", "transactions", "revenue"]
]

print("\nDOCUMENTED vs UNDOCUMENTED PRICE TIERS (CLEAN)")
display(merged_price_tiers)



DOCUMENTED vs UNDOCUMENTED PRICE TIERS (CLEAN)


,group,price_tier,price_tier_name,transactions,revenue
0,Documented,Refund / Adjustment,Refunds & Corrections,2,-405.00
1,Documented,$0–25,Low-Volume Accessories / One-Offs,30,491.01
2,Documented,$25–50,Diagnostics & Minor Services,37,1230.00
3,Documented,$50–75,Entry-Level Repair,60,3506.29
4,Documented,$75–100,Standard Repair (Low),82,6777.72
5,Documented,$100–125,Standard Repair,239,26563.62
6,Documented,$125–150,Standard Repair (High),120,16023.69
7,Documented,$150–175,Advanced Repair,114,17800.40
8,Documented,$175–200,Advanced Repair (High),29,5326.27
9,Documented,$200–250,Premium Repair,122,25960.00


In [304]:
# ------------------------------------
# COMPANY-LEVEL PRICE TIER × DOCUMENTATION
# ------------------------------------

store_price_doc = (
    pd.concat([
        df_doc.assign(group="Documented"),
        df_undoc.assign(group="Undocumented")
    ])
    .assign(company="Paragon Geeks")
    .groupby(
        ["company", "group", "price_tier"],
        observed=False
    )
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .reset_index()
)

# Attach readable tier names
store_price_doc["price_tier_name"] = store_price_doc["price_tier"].map(tier_name_map)

# Final column order
store_price_doc = store_price_doc[
    ["company", "group", "price_tier", "price_tier_name", "transactions", "revenue"]
]

print("\nPARAGON GEEKS — PRICE TIER × DOCUMENTATION")
display(store_price_doc)



PARAGON GEEKS — PRICE TIER × DOCUMENTATION


,company,group,price_tier,price_tier_name,transactions,revenue
0,Paragon Geeks,Documented,Refund / Adjustment,Refunds & Corrections,2,-405.00
1,Paragon Geeks,Documented,$0–25,Low-Volume Accessories / One-Offs,30,491.01
2,Paragon Geeks,Documented,$25–50,Diagnostics & Minor Services,37,1230.00
3,Paragon Geeks,Documented,$50–75,Entry-Level Repair,60,3506.29
4,Paragon Geeks,Documented,$75–100,Standard Repair (Low),82,6777.72
5,Paragon Geeks,Documented,$100–125,Standard Repair,239,26563.62
6,Paragon Geeks,Documented,$125–150,Standard Repair (High),120,16023.69
7,Paragon Geeks,Documented,$150–175,Advanced Repair,114,17800.40
8,Paragon Geeks,Documented,$175–200,Advanced Repair (High),29,5326.27
9,Paragon Geeks,Documented,$200–250,Premium Repair,122,25960.00


In [305]:
# ==============================
# GRAND TOTALS
# ==============================

grand_totals = (
    pd.concat([
        df_doc.assign(group="Documented"),
        df_undoc.assign(group="Undocumented")
    ])
    .groupby("group")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
)

# Add combined total
combined_total = pd.DataFrame({
    "transactions": [grand_totals["transactions"].sum()],
    "revenue": [grand_totals["revenue"].sum()],
    "avg_ticket": [
        grand_totals["revenue"].sum() / grand_totals["transactions"].sum()
    ]
}, index=["Combined"])

grand_totals = pd.concat([grand_totals, combined_total])

print("\nPARAGON GEEKS — GRAND TOTALS")
display(grand_totals)



PARAGON GEEKS — GRAND TOTALS


,transactions,revenue,avg_ticket
Documented,968,155265.6,160.398347
Undocumented,954,107653.5,112.844340
Combined,1922,262919.1,136.794537


In [306]:
# ==============================
# PRICE TIER TOTALS — COMBINED
# ==============================

price_tier_totals = (
    pd.concat([df_doc, df_undoc])
    .groupby("price_tier", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .reset_index()
)

price_tier_totals["price_tier_name"] = price_tier_totals["price_tier"].map(tier_name_map)

price_tier_totals = price_tier_totals[
    ["price_tier", "price_tier_name", "transactions", "revenue", "avg_ticket"]
]

print("\nPARAGON GEEKS — PRICE TIER TOTALS (COMBINED)")
display(price_tier_totals)



PARAGON GEEKS — PRICE TIER TOTALS (COMBINED)


,price_tier,price_tier_name,transactions,revenue,avg_ticket
0,Refund / Adjustment,Refunds & Corrections,8,-1280.00,-160.000000
1,$0–25,Low-Volume Accessories / One-Offs,138,2377.99,17.231812
2,$25–50,Diagnostics & Minor Services,112,3621.25,32.332589
3,$50–75,Entry-Level Repair,136,7833.29,57.597721
4,$75–100,Standard Repair (Low),187,15607.43,83.462193
5,$100–125,Standard Repair,456,50411.64,110.551842
6,$125–150,Standard Repair (High),230,30733.69,133.624739
7,$150–175,Advanced Repair,215,33611.53,156.332698
8,$175–200,Advanced Repair (High),61,11198.46,183.581311
9,$200–250,Premium Repair,196,41795.00,213.239796


In [307]:
# ==============================
# DOCUMENTATION SHARE BY PRICE TIER
# ==============================

doc_share = (
    store_price_doc
    .pivot(
        index=["price_tier", "price_tier_name"],
        columns="group",
        values=["transactions", "revenue"]
    )
)

# Flatten columns
doc_share.columns = [
    f"{metric}_{group}".lower()
    for metric, group in doc_share.columns
]

# Fill missing
doc_share = doc_share.fillna(0)

# Compute shares
doc_share["transaction_doc_pct"] = (
    doc_share["transactions_documented"] /
    (doc_share["transactions_documented"] + doc_share["transactions_undocumented"])
)

doc_share["revenue_doc_pct"] = (
    doc_share["revenue_documented"] /
    (doc_share["revenue_documented"] + doc_share["revenue_undocumented"])
)

print("\nPARAGON GEEKS — DOCUMENTATION SHARE BY PRICE TIER")
display(doc_share.reset_index())



PARAGON GEEKS — DOCUMENTATION SHARE BY PRICE TIER


,price_tier,price_tier_name,transactions_documented,transactions_undocumented,revenue_documented,revenue_undocumented,transaction_doc_pct,revenue_doc_pct
0,Refund / Adjustment,Refunds & Corrections,2.0,6.0,-405.00,-875.00,0.250000,0.316406
1,$0–25,Low-Volume Accessories / One-Offs,30.0,108.0,491.01,1886.98,0.217391,0.206481
2,$25–50,Diagnostics & Minor Services,37.0,75.0,1230.00,2391.25,0.330357,0.339662
3,$50–75,Entry-Level Repair,60.0,76.0,3506.29,4327.00,0.441176,0.447614
4,$75–100,Standard Repair (Low),82.0,105.0,6777.72,8829.71,0.438503,0.434262
5,$100–125,Standard Repair,239.0,217.0,26563.62,23848.02,0.524123,0.526934
6,$125–150,Standard Repair (High),120.0,110.0,16023.69,14710.00,0.521739,0.521372
7,$150–175,Advanced Repair,114.0,101.0,17800.40,15811.13,0.530233,0.529592
8,$175–200,Advanced Repair (High),29.0,32.0,5326.27,5872.19,0.475410,0.475625
9,$200–250,Premium Repair,122.0,74.0,25960.00,15835.00,0.622449,0.621127


In [308]:
PIPELINE_STAGE = "PRICE_TIER_AND_DOCUMENTATION_ANALYSIS_COMPLETE"
print("PIPELINE STAGE:", PIPELINE_STAGE)


PIPELINE STAGE: PRICE_TIER_AND_DOCUMENTATION_ANALYSIS_COMPLETE


In [309]:
# ==============================
# TIME INTELLIGENCE — HOUR OF DAY
# ==============================

df_all = pd.concat([
    df_doc.assign(group="Documented"),
    df_undoc.assign(group="Undocumented")
])

df_all["hour"] = pd.to_datetime(df_all["Time"], errors="coerce").dt.hour

hourly_perf = (
    df_all
    .groupby(["hour", "group"])
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .reset_index()
    .sort_values("hour")
)

display(hourly_perf)


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\1907136567.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_all["hour"] = pd.to_datetime(df_all["Time"], errors="coerce").dt.hour


,hour,group,transactions,revenue
0,0,Documented,4,385.00
1,0,Undocumented,3,330.00
2,1,Documented,2,270.00
3,1,Undocumented,2,175.00
4,2,Documented,1,140.00
5,6,Documented,3,470.00
6,6,Undocumented,3,330.00
7,7,Documented,5,635.00
8,7,Undocumented,2,210.00
10,8,Undocumented,15,1593.25


In [310]:
def hour_to_12h(h):
    if pd.isna(h):
        return "Unknown"
    h = int(h)
    if h == 0:
        return "12 AM"
    if h < 12:
        return f"{h} AM"
    if h == 12:
        return "12 PM"
    return f"{h - 12} PM"


In [311]:
print(df.columns)


Index(['Date', 'Time', 'Time Zone', 'Category', 'Item', 'Qty',
       'Price Point Name', 'SKU', 'Modifiers Applied', 'Gross Sales',
       'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Payment ID',
       'Device Name', 'Notes', 'Details', 'Event Type', 'Location',
       'Dining Option', 'Customer ID', 'Customer Name',
       'Customer Reference ID', 'Unit', 'Count', 'GTIN', 'Itemization Type',
       'Commission', 'Employee', 'Fulfillment Note', 'Channel', 'Token',
       'Card Brand', 'PAN Suffix', 'source_year', 'Service Charges',
       'Gift Card Sales', 'Tip', 'Partial Refunds', 'Total Collected',
       'Source', 'Card', 'Card Entry Methods', 'Cash', 'Square Gift Card',
       'Other Tender', 'Other Tender Type', 'Tender Note', 'Fees', 'Net Total',
       'Staff Name', 'Staff ID', 'Description', 'Device Nickname',
       'Third Party Fees', 'Deposit ID', 'Deposit Date', 'Deposit Details',
       'Fee Percentage Rate', 'Fee Fixed Rate', 'Refund Reason',
       'Discount N

In [312]:
# -----------------------------
# CREATE HOUR COLUMN
# -----------------------------

df["hour"] = pd.to_datetime(
    df["Time"],
    errors="coerce"
).dt.hour


C:\Users\kevin\AppData\Local\Temp\ipykernel_14540\860660719.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["hour"] = pd.to_datetime(


In [313]:
# -----------------------------
# HOURLY PERFORMANCE — TOTALS
# -----------------------------

hourly_perf = (
    df
    .groupby("hour", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .reset_index()
    .sort_values("hour")
)

def hour_to_12h(hour):
    if hour == 0:
        return "12 AM"
    elif hour < 12:
        return f"{hour} AM"
    elif hour == 12:
        return "12 PM"
    else:
        return f"{hour - 12} PM"

hourly_perf["hour_12h"] = hourly_perf["hour"].apply(hour_to_12h)

hourly_perf = hourly_perf[
    ["hour", "hour_12h", "transactions", "revenue"]
]

print("\nPARAGON GEEKS — HOURLY PERFORMANCE (12-HOUR FORMAT)")
display(hourly_perf)



PARAGON GEEKS — HOURLY PERFORMANCE (12-HOUR FORMAT)


,hour,hour_12h,transactions,revenue
0,0,12 AM,7,715.00
1,1,1 AM,4,445.00
2,2,2 AM,1,140.00
3,6,6 AM,6,800.00
4,7,7 AM,7,845.00
5,8,8 AM,29,5047.12
6,9,9 AM,40,5124.22
7,10,10 AM,107,13328.72
8,11,11 AM,177,22500.10
9,12,12 PM,247,34331.48


In [314]:
# -----------------------------
# CREATE DAY OF WEEK COLUMN
# -----------------------------

df["day_of_week"] = pd.to_datetime(
    df["Date"],
    errors="coerce"
).dt.day_name()


In [315]:
# -----------------------------
# DAY OF WEEK PERFORMANCE
# -----------------------------

day_order = [
    "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday", "Saturday", "Sunday"
]

dow_perf = (
    df
    .groupby("day_of_week", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum")
    )
    .reindex(day_order)
    .reset_index()
)

print("\nPARAGON GEEKS — DAY OF WEEK PERFORMANCE")
display(dow_perf)



PARAGON GEEKS — DAY OF WEEK PERFORMANCE


,day_of_week,transactions,revenue
0,Monday,96,14709.79
1,Tuesday,85,15160.68
2,Wednesday,97,15382.63
3,Thursday,94,12123.75
4,Friday,94,13795.00
5,Saturday,84,15366.34
6,Sunday,104,14639.47


In [316]:
# -----------------------------
# CREATE date_final SAFELY
# -----------------------------
df["date_final"] = pd.to_datetime(df["Date"], errors="coerce")


In [317]:
df["month"] = pd.to_datetime(df["date_final"]).dt.to_period("M")


In [318]:
df["month"].isna().sum()


1268

In [319]:
monthly_perf = (
    df
    .groupby("month", observed=True)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .reset_index()
    .sort_values("month")
)

display(monthly_perf)


,month,transactions,revenue,avg_ticket
0,2024-01,50,7530.72,150.614400
1,2024-02,30,5260.00,175.333333
2,2024-03,40,5348.29,133.707250
3,2024-04,24,6657.00,277.375000
4,2024-05,15,2297.84,153.189333
5,2024-06,54,7921.75,146.699074
6,2024-07,38,6614.38,174.062632
7,2024-08,58,8149.68,140.511724
8,2024-09,65,8415.00,129.461538
9,2024-10,74,11013.00,148.824324


In [320]:
monthly_perf["revenue"].sum().round(2)
df["net_sales_num"].sum().round(2)


262919.1

In [321]:
df.groupby("month").size().sort_index()


month
2024-01     50
2024-02     30
2024-03     40
2024-04     24
2024-05     15
2024-06     54
2024-07     38
2024-08     58
2024-09     65
2024-10     74
2024-11    113
2024-12     93
Freq: M, dtype: int64

In [322]:
raw_2024 = pd.read_csv(
    r"C:\Users\kevin\OneDrive\Desktop\raw data\square data\2024 sales.csv",
    parse_dates=["Date"]
)

raw_2025 = pd.read_csv(
    r"C:\Users\kevin\OneDrive\Desktop\raw data\square data\2025 sales.csv",
    parse_dates=["Date"]
)

raw_all = pd.concat([raw_2024, raw_2025], ignore_index=True)


In [323]:
raw_all["Date"].isna().value_counts()


Date
False    1922
Name: count, dtype: int64

In [324]:
raw_all["raw_month"] = raw_all["Date"].dt.to_period("M")


In [325]:
raw_all["raw_month"].isna().sum()


0

In [326]:
df["Date_parsed"] = pd.to_datetime(
    df["Date"],
    format="mixed",
    errors="coerce"
)




In [327]:
df["Date_parsed"].isna().value_counts()


Date_parsed
False    1922
Name: count, dtype: int64

In [328]:
df["final_month"] = df["Date_parsed"].dt.to_period("M")


In [329]:
df["final_month"].isna().sum()


0

In [330]:
df.groupby("final_month").size().sort_index()


final_month
2024-01     50
2024-02     30
2024-03     40
2024-04     24
2024-05     15
2024-06     54
2024-07     38
2024-08     58
2024-09     65
2024-10     74
2024-11    113
2024-12     93
2025-01     88
2025-02     51
2025-03     74
2025-04    123
2025-05    116
2025-06    138
2025-07    164
2025-08    156
2025-09    158
2025-10     56
2025-11     68
2025-12     76
Freq: M, dtype: int64

In [331]:
monthly_perf["revenue"].sum().round(2)
df["net_sales_num"].sum().round(2)


262919.1

In [332]:
monthly_perf_final = (
    df
    .groupby("final_month", observed=False)
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .reset_index()
    .rename(columns={"final_month": "month"})
    .sort_values("month")
)

monthly_perf_final["month_label"] = monthly_perf_final["month"].dt.strftime("%b %Y")

monthly_perf_final = monthly_perf_final[
    ["month", "month_label", "transactions", "revenue", "avg_ticket"]
]

print("\nPARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)")
display(monthly_perf_final)



PARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)


,month,month_label,transactions,revenue,avg_ticket
0,2024-01,Jan 2024,50,7530.72,150.614400
1,2024-02,Feb 2024,30,5260.00,175.333333
2,2024-03,Mar 2024,40,5348.29,133.707250
3,2024-04,Apr 2024,24,6657.00,277.375000
4,2024-05,May 2024,15,2297.84,153.189333
5,2024-06,Jun 2024,54,7921.75,146.699074
6,2024-07,Jul 2024,38,6614.38,174.062632
7,2024-08,Aug 2024,58,8149.68,140.511724
8,2024-09,Sep 2024,65,8415.00,129.461538
9,2024-10,Oct 2024,74,11013.00,148.824324


# ============================================================
# FINAL MONTHLY PERFORMANCE — LOCKED
# Source: Raw Square POS "Date" field only
# Validation:
# - No NaT months
# - Transactions reconcile to full dataset
# - Revenue matches transaction-level totals
# Do not modify date logic beyond this point
# ============================================================


In [333]:
# ============================================================
# FINAL PRESENTATION SNAPSHOT — PARAGON GEEKS
# (NO COMPLEXITY · NO RECOMPUTATION · LOCKED)
# ============================================================

PIPELINE_STAGE = "PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY"

print("\n==============================")
print("PIPELINE STAGE:", PIPELINE_STAGE)

print("\n==============================")
print("VALIDATION")
print("Rows (Documented):", len(df_doc))
print("Revenue (Documented):", round(df_doc["net_sales_num"].sum(), 2))

# -----------------------------
# PRICE TIER ANALYSIS
# -----------------------------

print("\n==============================")
print("PARAGON GEEKS — PRICE TIER × DOCUMENTATION")
display(store_price_doc)

print("\n==============================")
print("PARAGON GEEKS — PRICE TIER TOTALS (COMBINED)")
display(price_tier_totals)

print("\n==============================")
print("PARAGON GEEKS — DOCUMENTATION SHARE BY PRICE TIER")
display(doc_share.reset_index())

# -----------------------------
# TIME-BASED PERFORMANCE
# -----------------------------

print("\n==============================")
print("PARAGON GEEKS — HOURLY PERFORMANCE (12-HOUR FORMAT)")
display(hourly_perf)

print("\n==============================")
print("PARAGON GEEKS — DAY OF WEEK PERFORMANCE")
display(dow_perf)

print("\n==============================")
print("PARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)")
display(monthly_perf_final)

print("\n==============================")
print("END OF FINAL PRESENTATION SNAPSHOT")



PIPELINE STAGE: PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY

VALIDATION
Rows (Documented): 968
Revenue (Documented): 155265.6

PARAGON GEEKS — PRICE TIER × DOCUMENTATION


,company,group,price_tier,price_tier_name,transactions,revenue
0,Paragon Geeks,Documented,Refund / Adjustment,Refunds & Corrections,2,-405.00
1,Paragon Geeks,Documented,$0–25,Low-Volume Accessories / One-Offs,30,491.01
2,Paragon Geeks,Documented,$25–50,Diagnostics & Minor Services,37,1230.00
3,Paragon Geeks,Documented,$50–75,Entry-Level Repair,60,3506.29
4,Paragon Geeks,Documented,$75–100,Standard Repair (Low),82,6777.72
5,Paragon Geeks,Documented,$100–125,Standard Repair,239,26563.62
6,Paragon Geeks,Documented,$125–150,Standard Repair (High),120,16023.69
7,Paragon Geeks,Documented,$150–175,Advanced Repair,114,17800.40
8,Paragon Geeks,Documented,$175–200,Advanced Repair (High),29,5326.27
9,Paragon Geeks,Documented,$200–250,Premium Repair,122,25960.00



PARAGON GEEKS — PRICE TIER TOTALS (COMBINED)


,price_tier,price_tier_name,transactions,revenue,avg_ticket
0,Refund / Adjustment,Refunds & Corrections,8,-1280.00,-160.000000
1,$0–25,Low-Volume Accessories / One-Offs,138,2377.99,17.231812
2,$25–50,Diagnostics & Minor Services,112,3621.25,32.332589
3,$50–75,Entry-Level Repair,136,7833.29,57.597721
4,$75–100,Standard Repair (Low),187,15607.43,83.462193
5,$100–125,Standard Repair,456,50411.64,110.551842
6,$125–150,Standard Repair (High),230,30733.69,133.624739
7,$150–175,Advanced Repair,215,33611.53,156.332698
8,$175–200,Advanced Repair (High),61,11198.46,183.581311
9,$200–250,Premium Repair,196,41795.00,213.239796



PARAGON GEEKS — DOCUMENTATION SHARE BY PRICE TIER


...


PARAGON GEEKS — DAY OF WEEK PERFORMANCE


,day_of_week,transactions,revenue
0,Monday,96,14709.79
1,Tuesday,85,15160.68
2,Wednesday,97,15382.63
3,Thursday,94,12123.75
4,Friday,94,13795.00
5,Saturday,84,15366.34
6,Sunday,104,14639.47



PARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)


,month,month_label,transactions,revenue,avg_ticket
0,2024-01,Jan 2024,50,7530.72,150.614400
1,2024-02,Feb 2024,30,5260.00,175.333333
2,2024-03,Mar 2024,40,5348.29,133.707250
3,2024-04,Apr 2024,24,6657.00,277.375000
4,2024-05,May 2024,15,2297.84,153.189333
5,2024-06,Jun 2024,54,7921.75,146.699074
6,2024-07,Jul 2024,38,6614.38,174.062632
7,2024-08,Aug 2024,58,8149.68,140.511724
8,2024-09,Sep 2024,65,8415.00,129.461538
9,2024-10,Oct 2024,74,11013.00,148.824324



END OF FINAL PRESENTATION SNAPSHOT


finished

In [334]:
device_type_snapshot = (
    df_doc
    .groupby("device_type_final")
    .agg(
        transactions=("Transaction ID", "count"),
        revenue=("net_sales_num", "sum"),
        avg_ticket=("net_sales_num", "mean")
    )
    .sort_values("revenue", ascending=False)
)


last snapshot

In [335]:
# ============================================================
# FINAL PRESENTATION SNAPSHOT — PARAGON GEEKS
# (MASTER · AUDIT SAFE · NO RECOMPUTATION)
# ============================================================

PIPELINE_STAGE = "PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY"

# =====================================================
# PARAGON GEEKS — REVENUE DOCUMENTATION COVERAGE SUMMARY
# =====================================================

documented_revenue = df_doc["net_sales_num"].sum()
total_revenue = df["net_sales_num"].sum()
undocumented_revenue = total_revenue - documented_revenue

coverage_pct = documented_revenue / total_revenue

print("\n==============================")
print("REVENUE COVERAGE OVERVIEW")
print(f"Total Revenue (All Transactions): ${total_revenue:,.2f}")
print(f"Documented Revenue:               ${documented_revenue:,.2f}")
print(f"Undocumented Revenue:             ${undocumented_revenue:,.2f}")
print(f"Documentation Coverage:           {coverage_pct:.1%}")
print("==============================\n")


print("\n==============================")
print("PIPELINE STAGE:", PIPELINE_STAGE)

# -----------------------------
# VALIDATION (SOURCE OF TRUTH)
# -----------------------------

print("\n==============================")
print("VALIDATION")
print("Rows (Documented):", len(df_doc))
print("Revenue (Documented):", round(df_doc["net_sales_num"].sum(), 2))

# -----------------------------
# REPAIR / SERVICE SNAPSHOT
# -----------------------------

print("\n==============================")
print("FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)")
display(final_repair_service)

# -----------------------------
# DEVICE TYPE SNAPSHOT
# -----------------------------

print("\n==============================")
print("FINAL DEVICE TYPE SNAPSHOT")
display(device_type_snapshot)

# -----------------------------
# BRAND SNAPSHOT
# -----------------------------

print("\n==============================")
print("FINAL BRAND SNAPSHOT")
display(brand_snapshot)

# -----------------------------
# MODEL SNAPSHOT (TOP 25)
# -----------------------------

print("\n==============================")
print("FINAL MODEL SNAPSHOT — TOP 25")
display(model_snapshot)

# -----------------------------
# ACCESSORY & RETAIL KPIs
# -----------------------------

print("\n==============================")
print("ACCESSORY & RETAIL KPI SNAPSHOT")
display(accessory_kpis)

# -----------------------------
# PRICE TIER ANALYSIS
# -----------------------------

print("\n==============================")
print("PARAGON GEEKS — PRICE TIER × DOCUMENTATION")
display(store_price_doc)

print("\n==============================")
print("PARAGON GEEKS — PRICE TIER TOTALS (COMBINED)")
display(price_tier_totals)

print("\n==============================")
print("PARAGON GEEKS — DOCUMENTATION SHARE BY PRICE TIER")
display(doc_share.reset_index())

# -----------------------------
# TIME-BASED PERFORMANCE
# -----------------------------

print("\n==============================")
print("PARAGON GEEKS — HOURLY PERFORMANCE (12-HOUR FORMAT)")
display(hourly_perf)

print("\n==============================")
print("PARAGON GEEKS — DAY OF WEEK PERFORMANCE")
display(dow_perf)

print("\n==============================")
print("PARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)")
display(monthly_perf_final)

print("\n==============================")
print("END OF FINAL PRESENTATION SNAPSHOT")



REVENUE COVERAGE OVERVIEW
Total Revenue (All Transactions): $262,919.10
Documented Revenue:               $155,265.60
Undocumented Revenue:             $107,653.50
Documentation Coverage:           59.1%


PIPELINE STAGE: PARAGON_GEEKS_PRESENTATION_SNAPSHOT_FINAL_LOCKED_NO_COMPLEXITY

VALIDATION
Rows (Documented): 968
Revenue (Documented): 155265.6

FINAL REPAIR / SERVICE SNAPSHOT (PRESENTATION)


,bucket_final,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Repair – Hardware Install,11,7968.00
4,Repair – Charging Port,61,7205.00
5,Accessory Bundles,51,5872.00
6,Service – Other / Admin,34,4686.21
7,Repair – Generic (Explicit),21,3600.62
8,Repair – Camera / Lens,23,2930.00
9,Repair – Water Damage,11,2885.00



FINAL DEVICE TYPE SNAPSHOT


,transactions,revenue,avg_ticket
device_type_final,,,
phone,722,115118.98,159.444571
tablet,118,21736.62,184.208644
computer,41,11571.00,282.219512
unknown,28,2487.00,88.821429
accessory,21,1395.00,66.428571
console,6,1220.00,203.333333
smoke_vape,25,937.00,37.480000
watch,5,585.00,117.000000
tv,1,130.00,130.000000



FINAL BRAND SNAPSHOT


...


PARAGON GEEKS — DAY OF WEEK PERFORMANCE


,day_of_week,transactions,revenue
0,Monday,96,14709.79
1,Tuesday,85,15160.68
2,Wednesday,97,15382.63
3,Thursday,94,12123.75
4,Friday,94,13795.00
5,Saturday,84,15366.34
6,Sunday,104,14639.47



PARAGON GEEKS — MONTHLY PERFORMANCE (FINAL · AUDIT SAFE)


,month,month_label,transactions,revenue,avg_ticket
0,2024-01,Jan 2024,50,7530.72,150.614400
1,2024-02,Feb 2024,30,5260.00,175.333333
2,2024-03,Mar 2024,40,5348.29,133.707250
3,2024-04,Apr 2024,24,6657.00,277.375000
4,2024-05,May 2024,15,2297.84,153.189333
5,2024-06,Jun 2024,54,7921.75,146.699074
6,2024-07,Jul 2024,38,6614.38,174.062632
7,2024-08,Aug 2024,58,8149.68,140.511724
8,2024-09,Sep 2024,65,8415.00,129.461538
9,2024-10,Oct 2024,74,11013.00,148.824324



END OF FINAL PRESENTATION SNAPSHOT


In [337]:
import os

EXPORT_DIR = r"C:\Users\kevin\OneDrive\Desktop\powerbiflagshipproject"
os.makedirs(EXPORT_DIR, exist_ok=True)

exports = {
    "fact_repair_service.csv": final_repair_service.reset_index(drop=True),
    "dim_device_type.csv": device_type_snapshot.reset_index(),
    "dim_brand.csv": brand_snapshot.reset_index(),
    "dim_model_top25.csv": model_snapshot.reset_index(),
    "kpi_accessory.csv": accessory_kpis,
    "price_tier_totals.csv": price_tier_totals,
    "documentation_share_by_price_tier.csv": doc_share.reset_index(),
    "hourly_performance.csv": hourly_perf,
    "dow_performance.csv": dow_perf,
    "monthly_performance.csv": monthly_perf_final,
    
}

for name, df_out in exports.items():
    df_out.to_csv(os.path.join(EXPORT_DIR, name), index=False)

print("✅ POWER BI EXPORT COMPLETE — LOCKED SNAPSHOT ONLY")
for k in exports:
    print(" -", k)


✅ POWER BI EXPORT COMPLETE — LOCKED SNAPSHOT ONLY
 - fact_repair_service.csv
 - dim_device_type.csv
 - dim_brand.csv
 - dim_model_top25.csv
 - kpi_accessory.csv
 - price_tier_totals.csv
 - documentation_share_by_price_tier.csv
 - hourly_performance.csv
 - dow_performance.csv
 - monthly_performance.csv


In [339]:
print("df_doc columns:")
print(sorted(df_doc.columns.tolist()))


df_doc columns:
['Card', 'Card Brand', 'Card Entry Methods', 'Cash', 'Cash App', 'Category', 'Channel', 'Commission', 'Count', 'Customer ID', 'Customer Name', 'Customer Reference ID', 'Date', 'Deposit Date', 'Deposit Details', 'Deposit ID', 'Description', 'Details', 'Device Name', 'Device Nickname', 'Dining Option', 'Discount Name', 'Discounts', 'Employee', 'Event Type', 'Fee Fixed Rate', 'Fee Percentage Rate', 'Fees', 'Free Processing Applied', 'Fulfillment Note', 'GTIN', 'Gift Card Sales', 'Gross Sales', 'Item', 'Itemization Type', 'Location', 'Modifiers Applied', 'Net Sales', 'Net Total', 'Notes', 'Order Reference ID', 'Other Tender', 'Other Tender Type', 'PAN Suffix', 'Partial Refunds', 'Payment ID', 'Price Point Name', 'Qty', 'Refund Reason', 'SKU', 'Service Charges', 'Source', 'Square Gift Card', 'Staff ID', 'Staff Name', 'Table Info', 'Tax', 'Tender Note', 'Third Party Fees', 'Time', 'Time Zone', 'Tip', 'Token', 'Total Collected', 'Transaction ID', 'Transaction Status', 'Unattri

In [343]:
import os
import pandas as pd

EXPORT_DIR = r"C:\Users\kevin\OneDrive\Desktop\powerbiflagshipproject"
os.makedirs(EXPORT_DIR, exist_ok=True)

# ============================================================
# ✅ EXPORT LOCKED REPAIR/SERVICE SNAPSHOT (SOURCE OF TRUTH)
# Uses your already-correct final_repair_service table
# ============================================================

# Make a clean copy
frs = final_repair_service.copy()

# Normalize column names just in case
frs.columns = [c.strip().lower().replace(" ", "_") for c in frs.columns]

# Expect these fields from your printed snapshot
# bucket_final | transactions | revenue
required = {"bucket_final", "transactions", "revenue"}
missing = required - set(frs.columns)
if missing:
    raise ValueError(f"final_repair_service missing columns: {missing}. Columns found: {list(frs.columns)}")

# Final clean + sort
fact_repair_service_locked = (
    frs[["bucket_final", "transactions", "revenue"]]
    .copy()
    .sort_values("revenue", ascending=False)
)

# SANITY CHECK
print("\n==============================")
print("✅ LOCKED REPAIR/SERVICE EXPORT CHECK")
print("==============================")
print("Rows:", len(fact_repair_service_locked))
print("Revenue:", round(fact_repair_service_locked["revenue"].sum(), 2))
display(fact_repair_service_locked)

# EXPORT
out_path = os.path.join(EXPORT_DIR, "fact_repair_service_LOCKED.csv")
fact_repair_service_locked.to_csv(out_path, index=False)
print("✅ Exported:", out_path)



✅ LOCKED REPAIR/SERVICE EXPORT CHECK
Rows: 16
Revenue: 179089.47


,bucket_final,transactions,revenue
0,Repair – Screen,604,106086.21
1,Repair – Back Glass,82,14598.14
2,Repair – Battery,85,10478.29
3,Repair – Hardware Install,11,7968.00
4,Repair – Charging Port,61,7205.00
5,Accessory Bundles,51,5872.00
6,Service – Other / Admin,34,4686.21
7,Repair – Generic (Explicit),21,3600.62
8,Repair – Camera / Lens,23,2930.00
9,Repair – Water Damage,11,2885.00


✅ Exported: C:\Users\kevin\OneDrive\Desktop\powerbiflagshipproject\fact_repair_service_LOCKED.csv


In [344]:
# List candidate dataframes in memory
[c for c in globals().keys() if c.startswith("df")]


['df_2024',
 'df_2025',
 'df_raw',
 'df',
 'df_doc',
 'df_unknown',
 'df_unknown_2',
 'df_unknown_final',
 'df_unknown_brand',
 'df_undoc',
 'df_all',
 'df_out',
 'df_final_transactions']

In [345]:
for name in ["df_all", "df_final", "df_combined", "df_full", "df_powerbi", "df_clean"]:
    if name in globals():
        print(name, "rows:", len(globals()[name]))


df_all rows: 1922


In [346]:
# 🔍 List all dataframe-like variables currently in memory
[c for c in globals().keys() if c.startswith("df")]


['df_2024',
 'df_2025',
 'df_raw',
 'df',
 'df_doc',
 'df_unknown',
 'df_unknown_2',
 'df_unknown_final',
 'df_unknown_brand',
 'df_undoc',
 'df_all',
 'df_out',
 'df_final_transactions']

In [351]:
from pathlib import Path

# --------------------------------------------------
# 🔒 EXPLICIT PROJECT PATH (WINDOWS-SAFE)
# --------------------------------------------------
PROCESSED_DIR = Path(
    r"C:\Users\kevin\OneDrive\Desktop\paragon-geeks-sales-analytics\data\processed"
)

# Ensure folder exists
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Writing processed files to:")
print(PROCESSED_DIR)

# --------------------------------------------------
# EXPORT FINAL, LOCKED DATASETS (NO RECOMPUTE)
# --------------------------------------------------
exports = {
    "fact_repair_service.csv": final_repair_service.reset_index(drop=True),
    "dim_device_type.csv": device_type_snapshot.reset_index(),
    "dim_brand.csv": brand_snapshot.reset_index(),
    "dim_model_top25.csv": model_snapshot.reset_index(),
    "kpi_accessory.csv": accessory_kpis,
    "price_tier_totals.csv": price_tier_totals,
    "documentation_share_by_price_tier.csv": doc_share.reset_index(),
    "hourly_performance.csv": hourly_perf,
    "dow_performance.csv": dow_perf,
    "monthly_performance.csv": monthly_perf_final,
}

for name, df_out in exports.items():
    df_out.to_csv(PROCESSED_DIR / name, index=False)
    print("✅ Saved:", name)

print("\n✅ ALL PROCESSED FILES EXPORTED SUCCESSFULLY")


📁 Writing processed files to:
C:\Users\kevin\OneDrive\Desktop\paragon-geeks-sales-analytics\data\processed
✅ Saved: fact_repair_service.csv
✅ Saved: dim_device_type.csv
✅ Saved: dim_brand.csv
✅ Saved: dim_model_top25.csv
✅ Saved: kpi_accessory.csv
✅ Saved: price_tier_totals.csv
✅ Saved: documentation_share_by_price_tier.csv
✅ Saved: hourly_performance.csv
✅ Saved: dow_performance.csv
✅ Saved: monthly_performance.csv

✅ ALL PROCESSED FILES EXPORTED SUCCESSFULLY


# ✅ NOTEBOOK STATUS: FINAL — LOCKED

This notebook represents the final, audit-safe analytics pipeline
for the Paragon Geeks Power BI Flagship Project.

- All validation checks passed
- All exports completed successfully
- Power BI visuals built from exported CSVs
- No further transformations required


